In [306]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [795]:
import functools
import re

import numpy as np
import json
import pandas as pd
import openpyxl
from string_grouper import match_most_similar

import auxiliar_2

# Encuesta

In [308]:
encuesta = pd.read_csv(r'datos/results-survey272167.csv')
encuesta.head(3)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_18284/2011331124.py:1: DtypeWarning: Columns (14,17,27,30,34,36,40,41,42,49,56,58,59,73,77,79,85,124,136,140,152,154,158,160,164,168,169,175,183,193,194,202,203,214,215,216,225,226,228,242,246,258,264,272,279,288,314,316,318,330,332,370,372,374,380,382,388,390,394,395,402,413,414,415,423,434,435,443,458,462,487,493,505,507,515,519,521,525,534,550,556,562,568,574,594,599,601,604,608,615,621,624,627,637,654,655,656,665,674,686,690,696,702,708,714,726,728,731,736,740,742,746,755,771,773,774,783,789,813,818,821,827,829,831,835,838,842,848,849,860,866,870,873,874,879,886,888,916,946,948,950,956,958,960,962,969,977,984,993,1003,1012,1017,1024,1029,1034,1042,1045,1049,1051,1053,1057,1062,1080,1081,1082,1083,1087,1089,1097,1098,1099,1100,1101,1107,1109,1110,1111,1114,1115,1116,1119,1129,1132,1135,1147,1161,1172,1174,1176,1179,1186,1190,1196,1197,1202,1218,1222,1232,1241,1248,1253,1259,1270,1276,1280,1293,1295,1297,1309,1310,1312,1320,1

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,encuestador,...,S2LUD,S2LUD[other],C0NSTRUCC10N,C0NSTRUCC10N[other],FABRUIK,FABRUIK[other],AGRICULTUR3,AGRICULTUR3[other],AbuelaPatTrabajoApoy,AbuelaPatTrabajoOfic
0,7,2021-03-29 19:22:58,3,es-MX,125280212,2021-03-29 19:22:40,2021-03-29 19:22:58,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,2021-03-30 12:45:52,3,es-MX,817932738,2021-03-30 12:30:25,2021-03-30 12:45:52,2021-03-30 00:00:00,5300.0,A22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17,2021-03-30 14:46:16,3,es-MX,740600100,2021-03-30 14:02:49,2021-03-30 14:46:16,2021-03-30 00:00:00,5108.0,A03,...,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Diccionario de datos

Este diccionario contiene toda la información al respecto de las preguntas hechas en la encuesta de LimeSurvey, además de las descripciones a las preguntas, entre otras cosas

In [309]:
diccionario = pd.read_csv(r'datos/ls_dictionary_272167_434_419_435.csv')
diccionario

,code,question,options
0,opcagricultura,NaN,A2 = Agricultor|A3 = Granjero|A4 = Siembra
1,opcagricultura_other,: Otra opción,NaN
2,farbrig,NaN,A2 = Empleado
3,farbrig_other,: Otra opción,NaN
4,opcconstruccion,NaN,A2 = Arquitecto|A3 = Ingeniero civil/albañil o...
...,...,...,...
1771,q0r846,¿A qué edad falleció?,NaN
1772,AbueloPatTrabajoOfic,NaN,A1 = Carpintero|A2 = Obrero|A3 = Herrero|A4 = ...
1773,AbueloPatTrabajoElem,NaN,A1 = Trabajadores domésticos|A2 = Trabajadores...
1774,AbuelaPatTrabajoOfic,NaN,A1 = Carpintero|A2 = Obrero|A3 = Herrero|A4 = ...


In [310]:
traductor = pd.read_excel(r'datos/Diccionario Zoom 2 parentescos 11.04.2022 RCR.xlsx')
traductor

,item order,related,alive,new_code,code,original item lime survey,question,options,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1,G,_,fecha_encuesta,fecha,Fecha de realización,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,G,_,folio,folio,Folio,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,G,_,encuestador,encuestador,Encuestador,NaN,A02 = Shellye,A03 = Jorge,A04 = María del Carmen,...,A15 = Cristian,A16 = Georgina,A17 = Brenda,A18 = Erika,A19 = Mariana,A20 = Cynthia,A21 = Edgar Octavio,A22 = Paulina,A23 = Mayra Adriana,NaN
3,4,G,_,encuestador_otro,encuestador_other,Encuestador: Otra opción,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,G,_,edo_civil,EdoCivil,Estado Civil,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,1335,AoP,_,trab_const_otro,C0NSTRUCC10N_other,: Otra opción,"Tu abuelo paterno, ¿a qué se dedica/dedicaba p...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1335,1336,AoP,_,trab_fabrica,FABRUIK,NaN,"Tu abuelo paterno, ¿a qué se dedica/dedicaba p...",A2 = Empleado_1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1336,1337,AoP,_,trab_fab_otro,FABRUIK_other,: Otra opción,"Tu abuelo paterno, ¿a qué se dedica/dedicaba p...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1337,1338,AoP,_,trab_agricultura,AGRICULTUR3,NaN,"Tu abuelo paterno, ¿a qué se dedica/dedicaba p...",A2 = Agricultor_1,A3 = Granjero_2,A4 = Siembra_3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
archivo_json = 'Trabajos_familia.json'

# Leer el archivo JSON y convertirlo en un diccionario
with open(archivo_json, 'r', encoding='utf-8') as f:
    mapeo_reorganizado_importado = json.load(f)

In [312]:
mapeo_reorganizado_importado

{'actividad religiosa': 'A9901',
 'belleza': 'A9902',
 'cacique de rancho': 'A9903',
 'capataz': 'A9904',
 'carbón': 'A9905',
 'club': 'A9906',
 'coaching': 'A9907',
 'desconoce': 'A9908',
 'desconocido': 'A9908',
 'se desconoce': 'A9908',
 'distribuidor': 'A9909',
 'editorial': 'A9910',
 'em': 'A9911',
 'estacion de radio': 'A9912',
 'evaciones psicologicas': 'A9913',
 'facturas': 'A9914',
 'guantes': 'A9915',
 'hace trofeos y medallas por mayoreo': 'A9916',
 'homeopatia': 'A9917',
 'jubilado': 'A9918',
 'lo que hacía': 'A9919',
 'menos infractores e inadaptados': 'A9920',
 'ninguna': 'A9921',
 'no aplica': 'A9922',
 'no hay información': 'A9923',
 'no lo conoció': 'A9924',
 'no recuerdo': 'A9925',
 'no sabe': 'A9925',
 'no se': 'A9925',
 'no se sabe': 'A9925',
 'no sé': 'A9925',
 'omitir': 'A9926',
 'orador': 'A9927',
 'pastor (religion)': 'A9928',
 'pensionada': 'A9929',
 'pensionado': 'A9929',
 'pensionado de pgr (mecanico de helicopteros)': 'A9929',
 'por contrato': 'A9930',
 'pre

# Mapeo de enfermedades

Primero, de la encuesta, vamos a obtener todas las columnas que estén dentro de ese dataset, para luego poder filtrar y quedarnos con todas aquellas que contengan [other], y no pertenezcan a las ya mapeadas en el .json de edades

In [313]:
columnas = encuesta.columns.tolist()

In [314]:
columnas_filtradas = [col for col in columnas if '[other]' in col]
columnas_filtradas

['encuestador[other]',
 'edad31[other]',
 'ag3[other]',
 'eje3[other]',
 'ejeEdad[other]',
 'cigarro11[other]',
 'cigarroedad4[other]',
 'q0r95[other]',
 'q8620[other]',
 'q51888[other]',
 'q5199[other]',
 'q2666[other]',
 'q2444[other]',
 'q22222[other]',
 'q22221[other]',
 'edadalimentacion[other]',
 'queestudi[other]',
 'enfermedades[other]',
 'escolaridadmama[other]',
 'Trabajo[other]',
 'opcionesdeportivas[other]',
 'opcartista[other]',
 'opcchofer[other]',
 'opccomerciaFormal[other]',
 'comerciFromal[other]',
 'opcempresario[other]',
 'opccomerciovirtual[other]',
 'opcatencionaclientes[other]',
 'opcAdminis[other]',
 'opcGubernamental[other]',
 'opceducativ[other]',
 'opciondependiente[other]',
 'opcseguridad[other]',
 'opcionareadesalud[other]',
 'opcconstruccion[other]',
 'farbrig[other]',
 'opcagricultura[other]',
 'q0r764[other]',
 'q0r699[other]',
 'q0r85[other]',
 'q0r944[other]',
 'q0r29[other]',
 'q0r1009[other]',
 'q0r467[other]',
 'q0r209[other]',
 'enf470[other]',
 'q0

## Enfermedades


In [315]:
encuesta_enfermedades = ['q5199[other]', 'enfermedades[other]',
                         'enf470[other]', 'q0r629[other]', 'enfermedades28[other]]',
                         'q0r577[other]', 'q802[other]', 'q0r404[other]',
                         'enfr8[other]', 'q0r876[other]']
columnas_filtradas_enfermedades = [col for col in columnas_filtradas if col in encuesta_enfermedades]
columnas_filtradas_enfermedades

['q5199[other]',
 'enfermedades[other]',
 'enf470[other]',
 'q0r629[other]',
 'q0r577[other]',
 'q802[other]',
 'q0r404[other]',
 'enfr8[other]',
 'q0r876[other]']

In [316]:
def obtener_respuestas_por_folio(df, columnas):
    respuestas_por_folio = df.set_index('folio')[columnas].T.to_dict()
    return respuestas_por_folio

respuestas_por_folio = obtener_respuestas_por_folio(encuesta, columnas_filtradas_enfermedades)
respuestas_por_folio

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_18284/253374503.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  respuestas_por_folio = df.set_index('folio')[columnas].T.to_dict()


{nan: {'q5199[other]': nan,
  'enfermedades[other]': nan,
  'enf470[other]': nan,
  'q0r629[other]': nan,
  'q0r577[other]': nan,
  'q802[other]': nan,
  'q0r404[other]': nan,
  'enfr8[other]': nan,
  'q0r876[other]': nan},
 5300.0: {'q5199[other]': nan,
  'enfermedades[other]': nan,
  'enf470[other]': nan,
  'q0r629[other]': nan,
  'q0r577[other]': nan,
  'q802[other]': nan,
  'q0r404[other]': nan,
  'enfr8[other]': nan,
  'q0r876[other]': nan},
 5108.0: {'q5199[other]': nan,
  'enfermedades[other]': 'artritis reumatoide, posible lupus ',
  'enf470[other]': nan,
  'q0r629[other]': nan,
  'q0r577[other]': nan,
  'q802[other]': nan,
  'q0r404[other]': nan,
  'enfr8[other]': nan,
  'q0r876[other]': nan},
 5156.0: {'q5199[other]': nan,
  'enfermedades[other]': nan,
  'enf470[other]': nan,
  'q0r629[other]': nan,
  'q0r577[other]': nan,
  'q802[other]': nan,
  'q0r404[other]': nan,
  'enfr8[other]': nan,
  'q0r876[other]': nan},
 5301.0: {'q5199[other]': nan,
  'enfermedades[other]': nan,


In [317]:
def cargar_json(ruta):
    with open(ruta, 'r') as archivo:
        datos = json.load(archivo)
    return datos

def obtener_respuestas_validas(respuestas_por_folio):
    respuestas_validas = {folio: {k: v for k, v in respuestas.items() if pd.notna(v)} for folio, respuestas in respuestas_por_folio.items() if any(pd.notna(v) for v in respuestas.values())}
    return respuestas_validas

def mapear_enfermedades(respuestas_validas, enfermedades_dict):
    for folio, respuestas in respuestas_validas.items():
        for k, v in respuestas.items():
            for enfermedad, sinónimos in enfermedades_dict.items():
                if v.lower() in (sinónimo.lower() for sinónimo in sinónimos):
                    respuestas_validas[folio][k] = enfermedad
                elif v in sinónimos:
                    respuestas_validas[folio][k] = enfermedad
    return respuestas_validas


In [318]:
enfermedades_dict = cargar_json('datos/enfermedades.json')
respuestas_validas = obtener_respuestas_validas(respuestas_por_folio)
respuestas_validas_sin_nan = mapear_enfermedades(respuestas_validas, enfermedades_dict)
print(respuestas_validas)

{5108.0: {'enfermedades[other]': 'artritis reumatoide, posible lupus '}, 5299.0: {'enfermedades[other]': 'Enfermedad de hormonas'}, 5027.0: {'enfermedades[other]': 'Enfermedad del hígado'}, 5220.0: {'q0r876[other]': 'Enfermedad de hormonas'}, 5303.0: {'q5199[other]': 'Enfermedad de hormonas'}, 5155.0: {'enfermedades[other]': 'HIPOTIROIDISMO Y PRESION BAJA ', 'q0r577[other]': 'Enfermedad de hormonas', 'q0r404[other]': 'PROBLEMAS DE RIÑON'}, 5076.0: {'enfr8[other]': 'Enfermedad de los ojos'}, 5286.0: {'q0r577[other]': 'DEMENCIA SENIL ', 'q802[other]': 'ERC '}, 5268.0: {'enfermedades[other]': 'Enfermedad de hormonas', 'q0r876[other]': 'Enfermedad de nervios y cerebro'}, 5320.0: {'enfr8[other]': 'PROBLEMAS RENALES'}, 5013.0: {'enfermedades[other]': 'MIGRAÑA ESTOMACAL', 'q0r629[other]': 'Enfermedad de los oídos'}, 5067.0: {'enfr8[other]': 'Enfermedad de los riñones'}, 5288.0: {'enfr8[other]': 'ARTRITIS '}, 5323.0: {'enfermedades[other]': 'SOP', 'q0r629[other]': 'HIPERURICEMIA', 'q0r404[othe

## Estudios

In [319]:
encuesta_estudios = ['queestudi[other]','estudios[other]', 'q51888[other]']

columnas_filtradas_estudios = [col for col in columnas_filtradas if col in encuesta_estudios]
columnas_filtradas_estudios

['q51888[other]', 'queestudi[other]']

In [320]:
# Obtener los valores de las columnas filtradas
valores_columnas = encuesta[columnas_filtradas_estudios].values.flatten().tolist()

# Eliminar valores nulos o vacíos
valores_columnas = [valor for valor in valores_columnas if pd.notnull(valor) and valor != '']

# Convertir los valores a minúsculas
valores_columnas = [valor.lower() for valor in valores_columnas]

# Obtener valores únicosg
estudios_unicos = list(set(valores_columnas))

print("Lista original de enfermedades:")
print(estudios_unicos)

Lista original de enfermedades:
['desintometria', 'glucosa', 'densitometria osea', 'densitometria', 'alergias', 'orina', 'perfiles hormonales', 'ego, ultrasonidos, perfiles hormonales y tiroideos, entre otros', 'hipertiroidismo', 'radiografía rodilla', 'trombotest, densitometria', 'tiroides', 'hipotiroidismo', 'perfil tiroideo ', 'hipotiroidismo ', 'tomografia', 'piedras en el riñon', 'radiografias ', 'dental ,dermatologico y gastroenterologico', 'exploracion de colonoscopia ', 'ultrasonido ', 'perfil tiroideo', 'perfil hormonal ', 'electroencefalograma', 'radiografias rotulas', 'ultrasonido', 'hormonales', 'osteporeosis']


In [321]:
estudios_dict = cargar_json('datos/estudios.json')
respuestas_por_folio = obtener_respuestas_por_folio(encuesta, columnas_filtradas_estudios)

/var/folders/hb/c3zh8bns0mx4nw98_6k7rx8m0000gn/T/ipykernel_18284/253374503.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  respuestas_por_folio = df.set_index('folio')[columnas].T.to_dict()


In [322]:
def filtrar_respuestas_no_nulas(respuestas_por_folio):
    return {folio: respuestas for folio, respuestas in respuestas_por_folio.items() if not all(pd.isnull(v) for v in respuestas.values())}

respuestas_por_folio_sin_nan = filtrar_respuestas_no_nulas(respuestas_por_folio)

In [323]:
respuestas_validas = obtener_respuestas_validas(respuestas_por_folio_sin_nan)

In [324]:
respuestas_limpias = {}
for folio, respuestas in respuestas_validas.items():
    for key, valor in respuestas.items():
        if isinstance(valor, list):
            valor = [v.lower() for v in valor]
        else:
            valor = valor.lower()
            respuestas_limpias[folio] = respuestas
respuestas_validas = respuestas_limpias
print(respuestas_validas)

{5076.0: {'queestudi[other]': 'hormonales'}, 5013.0: {'queestudi[other]': 'DENTAL ,DERMATOLOGICO Y GASTROENTEROLOGICO'}, 5323.0: {'queestudi[other]': 'DENSITOMETRIA OSEA'}, 5002.0: {'queestudi[other]': 'OSTEPOREOSIS'}, 5160.0: {'queestudi[other]': 'TIROIDES'}, 5159.0: {'queestudi[other]': 'PERFIL HORMONAL '}, 5129.0: {'queestudi[other]': 'TIROIDES'}, 5008.0: {'queestudi[other]': 'HIPERTIROIDISMO'}, 5170.0: {'queestudi[other]': 'HIPOTIROIDISMO'}, 5032.0: {'queestudi[other]': 'piedras en el riñon'}, 5248.0: {'queestudi[other]': 'TIROIDES'}, 5360.0: {'queestudi[other]': 'EXPLORACION DE COLONOSCOPIA '}, 5365.0: {'queestudi[other]': 'ALERGIAS'}, 5387.0: {'queestudi[other]': 'PERFIL TIROIDEO'}, 5397.0: {'queestudi[other]': 'ULTRASONIDO '}, 7171.0: {'queestudi[other]': 'perfil tiroideo'}, 7266.0: {'q51888[other]': 'EGO, ULTRASONIDOS, PERFILES HORMONALES Y TIROIDEOS, ENTRE OTROS'}, 6512.0: {'queestudi[other]': 'TOMOGRAFIA'}, 6887.0: {'queestudi[other]': 'ELECTROENCEFALOGRAMA'}, 7554.0: {'quees

In [325]:
mapeo_respuestas = mapear_enfermedades(respuestas_validas, estudios_dict)
print(mapeo_respuestas)

{5076.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5013.0: {'queestudi[other]': 'Revisiones dentales'}, 5323.0: {'queestudi[other]': 'Estudios especiales de gabinete'}, 5002.0: {'queestudi[other]': 'Estudios especiales de gabinete'}, 5160.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5159.0: {'queestudi[other]': 'PERFIL HORMONAL '}, 5129.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5008.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5170.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5032.0: {'queestudi[other]': 'Estudios especiales de gabinete'}, 5248.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5360.0: {'queestudi[other]': 'EXPLORACION DE COLONOSCOPIA '}, 5365.0: {'queestudi[other]': 'Estudio especial de sangre'}, 5387.0: {'queestudi[other]': 'Estudios de bioquímica sanguínea (EsB)'}, 5397.0: {'queestudi[other]': 'ULTRASONIDO '}, 7171.0: {'queestudi[other]': 

# Mapeo de edades

In [326]:
# Cargando el archivo JSON
with open('output/edades.json', 'r') as file:
    age_ranges_dict = json.load(file)

In [327]:
# Identificar las columnas [other] que se relacionan con las edades en la encuesta
age_related_other_columns = [col for col in encuesta.columns if '[other]' in col and col.split('[')[0] in age_ranges_dict]

In [328]:
# Verificando los primeros elementos de age_ranges_dict y las primeras columnas de survey_df
age_ranges_dict[:5], list(encuesta.columns)[:20]

([{'q22221': [{'min': 0, 'max': 20, 'var': 'SQ004'},
    {'min': 21, 'max': 40, 'var': 'SQ005'},
    {'min': 41, 'max': 60, 'var': 'SQ006'},
    {'min': 61, 'max': 150, 'var': 'SQ007'}]},
  {'edadalimentacion': [{'min': 0, 'max': 20, 'var': 'SQ002'},
    {'min': 21, 'max': 40, 'var': 'SQ003'},
    {'min': 41, 'max': 60, 'var': 'SQ004'},
    {'min': 61, 'max': 150, 'var': 'SQ005'}]},
  {'q22222': [{'min': 0, 'max': 20, 'var': 'SQ004'},
    {'min': 21, 'max': 40, 'var': 'SQ005'},
    {'min': 41, 'max': 60, 'var': 'SQ006'},
    {'min': 61, 'max': 150, 'var': 'SQ007'}]},
  {'q2444': [{'min': 0, 'max': 20, 'var': 'SQ002'},
    {'min': 21, 'max': 40, 'var': 'SQ003'},
    {'min': 41, 'max': 60, 'var': 'SQ004'},
    {'min': 61, 'max': 150, 'var': 'SQ005'}]},
  {'q2888': [{'min': 0, 'max': 20, 'var': 'SQ004'},
    {'min': 21, 'max': 40, 'var': 'SQ005'},
    {'min': 41, 'max': 60, 'var': 'SQ006'},
    {'min': 61, 'max': 150, 'var': 'SQ007'}]}],
 ['id',
  'submitdate',
  'lastpage',
  'startlangu

In [329]:
# Ajustando age_ranges_dict para ser un diccionario plano en lugar de una lista de diccionarios
flat_age_ranges_dict = {}
for entry in age_ranges_dict:
    flat_age_ranges_dict.update(entry)

# Verificando la estructura del diccionario ajustado
dict(list(flat_age_ranges_dict.items())[:5])

{'q22221': [{'min': 0, 'max': 20, 'var': 'SQ004'},
  {'min': 21, 'max': 40, 'var': 'SQ005'},
  {'min': 41, 'max': 60, 'var': 'SQ006'},
  {'min': 61, 'max': 150, 'var': 'SQ007'}],
 'edadalimentacion': [{'min': 0, 'max': 20, 'var': 'SQ002'},
  {'min': 21, 'max': 40, 'var': 'SQ003'},
  {'min': 41, 'max': 60, 'var': 'SQ004'},
  {'min': 61, 'max': 150, 'var': 'SQ005'}],
 'q22222': [{'min': 0, 'max': 20, 'var': 'SQ004'},
  {'min': 21, 'max': 40, 'var': 'SQ005'},
  {'min': 41, 'max': 60, 'var': 'SQ006'},
  {'min': 61, 'max': 150, 'var': 'SQ007'}],
 'q2444': [{'min': 0, 'max': 20, 'var': 'SQ002'},
  {'min': 21, 'max': 40, 'var': 'SQ003'},
  {'min': 41, 'max': 60, 'var': 'SQ004'},
  {'min': 61, 'max': 150, 'var': 'SQ005'}],
 'q2888': [{'min': 0, 'max': 20, 'var': 'SQ004'},
  {'min': 21, 'max': 40, 'var': 'SQ005'},
  {'min': 41, 'max': 60, 'var': 'SQ006'},
  {'min': 61, 'max': 150, 'var': 'SQ007'}]}

In [330]:
# Función para verificar si un valor es numérico
def is_numeric(val):
    if isinstance(val, (int, float)):
        return True
    if isinstance(val, str) and val.replace('.', '', 1).isdigit():
        return True
    return False

In [331]:
# Eliminar columnas [other] que no tienen datos y corregir las respuestas no numéricas
columns_not_found = []  # Lista para guardar columnas que no se encuentran
for col in age_related_other_columns:
    try:
        # Si todos los valores en la columna son NaN, eliminar la columna
        if encuesta[col].isna().all():
            encuesta.drop(columns=[col], inplace=True)
        else:
            # Reemplazar valores no numéricos con 99
            encuesta[col] = encuesta[col].apply(lambda x: x if is_numeric(x) else 99)
    except KeyError:
        columns_not_found.append(col)

columns_not_found

[]

In [332]:
def extract_min_max_from_string(s):
    """
    Extracts minimum and maximum values from a string using regex.
    """
    numbers = re.findall(r'(\d+\.*\d*)', s)
    if len(numbers) == 1:
        return float(numbers[0]), float(numbers[0])
    elif len(numbers) > 1:
        return float(numbers[0]), float(numbers[1])
    return None, None

In [333]:
def regex_map_age_to_correct_column(row):
    for question_key, age_ranges in flat_age_ranges_dict.items():
        other_column = f"{question_key}[other]"
        
        # If the [other] column is not present or its value is NaN, continue with the next iteration
        if other_column not in row or pd.isna(row[other_column]):
            continue

        age_value_str = str(row[other_column])
        age_min, age_max = extract_min_max_from_string(age_value_str)

        # If no valid age values are found, continue with the next iteration
        if age_min is None or age_max is None:
            continue

        # Map the age range to the correct columns
        for age_range in age_ranges:
            defined_age_min, defined_age_max = age_range['min'], age_range['max']
            var = age_range['var']
            
            if (defined_age_min <= age_min <= defined_age_max) or (defined_age_min <= age_max <= defined_age_max):
                correct_column = f"{question_key}[{var}]"
                row[correct_column] = "Y"

    return row

In [334]:
# Apply the regex-based function to the DataFrame
regex_corrected_survey_df = encuesta.apply(regex_map_age_to_correct_column, axis=1)

In [335]:
# Mostrando las primeras filas del DataFrame corregido
regex_corrected_survey_df.head()

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,encuestador,...,S2LUD,S2LUD[other],C0NSTRUCC10N,C0NSTRUCC10N[other],FABRUIK,FABRUIK[other],AGRICULTUR3,AGRICULTUR3[other],AbuelaPatTrabajoApoy,AbuelaPatTrabajoOfic
0,7,2021-03-29 19:22:58,3,es-MX,125280212,2021-03-29 19:22:40,2021-03-29 19:22:58,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,2021-03-30 12:45:52,3,es-MX,817932738,2021-03-30 12:30:25,2021-03-30 12:45:52,2021-03-30 00:00:00,5300.0,A22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17,2021-03-30 14:46:16,3,es-MX,740600100,2021-03-30 14:02:49,2021-03-30 14:46:16,2021-03-30 00:00:00,5108.0,A03,...,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,2021-03-30 14:58:34,3,es-MX,175910133,2021-03-30 14:32:46,2021-03-30 14:58:34,2021-03-30 00:00:00,5156.0,A15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,2021-03-30 17:56:46,3,es-MX,1422063480,2021-03-30 17:34:06,2021-03-30 17:56:46,2021-03-30 00:00:00,5301.0,A05,...,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN


In [336]:
#regex_corrected_survey_df.to_csv(r"C:\Users\Elvira Magallanes\Documents\Servicio Social\encuestas\Zoom2\output\encuesta_limpia.csv", index=False)

# Mapeo de trabajos

Filtraremos todas las columnas bases 

In [337]:
columnas_trabajo = traductor[traductor['new_code'].str.contains('trabajo', case=False, na=False)]
# Crear un diccionario que mapee las columnas "code" y "new_code" para estas columnas específicas relacionadas con el trabajo
diccionario_trabajo = dict(zip(columnas_trabajo['code'], columnas_trabajo['new_code']))
# Mostrar algunos elementos del diccionario para verificar el mapeo
list(diccionario_trabajo.items())[:10]

[('Trabajo', 'trabajo'),
 ('Trabajo_other', 'trabajo_otro'),
 ('q0r977', 'trabajo'),
 ('q0r977_other', 'trabajo_otro'),
 ('q0r651', 'trabajo'),
 ('q0r651_other', 'trabajo_otro'),
 ('q0r207', 'trabajo'),
 ('q0r207_other', 'trabajo_otro'),
 ('q0r926', 'trabajo'),
 ('q0r926_other', 'trabajo_otro')]

In [338]:
# Modificar las claves del diccionario de trabajo para reflejar la diferencia en la nomenclatura
diccionario_trabajo_modificado = {key.replace('_other', '[other]'): value for key, value in diccionario_trabajo.items()}

# Seleccionar solo las columnas que existen en la encuesta y en el diccionario de trabajo modificado
columnas_existentes_modificadas = [col for col in diccionario_trabajo_modificado.keys() if col in encuesta.columns]
columnas_existentes_modificadas

['Trabajo',
 'Trabajo[other]',
 'q0r977',
 'q0r977[other]',
 'q0r651',
 'q0r651[other]',
 'q0r207',
 'q0r207[other]',
 'q0r926',
 'q0r926[other]',
 'q0r800',
 'q0r800[other]']

In [339]:
# Crear un DataFrame para almacenar las respuestas relacionadas con el trabajo
respuestas_trabajo_df = encuesta[columnas_existentes_modificadas]

In [340]:
# Renombrar las columnas usando el diccionario de trabajo modificado
respuestas_trabajo_df.columns = respuestas_trabajo_df.columns.map(lambda col: diccionario_trabajo_modificado.get(col, col))

# Limpiar las respuestas, eliminando los espacios en blanco y convirtiendo a minúsculas
respuestas_trabajo_df = respuestas_trabajo_df.apply(lambda x: x.str.strip().str.lower())

# Llenar los valores vacíos con 'ninguna'
respuestas_trabajo_df = respuestas_trabajo_df.fillna('ninguna')

In [341]:
respuestas_trabajo_df

,trabajo,trabajo_otro,trabajo,trabajo_otro,trabajo,trabajo_otro,trabajo,trabajo_otro,trabajo,trabajo_otro,trabajo,trabajo_otro
0,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna,ninguna
1,11,ninguna,16,ninguna,a25,ninguna,16,ninguna,25,ninguna,17,ninguna
2,11,ninguna,7,ninguna,a10,ninguna,16,ninguna,-oth-,biologa,20,ninguna
3,21,ninguna,10,ninguna,a25,ninguna,17,ninguna,6,ninguna,-oth-,carpintero
4,21,ninguna,4,ninguna,a6,ninguna,21,ninguna,25,ninguna,23,ninguna
...,...,...,...,...,...,...,...,...,...,...,...,...
699,21,ninguna,20,ninguna,-oth-,empleada de hogar,17,ninguna,25,ninguna,3,ninguna
700,21,ninguna,6,ninguna,a25,ninguna,19,ninguna,5,ninguna,5,ninguna
701,11,ninguna,19,ninguna,a23,ninguna,20,ninguna,25,ninguna,22,ninguna
702,8,ninguna,12,ninguna,-oth-,hogar,17,ninguna,-oth-,hogar,4,ninguna


In [342]:
# Almacenar los valores únicos de cada columna en una lista de listas
all_valores_unicos = [respuestas_trabajo_df.iloc[:, idx].astype(str).unique() for idx in range(respuestas_trabajo_df.shape[1])]

# Imprimir los valores únicos
for idx, columna in enumerate(respuestas_trabajo_df.columns):
    print(f"Los valores únicos en la columna '{columna}' (índice {idx}) son: {all_valores_unicos[idx]}")

Los valores únicos en la columna 'trabajo' (índice 0) son: ['ninguna' '11' '21' '5' '3' '2' '6' '7' '12' '-oth-' '14' '20' '4' '15'
 '18' '16' '8' '10' '9' '13']
Los valores únicos en la columna 'trabajo_otro' (índice 1) son: ['ninguna' 'em' 'ama de casa' 'hogar' 'estudiando' 'coaching'
 'guia de turismo' 'secretaria bilingüe' 'agente de viajes' 'chef'
 'empleada domestica' 'biologa' 'inmuebles' 'directora general'
 'historiadora' 'tiket master' 'laboratorista'
 'relaciones internacionales' 'casa' 'secretaria' 'diseñadora gráfica'
 'alimentos' 'supermercado' 'estilista' 'auxiliar de cocina' 'pensionada'
 'estudiando otra carrera' 'empleada doméstica'
 'asistente del director de operaciones' 'venta de seguros' 'empleada'
 'limpieza en casa ajena' 'empleada lavandería' 'limpieza' 'bienes raices'
 'empleada de limpieza' 'empleada tienda neto' 'negocio propio' 'costura'
 'terapia alternativa' 'tanatologa' 'costurera' 'personal limpieza'
 'promotor cultural' 'evaciones psicologicas' 'agente

In [343]:
# Filtrar los valores únicos de las columnas "trabajo_otro"
valores_unicos_trabajo_otro = [all_valores_unicos[i] for i in range(len(all_valores_unicos)) if i % 2 == 1]

# Mostrar los valores filtrados
for idx, valores in enumerate(valores_unicos_trabajo_otro):
    print(f"Los valores únicos en la columna 'trabajo_otro' (índice {2*idx + 1}) son: {valores}")


Los valores únicos en la columna 'trabajo_otro' (índice 1) son: ['ninguna' 'em' 'ama de casa' 'hogar' 'estudiando' 'coaching'
 'guia de turismo' 'secretaria bilingüe' 'agente de viajes' 'chef'
 'empleada domestica' 'biologa' 'inmuebles' 'directora general'
 'historiadora' 'tiket master' 'laboratorista'
 'relaciones internacionales' 'casa' 'secretaria' 'diseñadora gráfica'
 'alimentos' 'supermercado' 'estilista' 'auxiliar de cocina' 'pensionada'
 'estudiando otra carrera' 'empleada doméstica'
 'asistente del director de operaciones' 'venta de seguros' 'empleada'
 'limpieza en casa ajena' 'empleada lavandería' 'limpieza' 'bienes raices'
 'empleada de limpieza' 'empleada tienda neto' 'negocio propio' 'costura'
 'terapia alternativa' 'tanatologa' 'costurera' 'personal limpieza'
 'promotor cultural' 'evaciones psicologicas' 'agente de seguros'
 'homeopatia' 'banco' 'cocinera' 'personal de limpieza por cuenta propia'
 'asistente documental' 'aseo domestico' 'trabajadora doméstica'
 'sastreri

In [344]:
# Guardar los valores únicos de las columnas "trabajo_otro" en una lista de listas
lista_trabajo_otro = [list(valores) for valores in valores_unicos_trabajo_otro]
lista_trabajo_otro

[['ninguna',
  'em',
  'ama de casa',
  'hogar',
  'estudiando',
  'coaching',
  'guia de turismo',
  'secretaria bilingüe',
  'agente de viajes',
  'chef',
  'empleada domestica',
  'biologa',
  'inmuebles',
  'directora general',
  'historiadora',
  'tiket master',
  'laboratorista',
  'relaciones internacionales',
  'casa',
  'secretaria',
  'diseñadora gráfica',
  'alimentos',
  'supermercado',
  'estilista',
  'auxiliar de cocina',
  'pensionada',
  'estudiando otra carrera',
  'empleada doméstica',
  'asistente del director de operaciones',
  'venta de seguros',
  'empleada',
  'limpieza en casa ajena',
  'empleada lavandería',
  'limpieza',
  'bienes raices',
  'empleada de limpieza',
  'empleada tienda neto',
  'negocio propio',
  'costura',
  'terapia alternativa',
  'tanatologa',
  'costurera',
  'personal limpieza',
  'promotor cultural',
  'evaciones psicologicas',
  'agente de seguros',
  'homeopatia',
  'banco',
  'cocinera',
  'personal de limpieza por cuenta propia',
  

## Agregamos el JSON códigos de ocupaciones

In [345]:
with open('../../encuestas/codigos_preguntas/propuesta.json', 'r', encoding='utf-8') as file:
    codigos_ocupaciones = json.load(file)

In [346]:
# Verificar el tipo de las subcategorias en el JSON proporcionado
subcategories_type = {category: type(subcategories[0]) if subcategories else None for category, subcategories in codigos_ocupaciones.items()}
subcategories_type

{'actividades deportivas_1': str,
 'artista_2': str,
 'chofer_3': str,
 'comerciante sector informal_4': str,
 'comerciante sector formal_5': str,
 'empresario_6': str,
 'comercio virtual_7': str,
 'atención a clientes_8': str,
 'administrativo_9': str,
 'gubernamental administrativo_10': str,
 'empleado gubernamental_11': str,
 'personal técnico_12': str,
 'área educativa_13': str,
 'profesionista independiente_14': str,
 'seguridad_15': str,
 'área de la salud_16': str,
 'construcción_17': str,
 'fábrica_18': str,
 'agricultura_19': str,
 'actividades elementales y de apoyo_20': str,
 'oficios_21': str,
 'desconocido_99': str}

In [347]:
# Crear un diccionario inverso
inverted_propuesta = {}
for category, subcategories in codigos_ocupaciones.items():
    for subcategory in subcategories:
        inverted_propuesta[subcategory] = category

inverted_propuesta

{'entrenador': 'actividades deportivas_1',
 'instructor': 'actividades deportivas_1',
 'profesor de clases deportivas': 'actividades deportivas_1',
 'actor': 'artista_2',
 'ballet': 'artista_2',
 'cantante': 'artista_2',
 'diseñador': 'artista_2',
 'fotógrafo': 'artista_2',
 'modelo': 'artista_2',
 'músico': 'artista_2',
 'pintor': 'artista_2',
 'camiones': 'chofer_3',
 'microbús': 'chofer_3',
 'taxista': 'chofer_3',
 'uber': 'chofer_3',
 'bazar': 'comerciante sector informal_4',
 'flores y productos en un puesto no establecido': 'comerciante sector informal_4',
 'ropa': 'comerciante sector informal_4',
 'vendedor': 'comerciante sector informal_4',
 'vendedor por cuenta propia (negocio propio)': 'comerciante sector informal_4',
 'venta de cigarros': 'comerciante sector informal_4',
 'periódicos flores y productos en un puesto no establecido': 'comerciante sector formal_5',
 'productos en un puesto no establecido': 'comerciante sector formal_5',
 'taquero': 'comerciante sector formal_5'

In [348]:
# Mapear los valores en las columnas 'trabajo_otro' que corresponden a '-oth-' en las columnas 'trabajo'
mapped_values = []
for i in range(3):
    current_mapping = {}
    for value in lista_trabajo_otro[i]:
        if value in inverted_propuesta:
            current_mapping[value] = inverted_propuesta[value]
        else:
            current_mapping[value] = "unmapped"
    mapped_values.append(current_mapping)

mapped_values

[{'ninguna': 'desconocido_99',
  'em': 'unmapped',
  'ama de casa': 'unmapped',
  'hogar': 'unmapped',
  'estudiando': 'unmapped',
  'coaching': 'unmapped',
  'guia de turismo': 'unmapped',
  'secretaria bilingüe': 'unmapped',
  'agente de viajes': 'unmapped',
  'chef': 'unmapped',
  'empleada domestica': 'unmapped',
  'biologa': 'unmapped',
  'inmuebles': 'unmapped',
  'directora general': 'unmapped',
  'historiadora': 'unmapped',
  'tiket master': 'unmapped',
  'laboratorista': 'unmapped',
  'relaciones internacionales': 'unmapped',
  'casa': 'unmapped',
  'secretaria': 'administrativo_9',
  'diseñadora gráfica': 'unmapped',
  'alimentos': 'unmapped',
  'supermercado': 'unmapped',
  'estilista': 'unmapped',
  'auxiliar de cocina': 'unmapped',
  'pensionada': 'unmapped',
  'estudiando otra carrera': 'unmapped',
  'empleada doméstica': 'unmapped',
  'asistente del director de operaciones': 'unmapped',
  'venta de seguros': 'unmapped',
  'empleada': 'unmapped',
  'limpieza en casa ajena

In [349]:
# Definir palabras clave para cada categoría
palabras_clave_categorias = {
    'Actividades deportivas_1': ['entrenador', 'deporte', 'gimnasio', 'futbol', 'basquetbol'],
    'Artista_2': ['actor', 'actriz', 'pintor', 'escultor', 'músico', 'bailarín'],
    'Chofer_3': ['taxista', 'conductor', 'manejador', 'uber', 'camionero'],
    'Comerciante sector formal_4': ['vendedor', 'tienda', 'comercio', 'puesto', 'local', 'negocio'],
    'Comerciante sector informal_5': ['vendedor ambulante', 'tianguis', 'puesto callejero'],
    'empresario_6': ['director', 'dueño', 'negocio', 'empresa', 'CEO'],
    'Comercio Virtual_7': ['e-commerce', 'tienda online', 'mercado virtual', 'venta online'],
    'Atención a clientes_8': ['servicio al cliente', 'call center', 'soporte'],
    'Administrativo_9': ['oficina', 'secretaria', 'asistente', 'coordinador', "secretaria bilingüe", "asistente documental", 
                         "trabajo de oficina"],
    'Gubernamental administrativo_10': ['gobierno', 'municipal', 'federal', 'estado', "empleado federal"],
    'Empleado gubernamental_11': ['burocracia', 'oficial', 'funcionario'],
    'Operador de maquinaria móvil_12': ['operador', 'maquinaria', 'equipo pesado'],
    'Personal técnico_13': ['técnico', 'especialista', 'ingeniero', 'ingeniero en alimentos','ingeniero mecanico',
                            'ingeniero aeronautico','ingeniero electricista','ingeniero en sistemas'],
    'Área educativa_14': ['profesor', 'maestro', 'docente', 'educador', 'estudiando', 'coaching', 'historiadora', 'laboratorista'],
    'Profesionista independiente_15': ['freelance', 'consultor', 'asesor', 'independiente'],
    'Seguridad_16': ['guardia', 'seguridad', 'policía', 'vigilante', "guardia de seguridad"],
    'Área de la salud_17': ['médico', 'enfermero', 'terapeuta', 'psicólogo'],
    'Construcción_18': ['albañil', 'constructor', 'arquitecto', 'ingeniero civil'],
    'Fábrica_19': ['operador', 'fábrica', 'producción', 'ensamblador'],
    "actividades elementales y de apoyo_20": ['casa', 'domestica', 'limpieza', 'empleada doméstica', 'hogar'],
    "otros": ['inmuebles', 'tiket master', 'bienes raices', 'evaciones psicologicas', 'sastreria'],
    "desconocido_99": ["ninguna", 'no sé', 'no se', 'se desconoce', 'desconocido']
}

In [350]:
def categorizar_valor(valor, palabras_clave_categorias):
    """Categoriza un valor basado en palabras clave."""
    for categoria, palabras_clave in palabras_clave_categorias.items():
        for palabra in palabras_clave:
            if palabra in valor:
                return categoria
    return "unmapped"

In [351]:
# Iterar sobre los diccionarios dentro de la lista mapped_values y categorizar los valores "unmapped"
categorizados = {}
for diccionario in mapped_values:
    for valor, mapeo in diccionario.items():
        if mapeo == "unmapped":
            categoria = categorizar_valor(valor, palabras_clave_categorias)
            if categoria not in categorizados:
                categorizados[categoria] = []
            categorizados[categoria].append(valor)

categorizados

{'unmapped': ['em',
  'guia de turismo',
  'agente de viajes',
  'chef',
  'biologa',
  'relaciones internacionales',
  'diseñadora gráfica',
  'alimentos',
  'supermercado',
  'estilista',
  'auxiliar de cocina',
  'pensionada',
  'venta de seguros',
  'empleada',
  'empleada lavandería',
  'costura',
  'terapia alternativa',
  'tanatologa',
  'costurera',
  'promotor cultural',
  'agente de seguros',
  'homeopatia',
  'banco',
  'cocinera',
  'aseo domestico',
  'trabajadora doméstica',
  'economista',
  'sector bancario',
  'estacion de radio',
  'banco',
  'seguros',
  'economia',
  'biologo',
  'imprenta',
  'ingenieria mecanica',
  'empleado general',
  'productor de videos',
  'farmaceutica',
  'contador publico',
  'sector turistico',
  'intendencia',
  'mecanico',
  'ventas',
  'servicios',
  'gerente de ventas',
  'club',
  'no sabe',
  'joyero grabador',
  'programador',
  'pensionado',
  'herrero y diversos',
  'trabaja en banco',
  'facturas',
  'jubilado',
  'ing. industr

In [352]:
# Función para recategorizar los valores "unmapped"
def recategorizar_unmapped(categorizados, palabras_clave_categorias):
    nuevos_categorizados = {}
    for categoria, trabajos in categorizados.items():
        for trabajo in trabajos:
            nueva_categoria = categorizar_valor(trabajo, palabras_clave_categorias)
            if nueva_categoria not in nuevos_categorizados:
                nuevos_categorizados[nueva_categoria] = []
            nuevos_categorizados[nueva_categoria].append(trabajo)
    return nuevos_categorizados

In [353]:
# Recategorizar los valores "unmapped" usando las funciones y diccionarios previamente definidos
categorizados_actualizados = recategorizar_unmapped(categorizados, palabras_clave_categorias)
categorizados_actualizados

{'unmapped': ['em',
  'guia de turismo',
  'agente de viajes',
  'chef',
  'biologa',
  'relaciones internacionales',
  'diseñadora gráfica',
  'alimentos',
  'supermercado',
  'estilista',
  'auxiliar de cocina',
  'pensionada',
  'venta de seguros',
  'empleada',
  'empleada lavandería',
  'costura',
  'terapia alternativa',
  'tanatologa',
  'costurera',
  'promotor cultural',
  'agente de seguros',
  'homeopatia',
  'banco',
  'cocinera',
  'aseo domestico',
  'trabajadora doméstica',
  'economista',
  'sector bancario',
  'estacion de radio',
  'banco',
  'seguros',
  'economia',
  'biologo',
  'imprenta',
  'ingenieria mecanica',
  'empleado general',
  'productor de videos',
  'farmaceutica',
  'contador publico',
  'sector turistico',
  'intendencia',
  'mecanico',
  'ventas',
  'servicios',
  'gerente de ventas',
  'club',
  'no sabe',
  'joyero grabador',
  'programador',
  'pensionado',
  'herrero y diversos',
  'trabaja en banco',
  'facturas',
  'jubilado',
  'ing. industr

# Trabajos mamá

In [354]:
from auxiliar_2 import *

In [355]:
encuesta

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,fecha,folio,encuestador,...,S2LUD,S2LUD[other],C0NSTRUCC10N,C0NSTRUCC10N[other],FABRUIK,FABRUIK[other],AGRICULTUR3,AGRICULTUR3[other],AbuelaPatTrabajoApoy,AbuelaPatTrabajoOfic
0,7,2021-03-29 19:22:58,3,es-MX,125280212,2021-03-29 19:22:40,2021-03-29 19:22:58,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,2021-03-30 12:45:52,3,es-MX,817932738,2021-03-30 12:30:25,2021-03-30 12:45:52,2021-03-30 00:00:00,5300.0,A22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17,2021-03-30 14:46:16,3,es-MX,740600100,2021-03-30 14:02:49,2021-03-30 14:46:16,2021-03-30 00:00:00,5108.0,A03,...,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,2021-03-30 14:58:34,3,es-MX,175910133,2021-03-30 14:32:46,2021-03-30 14:58:34,2021-03-30 00:00:00,5156.0,A15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,2021-03-30 17:56:46,3,es-MX,1422063480,2021-03-30 17:34:06,2021-03-30 17:56:46,2021-03-30 00:00:00,5301.0,A05,...,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,768,2022-10-29 17:45:55,3,es-MX,1021658192,2022-10-29 17:32:41,2022-10-29 17:45:55,2022-10-29 00:00:00,9027.0,A05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,769,2022-10-29 18:35:03,3,es-MX,561370725,2022-10-29 18:23:45,2022-10-29 18:35:02,2022-10-29 00:00:00,9028.0,A05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,770,2022-10-29 20:41:22,3,es-MX,1664763119,2022-10-29 20:26:48,2022-10-29 20:41:22,2022-10-30 00:00:00,9029.0,A05,...,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN
702,771,2022-10-31 19:27:58,3,es-MX,1435893634,2022-10-31 19:03:23,2022-10-31 19:27:58,2022-10-31 00:00:00,9030.0,A11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Limpieza Nivel 1

In [356]:
mostrar_unicos(encuesta,"Trabajo")

0     -oth-
1        10
2        11
3        12
4        13
5        14
6        15
7        16
8        18
9         2
10       20
11       21
12        3
13        4
14        5
15        6
16        7
17        8
18        9
19      NaN
Name: Trabajo, dtype: object

In [357]:
mapeo_respuestas = {
    '1': 'Actividades deportivas_1',
    '2': 'Comerciante sector formal_4',
    '3': 'Empresario_6',
    '4': 'Comercio Virtual_7',
    '5': 'Atención a clientes_8',
    '6': 'Administrativo_9',
    '7': 'Gubernamental administrativo_10',
    '8': 'Empleado gubernamental_11',
    '9': 'Operador de maquinaria móvil_12',
    '10': 'Personal técnico_13',
    '11': 'Área educativa_14',
    '12': 'Profesionista independiente_15',
    '13': 'Seguridad_16',
    '14': 'Área de la salud_17',
    '15': 'Construcción_18',
    '16': 'Fábrica_19',
    '18': 'Artista_2',
    '19': 'Chofer_3',
    '20': 'Comerciante sector informal_5',
    '21': 'No trabaja/ trabajaba_21'
}


In [358]:
llenar_respuesta_libre(encuesta,"Trabajo", identificador_otro='-oth-', sufijo='[other]')
#No volver a correr

In [359]:
llenar_vacios(encuesta,"Trabajo",'ninguna')
encuesta['Trabajo'] = encuesta['Trabajo'].str.strip().str.lower()


In [360]:
encuesta['Trabajo'] = encuesta['Trabajo'].replace(mapeo_respuestas)
#No correr de nuevo

In [361]:
mostrar_unicos(encuesta,"Trabajo")

0                  Administrativo_9
1                         Artista_2
2             Atención a clientes_8
3       Comerciante sector formal_4
4     Comerciante sector informal_5
                  ...              
67                     tiket master
68            trabajadora doméstica
69                 venta de seguros
70              Área de la salud_17
71                Área educativa_14
Name: Trabajo, Length: 72, dtype: object

## Limpieza de nivel 2

Primeramente lo que se realizará será buscar las columnas base para los trabajos de mamá y posteriormente combinarlas.

### columnas base

In [362]:
columnas_base = ['opcionesdeportivas', 'opcartista', 'opcchofer', 'opccomerciaFormal', 'comerciFromal', 'opcempresario', 'opccomerciovirtual', 'opcatencionaclientes', 'opcAdminis', 'opcempleadoguber', 'opcGubernamental', 'Operadormaquinaria', 'Opcpersonaltecnico', 'opceducativ', 'opciondependiente', 'opcseguridad', 'opcionareadesalud', 'opcconstruccion', 'farbrig', 'opcagricultura']
columnas_base


['opcionesdeportivas',
 'opcartista',
 'opcchofer',
 'opccomerciaFormal',
 'comerciFromal',
 'opcempresario',
 'opccomerciovirtual',
 'opcatencionaclientes',
 'opcAdminis',
 'opcempleadoguber',
 'opcGubernamental',
 'Operadormaquinaria',
 'Opcpersonaltecnico',
 'opceducativ',
 'opciondependiente',
 'opcseguridad',
 'opcionareadesalud',
 'opcconstruccion',
 'farbrig',
 'opcagricultura']

In [363]:
encuesta[columnas_base]

,opcionesdeportivas,opcartista,opcchofer,opccomerciaFormal,comerciFromal,opcempresario,opccomerciovirtual,opcatencionaclientes,opcAdminis,opcempleadoguber,opcGubernamental,Operadormaquinaria,Opcpersonaltecnico,opceducativ,opciondependiente,opcseguridad,opcionareadesalud,opcconstruccion,farbrig,opcagricultura
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [364]:
encuesta['opcartista'] = encuesta['opcartista'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2'})
encuesta['opccomerciaFormal'] = encuesta['opccomerciaFormal'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})
encuesta['comerciFromal'] = encuesta['comerciFromal'].replace({'A2': 'Vendedor de comida_1','A4':'Flores y productos en un puesto no establecido_3'})
encuesta['opcempresario'] = encuesta['opcempresario'].replace({'A2': ' Dueño_1','A3':'Director o gerente_2'})
encuesta['opccomerciovirtual'] = encuesta['opccomerciovirtual'].replace({'A2': ' Vendedor de productos por internet (mercado pago, amazon, instagram, etc.)_1'})

In [365]:
encuesta['opcartista'] = encuesta['opcartista'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2'})
encuesta['opccomerciaFormal'] = encuesta['opccomerciaFormal'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})
encuesta['comerciFromal'] = encuesta['comerciFromal'].replace({'A2': 'Vendedor de comida_1','A4':'Flores y productos en un puesto no establecido_3'})

In [366]:
encuesta['opcatencionaclientes'] = encuesta['opcatencionaclientes'].replace({'A2': 'Atención a clientes_1','A3':'Recepcionista_2','A4':'Vendedor de tiendas departamentales_3'})

In [367]:
encuesta['opcAdminis'] = encuesta['opcAdminis'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})
encuesta['opcempleadoguber'] = encuesta['opcempleadoguber'].replace({'A1': '¿DE QUÉ TIPO?_1'})
encuesta['opcGubernamental'] = encuesta['opcGubernamental'].replace({'A6': 'Administrador_5','A8':'Administrador general_7'})

In [368]:
encuesta['Opcpersonaltecnico'] = encuesta['Opcpersonaltecnico'].replace({'A1': '¿DE QUÉ TIPO?_1'})
encuesta['opceducativ'] = encuesta['opceducativ'].replace({'A2': 'Investigador-profesor en universidad/preparatoria_1','A3':' Educación básica_2','A4':'Técnico en educación (educador o guardería)_3'})
encuesta['opciondependiente'] = encuesta['opciondependiente'].replace({'A2': 'Abogado_1','A3':' Psicólogo_2','A4':'Contador_3','A6':'Médico independiente_5','A7':'Terapeuta_6'})
encuesta['opcseguridad'] = encuesta['opcseguridad'].replace({'A4':'Vigilante_3'})
encuesta['opcionareadesalud'] = encuesta['opcionareadesalud'].replace({'A2':'Médico_1','A3': 'Enfermera_2'})
encuesta['farbrig'] = encuesta['farbrig'].replace({'A2':'Empleado_1'})

In [369]:
encuesta[columnas_base]

,opcionesdeportivas,opcartista,opcchofer,opccomerciaFormal,comerciFromal,opcempresario,opccomerciovirtual,opcatencionaclientes,opcAdminis,opcempleadoguber,opcGubernamental,Operadormaquinaria,Opcpersonaltecnico,opceducativ,opciondependiente,opcseguridad,opcionareadesalud,opcconstruccion,farbrig,opcagricultura
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investigador-profesor en universidad/preparato...,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Técnico en educación (educador o guardería)_3,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,¿DE QUÉ TIPO?_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### columnas con sufijos
El objetivo es pasar los datos de las columnas con sufijo a las columnas base.
No todas las columnas tienen el mismo sufijo, pero las dos opciones son `"[other]"` y `"[comment]"`:

In [370]:
columnas_sufijo = []
for col in columnas_base:
    if col + '[other]' in encuesta.columns:
        columnas_sufijo.append(col + '[other]')
    elif col + '[comment]' in encuesta.columns:
        columnas_sufijo.append(col + '[comment]')
assert len(columnas_sufijo) == len(columnas_base)
columnas_sufijo.sort()
columnas_sufijo

['Opcpersonaltecnico[comment]',
 'Operadormaquinaria[comment]',
 'comerciFromal[other]',
 'farbrig[other]',
 'opcAdminis[other]',
 'opcGubernamental[other]',
 'opcagricultura[other]',
 'opcartista[other]',
 'opcatencionaclientes[other]',
 'opcchofer[other]',
 'opccomerciaFormal[other]',
 'opccomerciovirtual[other]',
 'opcconstruccion[other]',
 'opceducativ[other]',
 'opcempleadoguber[comment]',
 'opcempresario[other]',
 'opcionareadesalud[other]',
 'opciondependiente[other]',
 'opcionesdeportivas[other]',
 'opcseguridad[other]']

In [371]:
for col_con_sufijo in columnas_sufijo:
    columna_sin_sufijo = re.sub(r'\[\w+\]', '', col_con_sufijo)
    encuesta[columna_sin_sufijo] = encuesta[columna_sin_sufijo].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo] = np.where(encuesta[columna_sin_sufijo] == '-oth-',
                                         encuesta[col_con_sufijo],
                                         encuesta[columna_sin_sufijo])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [372]:
encuesta[columnas_base]

,opcionesdeportivas,opcartista,opcchofer,opccomerciaFormal,comerciFromal,opcempresario,opccomerciovirtual,opcatencionaclientes,opcAdminis,opcempleadoguber,opcGubernamental,Operadormaquinaria,Opcpersonaltecnico,opceducativ,opciondependiente,opcseguridad,opcionareadesalud,opcconstruccion,farbrig,opcagricultura
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investigador-profesor en universidad/preparato...,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLASES DE DANZA,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Técnico en educación (educador o guardería)_3,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,¿DE QUÉ TIPO?_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asignación de Trabajo
Ahora que toda la información sobre las distintas ocupaciones está en las columnas base, se puede crear una columna que combine la información de todas ellas, la cual se llamará `'trabajo'`.

In [373]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)  # columna nueva
for columna in columnas_base:
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['Trabajo'])
encuesta.drop(columns='Trabajo', inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

# NO VOLVER A CORRER

0                abarrotes
1                  abogada
2                abogado_1
3                 actuario
4           administración
              ...         
178    ventas por catálogo
179          vienes raices
180            vigilante_3
181        ¿de qué tipo?_1
182    área de la salud_17
Name: trabajo, Length: 183, dtype: string

In [374]:
encuesta['trabajo']

0                                                ninguna
1      investigador-profesor en universidad/preparato...
2                                        clases de danza
3                               no trabaja/ trabajaba_21
4                               no trabaja/ trabajaba_21
                             ...                        
699                             no trabaja/ trabajaba_21
700                             no trabaja/ trabajaba_21
701        técnico en educación (educador o guardería)_3
702                                      ¿de qué tipo?_1
703                                          enfermera_2
Name: trabajo, Length: 704, dtype: string

In [375]:
trabajos_nuevos = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos

0                         clases de danza
1                             en farmacia
2      direccion general educacion basica
3                           bibliotecaria
4                 auxiliar administrativo
                      ...                
238                             recepcion
239                             sastreria
240                              limpieza
241                            secretaria
242                           ama de casa
Name: trabajo, Length: 243, dtype: string

In [376]:
#for palabra, sinonimos in tesauro_ama_de_casa.items():
 # encuesta['Trabajo[other]'] = encuesta['Trabajo[other]'].replace(sinonimos, palabra, regex=True)

### Corrección de trabajos

In [377]:
codigos_ocupaciones

{'actividades deportivas_1': ['entrenador',
  'instructor',
  'profesor de clases deportivas'],
 'artista_2': ['actor',
  'ballet',
  'cantante',
  'diseñador',
  'fotógrafo',
  'modelo',
  'músico',
  'pintor'],
 'chofer_3': ['camiones', 'microbús', 'taxista', 'uber'],
 'comerciante sector informal_4': ['bazar',
  'flores y productos en un puesto no establecido',
  'ropa',
  'vendedor',
  'vendedor por cuenta propia (negocio propio)',
  'venta de cigarros'],
 'comerciante sector formal_5': ['periódicos flores y productos en un puesto no establecido',
  'productos en un puesto no establecido',
  'taquero',
  'vendedor de comida'],
 'empresario_6': ['asesor',
  'consultor',
  'director',
  'dueño',
  'fundación',
  'gerente'],
 'comercio virtual_7': ['vendedor de productos por internet (mercado pago, amazon, instagram, etc.)'],
 'atención a clientes_8': ['atención a clientes',
  'cajero',
  'call center',
  'promotor de seguros',
  'recepcionista',
  'telefonía',
  'vendedor de tiendas 

In [378]:
trabajos_oficiales = pd.Series(dtype='string')
for t in codigos_ocupaciones.values():
    trabajos_oficiales = pd.concat([trabajos_oficiales, pd.Series(t)])
trabajos_oficiales.reset_index(drop=True, inplace=True)
trabajos_oficiales

0                         entrenador
1                         instructor
2      profesor de clases deportivas
3                              actor
4                             ballet
                   ...              
119                           sastre
120                         zapatero
121                         zapatero
122                         mecánico
123                          ninguna
Length: 124, dtype: object

In [796]:
from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper
from auxiliar_2 import *

In [380]:
#creación de un tesauro
tesauro_ama_de_casa = {
    "trabajadores domésticos": ["ama de casa", "AMA DE CASA", "ASEO DOMESTICO", "EMPLEADA DOMESTICA","empleada doméstica","Empleada doméstica","HOGAR","hogar","casa","limpieza en casa ajena","Ama de casa","Hogar","trabajadora doméstica","trabajadora domestica","Ama de trabajadores domésticos","empleada domestica","Limpieza en trabajadores domésticos ajena","trabajó en trabajadores domésticos","limpieza en trabajadores domésticos ajena","limpieza de trabajadores domésticoses","empleada de trabajadores domésticos","empleado domestico","empleado domestica","trabajó en trabajadores domésticos","empleada de trabajadores domésticos","trabajadores de limpiezaes de trabajadores de limpiezaes de limpieza","trabajadores de limpiezaes domésticos"]}

In [793]:
for palabra, sinonimos in tesauro_ama_de_casa.items():
   trabajos_nuevos = trabajos_nuevos.replace(sinonimos, palabra, regex=True)

In [797]:
correccion = match_most_similar(trabajos_oficiales,
                                trabajos_nuevos,
                                min_similarity=.42,
                                ignore_index=True)
comparar = pd.DataFrame({'original': trabajos_nuevos, 'correccion': correccion})
comparar['hubo_correccion'] = comparar['original'].ne(comparar['correccion'])
comparar.sort_values(by='hubo_correccion', ascending=False)

TypeError: __new__() got an unexpected keyword argument 'ignore_index'

In [383]:
mapeo_correcciones = dict(zip(trabajos_nuevos, correccion))
mapeo_correcciones

NameError: name 'correccion' is not defined

In [384]:
len(mapeo_correcciones)

NameError: name 'mapeo_correcciones' is not defined

In [385]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()


In [386]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip()

In [387]:
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones)


NameError: name 'mapeo_correcciones' is not defined

In [388]:
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta,'trabajo')

0                abarrotes
1                  abogada
2                  abogado
3                 actuario
4           administración
              ...         
176    ventas por catálogo
177          vienes raices
178              vigilante
179          ¿de qué tipo?
180       área de la salud
Name: trabajo, Length: 181, dtype: string

####  Corrección de algunos datos 

In [389]:
for palabra, sinonimos in tesauro_ama_de_casa.items():
    for sinonimo in sinonimos:
        encuesta['trabajo'] = encuesta['trabajo'].replace(sinonimo, palabra, regex=True)

In [390]:
encuesta[['trabajo']]



,trabajo
0,ninguna
1,investigador-profesor en universidad/preparatoria
2,clases de danza
3,no trabaja/ trabajaba
4,no trabaja/ trabajaba
...,...
699,no trabaja/ trabajaba
700,no trabaja/ trabajaba
701,técnico en educación (educador o guardería)
702,¿de qué tipo?


In [391]:
mostrar_unicos(encuesta,'trabajo')

0                abarrotes
1                  abogada
2                  abogado
3                 actuario
4           administración
              ...         
170    ventas por catálogo
171          vienes raices
172              vigilante
173          ¿de qué tipo?
174       área de la salud
Name: trabajo, Length: 175, dtype: string

In [392]:
encuesta.loc[432, 'trabajo']

'¿de qué tipo?'

In [393]:
encuesta.loc[278, 'trabajo']

'¿de qué tipo?'

In [394]:
encuesta.loc[81, 'trabajo']

'¿de qué tipo?'

In [395]:
encuesta.loc[702, 'trabajo']

'¿de qué tipo?'

In [396]:
encuesta.iloc[81, encuesta.columns.get_loc('trabajo')] = 'director'


In [397]:
encuesta.iloc[278, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[432, encuesta.columns.get_loc('trabajo')] = 'laboratorio'
encuesta.iloc[207, encuesta.columns.get_loc('trabajo')] = 'secretaria'
encuesta.iloc[213, encuesta.columns.get_loc('trabajo')] = 'administrador'
encuesta.iloc[256, encuesta.columns.get_loc('trabajo')] = 'abogado'
encuesta.iloc[287, encuesta.columns.get_loc('trabajo')] = 'servidor de la nación'
encuesta.iloc[302, encuesta.columns.get_loc('trabajo')] = 'secretaria'
encuesta.iloc[310, encuesta.columns.get_loc('trabajo')] = 'abogado'
encuesta.iloc[333, encuesta.columns.get_loc('trabajo')] = 'secretaria'
encuesta.iloc[368, encuesta.columns.get_loc('trabajo')] = 'secretaria'
encuesta.iloc[373, encuesta.columns.get_loc('trabajo')] = 'profesor'
encuesta.iloc[385, encuesta.columns.get_loc('trabajo')] = 'metro'
encuesta.iloc[458, encuesta.columns.get_loc('trabajo')] = 'no sé'
encuesta.iloc[503, encuesta.columns.get_loc('trabajo')] = 'empleado en seguridad pública'
encuesta.iloc[507, encuesta.columns.get_loc('trabajo')] = 'secretaria'
encuesta.iloc[511, encuesta.columns.get_loc('trabajo')] = 'sat'
encuesta.iloc[523, encuesta.columns.get_loc('trabajo')] = 'educación'
encuesta.iloc[702, encuesta.columns.get_loc('trabajo')] = 'abogado'


In [398]:
encuesta.iloc[246, encuesta.columns.get_loc('trabajo')] = 'fisioterapeuta'
encuesta.iloc[245, encuesta.columns.get_loc('trabajo')] = 'asistente médica'
encuesta.iloc[588, encuesta.columns.get_loc('trabajo')] = 'hipotecario'
encuesta.iloc[592, encuesta.columns.get_loc('trabajo')] = 'cuidado geriátrico'
encuesta.iloc[674, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'

In [399]:
encuesta[['trabajo']]

,trabajo
0,ninguna
1,investigador-profesor en universidad/preparatoria
2,clases de danza
3,no trabaja/ trabajaba
4,no trabaja/ trabajaba
...,...
699,no trabaja/ trabajaba
700,no trabaja/ trabajaba
701,técnico en educación (educador o guardería)
702,abogado


### Modificar algunos sinónimos

In [400]:
encuesta.loc[523, 'trabajo'] 

'educación'

In [401]:
encuesta.iloc[99, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[583, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[579, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[165, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[674, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'
encuesta.iloc[452, encuesta.columns.get_loc('trabajo')] = 'peluquero'
encuesta.iloc[321, encuesta.columns.get_loc('trabajo')] = 'peluquero'
encuesta.iloc[300, encuesta.columns.get_loc('trabajo')] = 'peluquero'
encuesta.iloc[537, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[457, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[72, encuesta.columns.get_loc('trabajo')] = 'psicólogo'
encuesta.iloc[65, encuesta.columns.get_loc('trabajo')] = 'psicólogo'
encuesta.iloc[471, encuesta.columns.get_loc('trabajo')] = 'policía'
encuesta.iloc[112, encuesta.columns.get_loc('trabajo')] = 'biólogo'


In [402]:
encuesta.iloc[582, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[612, encuesta.columns.get_loc('trabajo')] = 'empleado'
encuesta.iloc[285, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[509, encuesta.columns.get_loc('trabajo')] = 'administrador'
encuesta.iloc[541, encuesta.columns.get_loc('trabajo')] = 'idiomas'
encuesta.iloc[682, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[586, encuesta.columns.get_loc('trabajo')] = 'estudiante'
encuesta.iloc[337, encuesta.columns.get_loc('trabajo')] = 'cocinero'
encuesta.iloc[275, encuesta.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
encuesta.iloc[675, encuesta.columns.get_loc('trabajo')] = 'vendedor por cuenta propia (negocio propio)'
encuesta.iloc[490, encuesta.columns.get_loc('trabajo')] = 'vendedor'
encuesta.iloc[523, encuesta.columns.get_loc('trabajo')] = 'área educativa'


In [403]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip()

### Creación de categorías

In [404]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion clases de danza es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion en farmacia es desconocida
La ocupacion direccion general educacion basica es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion bibliotecaria es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion auxiliar administrativo es desconocida
La ocupacion profesionista independiente es desconocida
La ocupacion em es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion agente de viajes es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion no trabaja/ tr

In [405]:
encuesta[['trabajo']]

,trabajo
0,ninguna
1,investigador-profesor en universidad/preparatoria
2,clases de danza
3,no trabaja/ trabajaba
4,no trabaja/ trabajaba
...,...
699,no trabaja/ trabajaba
700,no trabaja/ trabajaba
701,técnico en educación (educador o guardería)
702,abogado


In [406]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0     actividades elementales y de apoyo_20
1                          administrativo_9
2                                 artista_2
3                     atención a clientes_8
4               comerciante sector formal_5
5             comerciante sector informal_4
6                        comercio virtual_7
7                            desconocido_99
8                 empleado gubernamental_11
9                              empresario_6
10                               fábrica_18
11                               oficios_21
12           profesionista independiente_14
13                             seguridad_15
14                      área de la salud_16
15                        área educativa_13
dtype: string

In [407]:
categorias.value_counts()

desconocido_99                           327
actividades elementales y de apoyo_20     80
área educativa_13                         64
profesionista independiente_14            41
administrativo_9                          36
comerciante sector informal_4             33
área de la salud_16                       26
empresario_6                              23
empleado gubernamental_11                 22
comerciante sector formal_5               20
atención a clientes_8                     12
fábrica_18                                10
oficios_21                                 4
seguridad_15                               3
artista_2                                  2
comercio virtual_7                         1
Name: count, dtype: Int64

In [408]:
encuesta['categoria'] = categorias

In [409]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,investigador-profesor en universidad/preparatoria,área educativa_13
2,clases de danza,desconocido_99
3,no trabaja/ trabajaba,desconocido_99
4,no trabaja/ trabajaba,desconocido_99
...,...,...
699,no trabaja/ trabajaba,desconocido_99
700,no trabaja/ trabajaba,desconocido_99
701,técnico en educación (educador o guardería),área educativa_13
702,abogado,profesionista independiente_14


### Asignación de la columna "Trabajas[Madre]"
Esta columna se crea para corregir los datos de zoom 2, donde no se realiza la pregunta: ¿En qué trabajas".La creación de esta columna nos ayudará a poder limpiar aquellos trabajos que esten nombrados como: "no trabaja/trabajaba".

In [410]:
encuesta['Trabajas[Madre]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] == 'no trabaja/ trabajaba':
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[Madre]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[Madre]'] = 'Y'

In [411]:
#Esta función nos ayudará a desplazar la columna "Trabajas"
column_order = list(encuesta.columns)

# Reordenar la lista de columnas para que 'Trabajas[Madre]' esté antes de 'Trabajas'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[Madre]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [412]:
encuesta[['Trabajas[Madre]','trabajo', 'categoria']]

,Trabajas[Madre],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,investigador-profesor en universidad/preparatoria,área educativa_13
2,Y,clases de danza,desconocido_99
3,N,no trabaja/ trabajaba,desconocido_99
4,N,no trabaja/ trabajaba,desconocido_99
...,...,...,...
699,N,no trabaja/ trabajaba,desconocido_99
700,N,no trabaja/ trabajaba,desconocido_99
701,Y,técnico en educación (educador o guardería),área educativa_13
702,Y,abogado,profesionista independiente_14


In [413]:
# Reemplazar todos los valores 'no trabaja/trabajaba' en la columna 'trabajo' con 'ninguna'
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/ trabajaba', 'ninguna')


In [414]:
encuesta['trabajo']

0                                                ninguna
1      investigador-profesor en universidad/preparatoria
2                                        clases de danza
3                                                ninguna
4                                                ninguna
                             ...                        
699                                              ninguna
700                                              ninguna
701          técnico en educación (educador o guardería)
702                                              abogado
703                                            enfermera
Name: trabajo, Length: 704, dtype: string

### Últimas modificaciones sobre categorías

In [415]:
#Últimas modificaciones
encuesta.loc[52, 'trabajo']

'prescolar'

In [416]:
encuesta.iloc[35, encuesta.columns.get_loc('trabajo')] ='educación básica'
encuesta.iloc[117, encuesta.columns.get_loc('trabajo')] ='educación básica'
encuesta.iloc[52, encuesta.columns.get_loc('trabajo')] ='educación básica'
encuesta.iloc[179, encuesta.columns.get_loc('trabajo')] ='administrador'
encuesta.iloc[8, encuesta.columns.get_loc('trabajo')] ='bibliotecario'
encuesta.iloc[289, encuesta.columns.get_loc('trabajo')] ='joyero'
encuesta.iloc[38, encuesta.columns.get_loc('trabajo')] ='trabaja en banco'
encuesta.iloc[94, encuesta.columns.get_loc('trabajo')] ='financiero'
encuesta.iloc[632, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[418, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[166, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[46, encuesta.columns.get_loc('trabajo')] ='técnico'
encuesta.iloc[271, encuesta.columns.get_loc('trabajo')] ='zapatero'
encuesta.iloc[357, encuesta.columns.get_loc('trabajo')] ='telefonía'
encuesta.iloc[167, encuesta.columns.get_loc('trabajo')] ='diseñador'
encuesta.iloc[122, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[113, encuesta.columns.get_loc('trabajo')] ='flores y productos en un puesto no establecido'
encuesta.iloc[253, encuesta.columns.get_loc('trabajo')] ='empleado'
encuesta.iloc[585, encuesta.columns.get_loc('trabajo')] ='trabaja en banco'
encuesta.iloc[642, encuesta.columns.get_loc('trabajo')] ='obrero'

In [417]:
encuesta.iloc[524, encuesta.columns.get_loc('trabajo')] ='bienes raíces'
encuesta.iloc[453, encuesta.columns.get_loc('trabajo')] ='bienes raíces'
encuesta.iloc[190, encuesta.columns.get_loc('trabajo')] ='bienes raíces'
encuesta.iloc[170, encuesta.columns.get_loc('trabajo')] ='bienes raíces'
encuesta.iloc[177, encuesta.columns.get_loc('trabajo')] ='bienes raíces'

In [418]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion clases de danza es desconocida
La ocupacion en farmacia es desconocida
La ocupacion direccion general educacion basica es desconocida
La ocupacion bibliotecario es desconocida
La ocupacion auxiliar administrativo es desconocida
La ocupacion profesionista independiente es desconocida
La ocupacion em es desconocida
La ocupacion agente de viajes es desconocida
La ocupacion trabaja en banco es desconocida
La ocupacion trabajadora social es desconocida
La ocupacion estudiando es desconocida
La ocupacion técnico es desconocida
La ocupacion coaching es desconocida
La ocupacion guia de turismo es desconocida
La ocupacion dueña del laboratorio es desconocida
La ocupacion ingeniera industrial es desconocida
La ocupacion productora fotografia es desconocida
La ocupacion no se es desconocida
La ocupacion logistica es desconocida
La ocupacion secretaria bilingüe es desconocida
La ocupacion agente de viajes es desconocida
La ocupacion financiero es desconocida
La ocupacion consultoria e

In [419]:
encuesta.iloc[179, encuesta.columns.get_loc('categoria')] ='empleado gubernamental_11'

In [420]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()

# Asigna la categoría correspondiente a las ocupaciones específicas
ocupaciones_formal = ["comerciante sector formal"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal","sobrerruedas","vendedor",'flores y productos en un puesto no establecido']
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [421]:
ocupaciones_especificas = ["bibliotecario", "laboratorio", "supervision en sep", "laboratorista químico",'direccion general educacion basica','dueña del laboratorio','laboratorista',"educación básica","consultora en educación y terapia psicológica","editorial"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_especificas), 'categoria'] = "área educativa_13"

In [422]:
# Asigna la categoría correspondiente a las ocupaciones específicas
ocupaciones_profesionista_ind = ["profesionista independiente", "relaciones internacionales", "fisico medico", "internacionalista",'actuario','comunicacion','promotor cultural',"trabajadora social",'historiadora',"oficinista","financiero","revisora en revista"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profesionista_ind), 'categoria'] = "profesionista independiente_14"


In [423]:
ocupaciones_gub = ["empleado gubernamental","legisladora","sat","servidor de la nación"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_gub), 'categoria'] = "empleado gubernamental_11"

In [424]:
ocupaciones_salud = ["área de la salud","cuidado geriátrico","asistente médica","fisioterapeuta"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_salud), 'categoria'] = "área de la salud_16"

In [425]:
#Ver esto de nuevo
ocupaciones_apoyo = ["logistica"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_apoyo), 'categoria'] = "actividades elementales y de apoyo_20"

In [426]:
ocupaciones_empresario = ["director o gerente", "socio"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empresario), 'categoria'] = "empresario_6"


In [427]:
ocupaciones_of = ["ceramica","bolero","panadero","petrolero","plomero","joyero","mecánico","armero", "bombero","ganadero","lacteos","zapatero","lavanderia"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"

In [428]:
ocupaciones_cliente= ["tienda","oxxo","supermercado","guia de turismo","agente de viajes","telefonía",'bienes raíces']
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"

In [429]:
ocupaciones_admin = ["trabaja en banco","hipotecario"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_admin), 'categoria'] = "administrativo_9"

In [430]:
ocupaciones_fabrica=["obrero","fábrica","empleado"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_fabrica), 'categoria'] = "fábrica_18"

In [431]:
ocupaciones_seg = ["seguridad","militar"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_seg), 'categoria'] = "seguridad_15"

In [432]:
ocupaciones_art= ["diseñador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_art), 'categoria'] = "artista_2"
ocupaciones_tec = ["técnico"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_tec), 'categoria'] = "personal técnico_12"

In [433]:
encuesta[['Trabajas[Madre]','trabajo', 'categoria']]

,Trabajas[Madre],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,investigador-profesor en universidad/preparatoria,área educativa_13
2,Y,clases de danza,desconocido_99
3,N,ninguna,desconocido_99
4,N,ninguna,desconocido_99
...,...,...,...
699,N,ninguna,desconocido_99
700,N,ninguna,desconocido_99
701,Y,técnico en educación (educador o guardería),área educativa_13
702,Y,abogado,profesionista independiente_14


In [434]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           258
actividades elementales y de apoyo_20     81
área educativa_13                         76
profesionista independiente_14            52
comerciante sector informal_4             40
administrativo_9                          39
área de la salud_16                       30
empresario_6                              27
empleado gubernamental_11                 27
atención a clientes_8                     25
comerciante sector formal_5               21
fábrica_18                                12
oficios_21                                 8
artista_2                                  3
seguridad_15                               3
personal técnico_12                        1
comercio virtual_7                         1
Name: count, dtype: Int64

In [435]:
#encuesta.rename(columns={'Trabajo[Mamá]': 'trabajo'}, inplace=True)

In [436]:
from auxiliar_2 import *

### Hasta aquí llegamos

In [437]:
mapeo_codigos_categoria = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria

{'desconocido_99': 'A99',
 'área educativa_13': 'A13',
 'empresario_6': 'A06',
 'comerciante sector informal_4': 'A04',
 'administrativo_9': 'A09',
 'actividades elementales y de apoyo_20': 'A20',
 'profesionista independiente_14': 'A14',
 'empleado gubernamental_11': 'A11',
 'comerciante sector formal_5': 'A05',
 'comercio virtual_7': 'A07',
 'fábrica_18': 'A18',
 'artista_2': 'A02',
 'área de la salud_16': 'A16',
 'atención a clientes_8': 'A08',
 'oficios_21': 'A21',
 'seguridad_15': 'A15'}

In [438]:
mapeo_codigos_categoria['personal técnico_12'] = 'A12'

mapeo_codigos_categoria

{'desconocido_99': 'A99',
 'área educativa_13': 'A13',
 'empresario_6': 'A06',
 'comerciante sector informal_4': 'A04',
 'administrativo_9': 'A09',
 'actividades elementales y de apoyo_20': 'A20',
 'profesionista independiente_14': 'A14',
 'empleado gubernamental_11': 'A11',
 'comerciante sector formal_5': 'A05',
 'comercio virtual_7': 'A07',
 'fábrica_18': 'A18',
 'artista_2': 'A02',
 'área de la salud_16': 'A16',
 'atención a clientes_8': 'A08',
 'oficios_21': 'A21',
 'seguridad_15': 'A15',
 'personal técnico_12': 'A12'}

In [439]:
encuesta['categoria'].replace(mapeo_codigos_categoria, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

0     A02
1     A04
2     A05
3     A06
4     A07
5     A08
6     A09
7     A11
8     A12
9     A13
10    A14
11    A15
12    A16
13    A18
14    A20
15    A21
16    A99
Name: categoria, dtype: string

### Resultado Final

In [440]:
mapeo_codigos_ocupaciones = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones

{'abogada': 'A9901',
 'administración': 'A9902',
 'administrativa': 'A9903',
 'agente de seguros': 'A9904',
 'agente telefonico en inova': 'A9905',
 'asistente del director de operaciones': 'A9906',
 'asistente documental': 'A9907',
 'asistente ejecutiva': 'A9908',
 'auxiliar administrativo': 'A9909',
 'clases de arte': 'A9910',
 'clases de danza': 'A9911',
 'coaching': 'A9912',
 'cocinera': 'A9913',
 'consultoria': 'A9914',
 'costura': 'A9915',
 'costurera': 'A9916',
 'departamento de ventas aereolinea': 'A9917',
 'desconocido': 'A9918',
 'directora': 'A9919',
 'directora general': 'A9920',
 'diseñadora gráfica': 'A9921',
 'em': 'A9922',
 'empleada': 'A9923',
 'empleada de limpieza': 'A9924',
 'empleada lavandería': 'A9925',
 'empleada tienda neto': 'A9926',
 'en farmacia': 'A9927',
 'estudiando': 'A9928',
 'estudiando otra carrera': 'A9929',
 'evaciones psicologicas': 'A9930',
 'farmacologia': 'A9931',
 'homeopatia': 'A9932',
 'ingeniera en sistemas y computacion': 'A9933',
 'ingenie

In [441]:
mapeo_codigos_ocupaciones["técnico"] = "A1201"
mapeo_codigos_ocupaciones

{'abogada': 'A9901',
 'administración': 'A9902',
 'administrativa': 'A9903',
 'agente de seguros': 'A9904',
 'agente telefonico en inova': 'A9905',
 'asistente del director de operaciones': 'A9906',
 'asistente documental': 'A9907',
 'asistente ejecutiva': 'A9908',
 'auxiliar administrativo': 'A9909',
 'clases de arte': 'A9910',
 'clases de danza': 'A9911',
 'coaching': 'A9912',
 'cocinera': 'A9913',
 'consultoria': 'A9914',
 'costura': 'A9915',
 'costurera': 'A9916',
 'departamento de ventas aereolinea': 'A9917',
 'desconocido': 'A9918',
 'directora': 'A9919',
 'directora general': 'A9920',
 'diseñadora gráfica': 'A9921',
 'em': 'A9922',
 'empleada': 'A9923',
 'empleada de limpieza': 'A9924',
 'empleada lavandería': 'A9925',
 'empleada tienda neto': 'A9926',
 'en farmacia': 'A9927',
 'estudiando': 'A9928',
 'estudiando otra carrera': 'A9929',
 'evaciones psicologicas': 'A9930',
 'farmacologia': 'A9931',
 'homeopatia': 'A9932',
 'ingeniera en sistemas y computacion': 'A9933',
 'ingenie

In [442]:
encuesta['trabajo'].replace(mapeo_reorganizado_importado,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

0                     A0203
1                     A0205
2                     A0209
3                     A0402
4                     A0403
               ...         
147     tienda de abarrotes
148    vendedor de catalogo
149               vendedora
150           venta de ropa
151        venta de seguros
Name: trabajo, Length: 152, dtype: string

In [443]:
encuesta[['Trabajas[Madre]','trabajo', 'categoria']]

,Trabajas[Madre],trabajo,categoria
0,Y,A9921,A99
1,Y,A1310,A13
2,Y,clases de danza,A99
3,N,A9921,A99
4,N,A9921,A99
...,...,...,...
699,N,A9921,A99
700,N,A9921,A99
701,Y,A1319,A13
702,Y,A1401,A14


In [444]:
#Renombrar las columnas
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[Madre]', 'categoria': 'Categoria[Madre]'})


In [445]:
encuesta.iloc[341, encuesta.columns.get_loc('Categoria[Madre]')] ='A99'

In [446]:
encuesta[['Trabajas[Madre]','Trabajo[Madre]', 'Categoria[Madre]']]

,Trabajas[Madre],Trabajo[Madre],Categoria[Madre]
0,Y,A9921,A99
1,Y,A1310,A13
2,Y,clases de danza,A99
3,N,A9921,A99
4,N,A9921,A99
...,...,...,...
699,N,A9921,A99
700,N,A9921,A99
701,Y,A1319,A13
702,Y,A1401,A14


In [447]:
#encuesta.to_csv('zoom2_limpio(prueba).csv', index=False)

In [448]:
#import json

#trabajos = { ... }

#with open("trabajos(madre).json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones, archivo)

# Trabajos Padre

## Limpieza de nivel 1

In [449]:
from auxiliar_2 import *

In [450]:
mostrar_unicos(encuesta,"q0r977")

0     -oth-
1        10
2        11
3        12
4        13
5        14
6        15
7        16
8        17
9        18
10       19
11        2
12       20
13       21
14       22
15        3
16        4
17        5
18        6
19        7
20        9
21      NaN
Name: q0r977, dtype: object

In [451]:
mapeo_respuestas1 = {
    '2':'Actividades deportivas_1',
    '3':'Artista_2',
    '4':'Chofer_3',
    '5':'Comerciante sector formal_4',
    '6':'Comerciante sector informal_5',
    '7':'Empresario_6',
    '8':' Comercio Virtual_7',
    '9': 'Atención a clientes_8',
    '10':'Administrativo_9',
    '11':'Gubernamental administrativo_10',
    '12':'Empleado gubernamental_11',
    '13':'Operador de maquinaria móvil_12',
    '14':'Personal técnico_13',
    '15':'Área educativa_14',
    '16':'Profesionista independiente_15',
    '17':'Seguridad_16',
    '18':'Área de la salud_17',
    '19':'Construcción_18',
    '20':'Fábrica_19',
    '21':'Agricultura_20',
    '22':'No trabaja/ trabajaba_21'
}

In [452]:
llenar_respuesta_libre(encuesta,"q0r977", identificador_otro='-oth-', sufijo='[other]')

In [453]:
llenar_vacios(encuesta,"q0r977",'ninguna')
encuesta['q0r977'] = encuesta['q0r977'].str.strip().str.lower()

In [454]:
encuesta['q0r977'] = encuesta['q0r977'].replace(mapeo_respuestas1)
#No correr de nuevo

In [455]:
mostrar_unicos(encuesta,"q0r977")

0      Actividades deportivas_1
1              Administrativo_9
2                Agricultura_20
3                     Artista_2
4         Atención a clientes_8
                 ...           
108               valet parking
109                      ventas
110                vulcanizador
111         Área de la salud_17
112           Área educativa_14
Name: q0r977, Length: 113, dtype: object

## Limpieza del nivel 2
Primeramente lo que se realizará será buscar las columnas base para los trabajos de mamá y posteriormente combinarlas.

### Columnas base

In [456]:
columnas_base1 = ['deporte','artista76','cho0000','formal0000','FOrla0000','empre00000','virtual0000','clientes552','admi0000','gub0000','emgub0000','maquina0000','tcnik','eduk4','Indepen0000','segurida0000','salud0000','construccion3000','fab56r','agriculta000']
columnas_base1

['deporte',
 'artista76',
 'cho0000',
 'formal0000',
 'FOrla0000',
 'empre00000',
 'virtual0000',
 'clientes552',
 'admi0000',
 'gub0000',
 'emgub0000',
 'maquina0000',
 'tcnik',
 'eduk4',
 'Indepen0000',
 'segurida0000',
 'salud0000',
 'construccion3000',
 'fab56r',
 'agriculta000']

In [457]:
len(columnas_base1)

20

In [458]:
encuesta[columnas_base1]

,deporte,artista76,cho0000,formal0000,FOrla0000,empre00000,virtual0000,clientes552,admi0000,gub0000,emgub0000,maquina0000,tcnik,eduk4,Indepen0000,segurida0000,salud0000,construccion3000,fab56r,agriculta000
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN
700,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A3,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [459]:
encuesta['deporte'] = encuesta['deporte'].replace({'A2': 'Entrenador de gimnasio_1', 'A3': 'Profesor de clases deportivas_2'})
encuesta['artista76'] = encuesta['artista76'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2','A4': 'Músico_3', 'A5':'Cantante_4'})
encuesta['cho0000'] = encuesta['cho0000'].replace({'A2': 'Taxista_1', 'A3': 'Uber_2','A4': 'Microbús_3', 'A5':'Camiones_4'})

In [460]:
encuesta['FOrla0000'] = encuesta['FOrla0000'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2','A4':'Flores y productos en un puesto no establecido_3'})

In [461]:
encuesta['formal0000'] = encuesta['formal0000'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})
encuesta['FOrla0000'] = encuesta['FOrla0000'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2'})
encuesta['empre00000'] = encuesta['empre00000'].replace({'A2': 'Dueño_1','A3':'Director o gerente_2'})
encuesta['clientes552'] = encuesta['clientes552'].replace({'A5': 'Promotor de seguros_4','A4':'Vendedor de tiendas departamentales_3'})
encuesta['admi0000'] = encuesta['admi0000'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})

In [462]:
encuesta['gub0000'] = encuesta['gub0000'].replace({'A2': 'Jefe de departamento_1','A3':'Jefe de área, subdirector_2','A4':'Subadministrador_3','A5':' Director_4','A6':'Administrador_5','A7':'Director general_6','A8':'Administrador general_7'})

encuesta['eduk4'] = encuesta['eduk4'].replace({'A2': 'Investigador-profesor en universidad/preparatoria_1','A3':'Educación básica_2','A4':'Técnico en educación (educador o guardería)_3'})

encuesta['Indepen0000'] = encuesta['Indepen0000'].replace({'A2': 'Abogado_1','A3':'Psicólogo_2','A4':'Contador_3','A7':' Terapeuta_6'})
encuesta['segurida0000'] = encuesta['segurida0000'].replace({'A2': 'Guardaespaldas_1','A3':'Patrullero_2','A4':'Vigilante_3','A5':'Empleado de seguridad en bancos_4','A6':'Empleado de seguridad privada_5','A7':'Policía de tránsito_6'})


In [463]:
encuesta['salud0000'] = encuesta['salud0000'].replace({'A2': 'Médico_1','A3':'Enfermera_2','A4':'Camillero_3','A5':'Paramédico_4'})
encuesta['construccion3000'] = encuesta['construccion3000'].replace({'A2': 'Arquitecto_1','A3':'Ingeniero civil/albañil o trabajador general_2'})

encuesta['fab56r'] = encuesta['fab56r'].replace({'A2':'Empleado_1'})
encuesta['agriculta000'] = encuesta['agriculta000'].replace({'A2':'Agricultor_1','A3':'Granjero_2','A4':'Siembra_3'})

In [464]:
encuesta['tcnik'] = encuesta['tcnik[comment]']

In [465]:
#Esto se hace para sustituir los valores de los cuáles dicen '¿qué tipo'
encuesta['maquina0000']=encuesta['maquina0000[comment]']
encuesta['emgub0000']=encuesta['emgub0000[comment]']

In [466]:
encuesta[columnas_base1]

,deporte,artista76,cho0000,formal0000,FOrla0000,empre00000,virtual0000,clientes552,admi0000,gub0000,emgub0000,maquina0000,tcnik,eduk4,Indepen0000,segurida0000,salud0000,construccion3000,fab56r,agriculta000
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Dueño_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Microbús_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
700,NaN,NaN,NaN,NaN,Vendedor de comida_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ingeniero civil/albañil o trabajador general_2,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### columnas con sufijos
El objetivo es pasar los datos de las columnas con sufijo a las columnas base.
No todas las columnas tienen el mismo sufijo, pero las dos opciones son `"[other]"` y `"[comment]"`:

In [467]:
columnas_sufijo1 = []
for col in columnas_base1:
    if col + '[other]' in encuesta.columns:
        columnas_sufijo1.append(col + '[other]')
    elif col + '[comment]' in encuesta.columns:
        columnas_sufijo1.append(col + '[comment]')
assert len(columnas_sufijo1) == len(columnas_base1)-1
columnas_sufijo1.sort()
columnas_sufijo1

['FOrla0000[other]',
 'Indepen0000[other]',
 'admi0000[other]',
 'agriculta000[other]',
 'artista76[other]',
 'cho0000[other]',
 'clientes552[other]',
 'construccion3000[other]',
 'deporte[other]',
 'eduk4[other]',
 'emgub0000[comment]',
 'empre00000[other]',
 'fab56r[other]',
 'gub0000[other]',
 'maquina0000[comment]',
 'salud0000[other]',
 'segurida0000[other]',
 'tcnik[comment]',
 'virtual0000[other]']

In [468]:
for col_con_sufijo in columnas_sufijo1:
    columna_sin_sufijo1 = re.sub(r'\[\w+\]', '', col_con_sufijo)
    encuesta[columna_sin_sufijo1] = encuesta[columna_sin_sufijo1].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo1] = np.where(encuesta[columna_sin_sufijo1] == '-oth-',
                                            encuesta[col_con_sufijo],
                                            encuesta[columna_sin_sufijo1])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [469]:
encuesta[columnas_base1]

,deporte,artista76,cho0000,formal0000,FOrla0000,empre00000,virtual0000,clientes552,admi0000,gub0000,emgub0000,maquina0000,tcnik,eduk4,Indepen0000,segurida0000,salud0000,construccion3000,fab56r,agriculta000
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INGENIERO EN SISTEMAS,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Dueño_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GERENTE SUCURSAL DE BANCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Microbús_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
700,NaN,NaN,NaN,NaN,Vendedor de comida_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ingeniero civil/albañil o trabajador general_2,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asignación de Trabajo

In [470]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)  # columna nueva
for columna in columnas_base1:
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['q0r977'])
encuesta.drop(columns='q0r977', inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

0                                           abogado
1                                         abogado_1
2                                          actuario
3                         administrador de escuelas
4                                   administrador_2
                           ...                     
262                                          ventas
263                                   vice director
264    vicepresidente de comisión nacional bancaria
265                                     vigilante_3
266                                    vulcanizador
Name: trabajo, Length: 267, dtype: string

In [471]:
trabajos_nuevos1 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos1

0           ingeniero en sistemas
1       gerente sucursal de banco
2                      no se sabe
3      productos para desinfectar
4                           no se
                  ...            
292                        sastre
293                 bibliotecario
294                        tienda
295                    trabajador
296                          juez
Name: trabajo, Length: 297, dtype: string

In [472]:
encuesta['trabajo']

0                                             ninguna
1                               ingeniero en sistemas
2                                             dueño_1
3                           gerente sucursal de banco
4                                          microbús_3
                            ...                      
699                                        empleado_1
700                              vendedor de comida_1
701    ingeniero civil/albañil o trabajador general_2
702                                              juez
703                                       enfermera_2
Name: trabajo, Length: 704, dtype: string

### Corrección de trabajos

In [473]:
trabajos_oficiales

0                         entrenador
1                         instructor
2      profesor de clases deportivas
3                              actor
4                             ballet
                   ...              
119                           sastre
120                         zapatero
121                         zapatero
122                         mecánico
123                          ninguna
Length: 124, dtype: object

In [474]:
correccion1 = match_most_similar(trabajos_oficiales,
                                trabajos_nuevos1,
                                min_similarity=.42,
                                ignore_index=True)
comparar1 = pd.DataFrame({'original': trabajos_nuevos1, 'correccion': correccion1})
comparar1['hubo_correccion'] = comparar['original'].ne(comparar1['correccion'])
comparar1.sort_values(by='hubo_correccion', ascending=False)

TypeError: __new__() got an unexpected keyword argument 'ignore_index'

In [475]:
mapeo_correcciones1 = dict(zip(trabajos_nuevos1, correccion1))
mapeo_correcciones1

NameError: name 'correccion1' is not defined

In [476]:
len(mapeo_correcciones1)

NameError: name 'mapeo_correcciones1' is not defined

In [477]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()

In [478]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip()

In [479]:
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones1)

NameError: name 'mapeo_correcciones1' is not defined

In [480]:
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta,'trabajo')

0                                           abogado
1                                          actuario
2                                     administrador
3                         administrador de escuelas
4                                    administrativo
                           ...                     
258                                          ventas
259                                   vice director
260    vicepresidente de comisión nacional bancaria
261                                       vigilante
262                                    vulcanizador
Name: trabajo, Length: 263, dtype: string

In [481]:
encuesta['trabajo']

0                                           ninguna
1                             ingeniero en sistemas
2                                             dueño
3                         gerente sucursal de banco
4                                          microbús
                           ...                     
699                                        empleado
700                              vendedor de comida
701    ingeniero civil/albañil o trabajador general
702                                            juez
703                                       enfermera
Name: trabajo, Length: 704, dtype: string

### Modificar algunos sinónimos
Esta parte se realiza manualmente debido a las limitaciones de la función match_most_similar y a la existencia de sinónimos que pueden condensarse en la propuesta.json

In [482]:
encuesta.loc[461, 'trabajo']

'miselanea'

In [483]:
encuesta.iloc[112, encuesta.columns.get_loc('trabajo')] = 'biólogo'
encuesta.iloc[201, encuesta.columns.get_loc('trabajo')] = 'diseñador gráfico'
encuesta.iloc[272, encuesta.columns.get_loc('trabajo')] = 'dueño'
encuesta.iloc[399, encuesta.columns.get_loc('trabajo')] = 'dueño'
encuesta.iloc[111, encuesta.columns.get_loc('trabajo')] = 'economista'
encuesta.iloc[267, encuesta.columns.get_loc('trabajo')] = 'electricista'
encuesta.iloc[610, encuesta.columns.get_loc('trabajo')] = 'electrónico'
encuesta.iloc[424, encuesta.columns.get_loc('trabajo')] = 'chofer'
encuesta.iloc[642, encuesta.columns.get_loc('trabajo')] = 'chofer'
encuesta.iloc[348, encuesta.columns.get_loc('trabajo')] = 'repartidor'
encuesta.iloc[492, encuesta.columns.get_loc('trabajo')] = 'federal'
encuesta.iloc[203, encuesta.columns.get_loc('trabajo')] = "flores y productos en un puesto no establecido"
encuesta.iloc[674, encuesta.columns.get_loc('trabajo')] = "flores y productos en un puesto no establecido"
encuesta.iloc[3, encuesta.columns.get_loc('trabajo')] = "gerente"
encuesta.iloc[393, encuesta.columns.get_loc('trabajo')] = "ingeniero"
encuesta.iloc[173, encuesta.columns.get_loc('trabajo')] = "investigador-profesor en universidad/preparatoria"
encuesta.iloc[289, encuesta.columns.get_loc('trabajo')] = "vendedor"
encuesta.iloc[581, encuesta.columns.get_loc('trabajo')] = "vendedor"
encuesta.iloc[27, encuesta.columns.get_loc('trabajo')] = "vendedor"
encuesta.iloc[636, encuesta.columns.get_loc('trabajo')] = "vendedor"
encuesta.iloc[331, encuesta.columns.get_loc('trabajo')] = "vendedor"
encuesta.iloc[62, encuesta.columns.get_loc('trabajo')] = "vendedor de comida"
encuesta.iloc[461, encuesta.columns.get_loc('trabajo')] = "abarrotes"

In [484]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip()

### Creación de categorías

In [485]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion ingeniero en sistemas es desconocida
La ocupacion no se sabe es desconocida
La ocupacion productos para desinfectar es desconocida
La ocupacion no se es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion técnico en hemodinamia es desconocida
La ocupacion productos de belleza es desconocida
La ocupacion no trabaja/ trabajaba es desconocida
La ocupacion director o gerente es desconocida
La ocupacion director o gerente es desconocida
La ocupacion director o gerente es desconocida
La ocupacion ingeniero industrial - empresa es desconocida
La ocupacion ingeniero civil/albañil o trabajador general es desconocida
La ocupacion economista es desconocida
La ocupacion sector bancario es desconocida
La ocupacion director o gerente es desconocida
La ocupacion comerciante sector formal es desconocida
La ocupacion banco es desconocida
La ocupacion estacion de radio es desconocida
La ocupacion tecnico farmacobiologo es desconocida
La ocupacion director o gerente es

In [486]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0     actividades elementales y de apoyo_20
1                          administrativo_9
2                            agricultura_19
3                                 artista_2
4                     atención a clientes_8
5                                  chofer_3
6               comerciante sector formal_5
7             comerciante sector informal_4
8                           construcción_17
9                            desconocido_99
10                empleado gubernamental_11
11                             empresario_6
12                               fábrica_18
13          gubernamental administrativo_10
14                               oficios_21
15           profesionista independiente_14
16                             seguridad_15
17                      área de la salud_16
18                        área educativa_13
dtype: string

In [487]:
categorias.value_counts()

desconocido_99                           326
comerciante sector informal_4             57
profesionista independiente_14            54
empresario_6                              43
área educativa_13                         33
fábrica_18                                30
chofer_3                                  28
empleado gubernamental_11                 25
área de la salud_16                       18
seguridad_15                              16
comerciante sector formal_5               15
administrativo_9                          15
oficios_21                                14
actividades elementales y de apoyo_20      7
agricultura_19                             7
artista_2                                  6
construcción_17                            4
atención a clientes_8                      4
gubernamental administrativo_10            2
Name: count, dtype: Int64

In [488]:
encuesta['categoria'] = categorias

In [489]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,ingeniero en sistemas,desconocido_99
2,dueño,empresario_6
3,gerente,empresario_6
4,microbús,chofer_3
...,...,...
699,empleado,fábrica_18
700,vendedor de comida,comerciante sector formal_5
701,ingeniero civil/albañil o trabajador general,desconocido_99
702,juez,desconocido_99


### Asignación de la columna "Trabajas[Padre]"
Esta columna se crea para corregir los datos de zoom 2, donde no se realiza la pregunta: ¿En qué trabajas".La creación de esta columna nos ayudará a poder limpiar aquellos trabajos que esten nombrados como: "no trabaja/trabajaba".

In [490]:
encuesta['Trabajas[Padre]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] == 'no trabaja/ trabajaba':
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[Padre]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[Padre]'] = 'Y'

In [491]:
#Esta función nos ayudará a desplazar la columna "Trabajas"
column_order = list(encuesta.columns)

# Reordenar la lista de columnas para que 'Trabajas[Madre]' esté antes de 'Trabajas'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[Padre]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [492]:
encuesta[['Trabajas[Padre]','trabajo', 'categoria']]

,Trabajas[Padre],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,ingeniero en sistemas,desconocido_99
2,Y,dueño,empresario_6
3,Y,gerente,empresario_6
4,Y,microbús,chofer_3
...,...,...,...
699,Y,empleado,fábrica_18
700,Y,vendedor de comida,comerciante sector formal_5
701,Y,ingeniero civil/albañil o trabajador general,desconocido_99
702,Y,juez,desconocido_99


In [493]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/ trabajaba', 'ninguna')

### Últimas modificaciones sobre categorías

In [494]:
encuesta.iloc[659, encuesta.columns.get_loc('trabajo')] = "bibliotecario"
encuesta.iloc[689, encuesta.columns.get_loc('trabajo')] = "bibliotecario"
encuesta.iloc[565, encuesta.columns.get_loc('trabajo')] = "vicedirector"
encuesta.iloc[358, encuesta.columns.get_loc('trabajo')] = "vendedor"
encuesta.iloc[358, encuesta.columns.get_loc('categoria')] = "comerciante sector informal_4"
encuesta.iloc[489, encuesta.columns.get_loc('trabajo')] = "sastre"
encuesta.iloc[489, encuesta.columns.get_loc('categoria')] = "oficios_21"


In [495]:
encuesta.iloc[440, encuesta.columns.get_loc('trabajo')] = "transporte de materiales"
encuesta.iloc[168, encuesta.columns.get_loc('trabajo')] = "notario público"
encuesta.iloc[177, encuesta.columns.get_loc('trabajo')] = "notario público"
encuesta.iloc[639, encuesta.columns.get_loc('trabajo')] = "árbitro"
encuesta.iloc[303, encuesta.columns.get_loc('trabajo')] = "chofer"
encuesta.iloc[119, encuesta.columns.get_loc('trabajo')] = "ine"
encuesta.iloc[662, encuesta.columns.get_loc('trabajo')] = "diseñador gráfico"
encuesta.iloc[662, encuesta.columns.get_loc('categoria')] = "artista_2"

In [496]:
encuesta.iloc[331, encuesta.columns.get_loc('trabajo')] = "empleado en seguridad privada"
encuesta.iloc[311, encuesta.columns.get_loc('trabajo')] = "empleado en seguridad privada"
encuesta.iloc[515, encuesta.columns.get_loc('trabajo')] = "empleado en seguridad privada"
encuesta.iloc[229, encuesta.columns.get_loc('trabajo')] = "empleado en seguridad privada"
encuesta.iloc[474, encuesta.columns.get_loc('trabajo')] = "empleado"
encuesta.iloc[474, encuesta.columns.get_loc('categoria')] = "fábrica_18"

In [497]:
encuesta.iloc[646, encuesta.columns.get_loc('trabajo')] = "mecánico"
encuesta.iloc[633, encuesta.columns.get_loc('trabajo')] = "electromecánico"
encuesta.iloc[285, encuesta.columns.get_loc('trabajo')] = "mecánico"
encuesta.iloc[287, encuesta.columns.get_loc('trabajo')] = "almacenista"
encuesta.iloc[53, encuesta.columns.get_loc('trabajo')] = "cobrador"
encuesta.iloc[612, encuesta.columns.get_loc('trabajo')] = "lavandería"

In [498]:
encuesta.iloc[373, encuesta.columns.get_loc('trabajo')] = "oficinista"
encuesta.iloc[367, encuesta.columns.get_loc('trabajo')] = "oficinista"
encuesta.iloc[497, encuesta.columns.get_loc('trabajo')] = "químico farmacobiólogo"
encuesta.iloc[204, encuesta.columns.get_loc('trabajo')] = "radiólogo"
encuesta.iloc[220, encuesta.columns.get_loc('trabajo')] = "médico radiólogo"
encuesta.iloc[462, encuesta.columns.get_loc('trabajo')] = "representante médico"
encuesta.iloc[230, encuesta.columns.get_loc('trabajo')] = "médico veterinario"

In [499]:
encuesta.iloc[336, encuesta.columns.get_loc('trabajo')] = "técnico en calefacción"
encuesta.iloc[471, encuesta.columns.get_loc('trabajo')] = "técnico en reparaciones"
encuesta.iloc[46, encuesta.columns.get_loc('trabajo')] = "técnico farmacobiólogo"
encuesta.iloc[364, encuesta.columns.get_loc('trabajo')] = "técnico en biofumigación"


In [500]:
encuesta.iloc[441, encuesta.columns.get_loc('trabajo')] = "financiero"
encuesta.iloc[582, encuesta.columns.get_loc('trabajo')] = "financiero"
encuesta.iloc[376, encuesta.columns.get_loc('trabajo')] = "financiero"
encuesta.iloc[32, encuesta.columns.get_loc('trabajo')] = "financiero"
encuesta.iloc[242, encuesta.columns.get_loc('trabajo')] = "monitorista en medios de comunicación"
encuesta.iloc[208, encuesta.columns.get_loc('trabajo')] = "chofer"
encuesta.iloc[194, encuesta.columns.get_loc('trabajo')] = "trabajador en el metro"
encuesta.iloc[237, encuesta.columns.get_loc('trabajo')] = "trabajador en el metro"

In [501]:
ocupaciones_especificas = ["bibliotecario", "vicedirector", "pedagogía"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_especificas), 'categoria'] = "área educativa_13"

In [502]:
# Asigna la categoría correspondiente a las ocupaciones específicas
ocupaciones_formal = ["comerciante sector formal"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal","uniformes escolares","refacciones","purificadora","negocio reciclaje","productos de belleza","productos para desinfectar"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [503]:
ocupaciones_admin = ["banco"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_admin), 'categoria'] = "administrativo_9"

In [504]:
ocupaciones_guber = ["oficial mayor en sct","vicepresidente de comisión nacional bancaria","supervisor","servidor publico","gestor de transito","juez","notario público","perito","ine","programa pilares","pasaportes","jefe de área, subdirector","trabajador en el metro"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "gubernamental administrativo_10"

In [505]:
ocupaciones_empre = ["director o gerente","negocios"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empre), 'categoria'] = "empresario_6"

In [506]:
ocupaciones_seg = ["encargado de la planta","militar","guardia de seguridad","federal","empleado en seguridad privada"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_seg), 'categoria'] = "seguridad_15"

In [507]:
ocupaciones_cons = ["contratista","ingeniero civil/albañil o trabajador general"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cons), 'categoria'] = "construcción_17"

In [508]:
ocupaciones_dep = ["árbitro"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_dep), 'categoria'] = "actividades deportivas_1"

In [509]:
ocupaciones_chof = ["chofer","transporte de materiales","transportista","trailer","transporte colectivo metro","mudanzas","maneja un camión de basura"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_chof), 'categoria'] = "chofer_3"

In [510]:
ocupaciones_tec = ["técnico en calefacción","técnico en reparaciones","técnico farmacobiólogo","técnico en biofumigación","técnico en hemodinamia","monitorista en medios de comunicación"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_tec), 'categoria'] = "personal técnico_12"

In [511]:
ocupaciones_of = ["electrónico","mecánico","electromecánico","hojalatero","plomero","metalurgia","joyero grabador","almacenista","electricista","lavandería","imprenta","vulcanizador","valet parking","automotriz","cobrador","poligrafista"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"

In [512]:
ocupaciones_profin= ["oficinista","financiero","bienes raices","financiero","químico farmacobiólogo","programador","economista","actuario","revisor","representante de ventas","médico veterinario"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"

In [513]:
ocupaciones_saludd= ["radiólogo","médico radiólogo","representante médico"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"

In [514]:
ocupaciones_cliente= ["tienda","tienda de autoservicio","turismo","representante  de ventas"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"

In [515]:
ocupaciones_maquinaria= ["tienda","tienda de autoservicio","turismo","representante  de ventas"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_maquinaria), 'categoria'] = "atención a clientes_8"

In [516]:
ocupaciones_limp= ["afanador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_limp), 'categoria'] = "actividades elementales y de apoyo_20"

In [517]:
ocupaciones_fabrica=["operador de maquinaria móvil","maquinado industrial"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_fabrica), 'categoria'] = "fábrica_18"

In [518]:
encuesta[["trabajo","categoria"]]

,trabajo,categoria
0,ninguna,desconocido_99
1,ingeniero en sistemas,desconocido_99
2,dueño,empresario_6
3,gerente,empresario_6
4,microbús,chofer_3
...,...,...
699,empleado,fábrica_18
700,vendedor de comida,comerciante sector formal_5
701,ingeniero civil/albañil o trabajador general,construcción_17
702,juez,gubernamental administrativo_10


In [519]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           161
profesionista independiente_14            78
empresario_6                              67
comerciante sector informal_4             64
chofer_3                                  40
oficios_21                                39
área educativa_13                         37
fábrica_18                                33
empleado gubernamental_11                 25
seguridad_15                              25
construcción_17                           24
área de la salud_16                       21
administrativo_9                          19
gubernamental administrativo_10           18
comerciante sector formal_5               16
atención a clientes_8                      8
actividades elementales y de apoyo_20      8
artista_2                                  7
agricultura_19                             7
personal técnico_12                        6
actividades deportivas_1                   1
Name: count, dtype: Int64

### Resultado final

In [520]:
mapeo_codigos_categoria1 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria1[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria1

{'desconocido_99': 'A99',
 'empresario_6': 'A06',
 'chofer_3': 'A03',
 'comerciante sector informal_4': 'A04',
 'empleado gubernamental_11': 'A11',
 'comerciante sector formal_5': 'A05',
 'área educativa_13': 'A13',
 'profesionista independiente_14': 'A14',
 'construcción_17': 'A17',
 'fábrica_18': 'A18',
 'artista_2': 'A02',
 'área de la salud_16': 'A16',
 'gubernamental administrativo_10': 'A10',
 'administrativo_9': 'A09',
 'atención a clientes_8': 'A08',
 'oficios_21': 'A21',
 'seguridad_15': 'A15',
 'actividades elementales y de apoyo_20': 'A20',
 'agricultura_19': 'A19'}

In [521]:
mapeo_codigos_categoria1['personal técnico_12'] = 'A12'
mapeo_codigos_categoria1['actividades deportivas_1'] = 'A01'
mapeo_codigos_categoria1

{'desconocido_99': 'A99',
 'empresario_6': 'A06',
 'chofer_3': 'A03',
 'comerciante sector informal_4': 'A04',
 'empleado gubernamental_11': 'A11',
 'comerciante sector formal_5': 'A05',
 'área educativa_13': 'A13',
 'profesionista independiente_14': 'A14',
 'construcción_17': 'A17',
 'fábrica_18': 'A18',
 'artista_2': 'A02',
 'área de la salud_16': 'A16',
 'gubernamental administrativo_10': 'A10',
 'administrativo_9': 'A09',
 'atención a clientes_8': 'A08',
 'oficios_21': 'A21',
 'seguridad_15': 'A15',
 'actividades elementales y de apoyo_20': 'A20',
 'agricultura_19': 'A19',
 'personal técnico_12': 'A12',
 'actividades deportivas_1': 'A01'}

In [522]:
encuesta['categoria'].replace(mapeo_codigos_categoria1, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

0     A01
1     A02
2     A03
3     A04
4     A05
5     A06
6     A08
7     A09
8     A10
9     A11
10    A12
11    A13
12    A14
13    A15
14    A16
15    A17
16    A18
17    A19
18    A20
19    A21
20    A99
Name: categoria, dtype: string

In [523]:
mapeo_codigos_ocupaciones1 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria1[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones1[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones1

{'administrador de escuelas': 'A9901',
 'administrativo': 'A9902',
 'artesanía': 'A9903',
 'asesor financiero': 'A9904',
 'auxiliar administrativo': 'A9905',
 'camion de basura': 'A9906',
 'camioneta': 'A9907',
 'camioneta de carga': 'A9908',
 'camionetas 3 1/2': 'A9909',
 'carpinteria': 'A9910',
 'club': 'A9911',
 'cocina': 'A9912',
 'comercio tienda departamental': 'A9913',
 'consultor servicios de tecnologia': 'A9914',
 'contador privado': 'A9915',
 'contador publico': 'A9916',
 'costura': 'A9917',
 'departamento de limpias y transportes': 'A9918',
 'desconocido': 'A9919',
 'director de area': 'A9920',
 'diseñador gráfico': 'A0202',
 'dueño de restaurante': 'A9922',
 'empleado de almacen': 'A9923',
 'empleado de empresa': 'A9924',
 'empleado del gobierno': 'A9925',
 'empleado en la inea': 'A9926',
 'empleado en laboratorio': 'A9927',
 'empleado en una empresa': 'A9928',
 'empleado federal': 'A9929',
 'empleado general': 'A9930',
 'empleado negocio de comida': 'A9931',
 'empleado tal

In [524]:
mapeo_codigos_ocupaciones1["árbitro"] = "A0101"
mapeo_codigos_ocupaciones1["técnico en hemodinamia"] = "A1201"
mapeo_codigos_ocupaciones1["técnico farmacobiólogo"] = "A1202"
mapeo_codigos_ocupaciones1["técnico en reparaciones"] = "A1203"
mapeo_codigos_ocupaciones1["monitorista en medios de comunicación"] = "A1204"
mapeo_codigos_ocupaciones1["técnico en biofumigación"] = "A1205"
mapeo_codigos_ocupaciones1["técnico en calefacción"] = "A1206"
mapeo_codigos_ocupaciones1


{'administrador de escuelas': 'A9901',
 'administrativo': 'A9902',
 'artesanía': 'A9903',
 'asesor financiero': 'A9904',
 'auxiliar administrativo': 'A9905',
 'camion de basura': 'A9906',
 'camioneta': 'A9907',
 'camioneta de carga': 'A9908',
 'camionetas 3 1/2': 'A9909',
 'carpinteria': 'A9910',
 'club': 'A9911',
 'cocina': 'A9912',
 'comercio tienda departamental': 'A9913',
 'consultor servicios de tecnologia': 'A9914',
 'contador privado': 'A9915',
 'contador publico': 'A9916',
 'costura': 'A9917',
 'departamento de limpias y transportes': 'A9918',
 'desconocido': 'A9919',
 'director de area': 'A9920',
 'diseñador gráfico': 'A0202',
 'dueño de restaurante': 'A9922',
 'empleado de almacen': 'A9923',
 'empleado de empresa': 'A9924',
 'empleado del gobierno': 'A9925',
 'empleado en la inea': 'A9926',
 'empleado en laboratorio': 'A9927',
 'empleado en una empresa': 'A9928',
 'empleado federal': 'A9929',
 'empleado general': 'A9930',
 'empleado negocio de comida': 'A9931',
 'empleado tal

In [525]:
encuesta[["trabajo", "categoria"]]

,trabajo,categoria
0,ninguna,A99
1,ingeniero en sistemas,A99
2,dueño,A06
3,gerente,A06
4,microbús,A03
...,...,...
699,empleado,A18
700,vendedor de comida,A05
701,ingeniero civil/albañil o trabajador general,A17
702,juez,A10


In [526]:
encuesta['trabajo'].replace(mapeo_reorganizado_importado,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

0                                A0102
1                                A0203
2                                A0204
3                                A0205
4                                A0208
                    ...               
218    supervisor tienda departamental
219                          tablajero
220                tienda de abarrotes
221                         trabajador
222          vendedor de bienes raíces
Name: trabajo, Length: 223, dtype: string

In [527]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[Padre]', 'categoria': 'Categoria[Padre]'})

In [528]:
encuesta.iloc[563, encuesta.columns.get_loc('Categoria[Padre]')] ='A08'
encuesta.iloc[453, encuesta.columns.get_loc('Categoria[Padre]')] ='A08'
encuesta.iloc[457, encuesta.columns.get_loc('Categoria[Padre]')] ='A08'
encuesta.iloc[524, encuesta.columns.get_loc('Categoria[Padre]')] ='A08'
encuesta.iloc[90, encuesta.columns.get_loc('Categoria[Padre]')] ='A08'
encuesta.iloc[107, encuesta.columns.get_loc('Categoria[Padre]')] ='A08'

In [529]:
encuesta[['Trabajas[Padre]','Trabajo[Padre]', 'Categoria[Padre]']]

,Trabajas[Padre],Trabajo[Padre],Categoria[Padre]
0,Y,A9921,A99
1,Y,ingeniero en sistemas,A99
2,Y,A0607,A06
3,Y,A0610,A06
4,Y,A0306,A03
...,...,...,...
699,Y,A1801,A18
700,Y,A0504,A05
701,Y,A1707,A17
702,Y,A1006,A10


In [530]:
#import json

#trabajos = { ... }

#with open("trabajos(padre).json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones1, archivo)

# Trabajos Abuela Materna 

## Limpieza de Nivel 1

In [531]:
mostrar_unicos(encuesta,"q0r651")

0     -oth-
1       A10
2       A11
3       A12
4       A18
5       A19
6       A20
7       A21
8       A23
9       A24
10      A25
11       A3
12       A5
13       A6
14       A7
15       A9
16      NaN
Name: q0r651, dtype: object

In [532]:
mapeo_respuestas2 = {
    'a2':'Actividades deportivas_1',
    'a3':'Artista_2',
    'a4':'Chofer_3',
    'a5':'Comerciante sector formal_4',
    'a6':'Comerciante sector informal_5',
    'a7':'Empresario_6',
    'a8':' Comercio Virtual_7',
    'a9': 'Atención a clientes_8',
    'a10':'Administrativo_9',
    'a11':'Gubernamental administrativo_10',
    'a12':'Empleado gubernamental_11',
    'a16':'Operador de maquinaria móvil_12',
    'a17':'Personal técnico_13',
    'a18':'Área educativa_14',
    'a19':'Profesionista independiente_15',
    'a20':'Seguridad_16',
    'a21':'Área de la salud_17',
    'a22':'Construcción_18',
    'a23':'Fábrica_19',
    'a24':'Agricultura_20',
    'a25':'No trabaja/ trabajaba_21'
}

In [533]:
llenar_respuesta_libre(encuesta,"q0r651", identificador_otro='-oth-', sufijo='[other]')

In [534]:
llenar_vacios(encuesta,"q0r651",'ninguna')
encuesta['q0r651'] = encuesta['q0r651'].str.strip().str.lower()

In [535]:
encuesta['q0r651'] = encuesta['q0r651'].replace(mapeo_respuestas2)
#No correr de nuevo

In [536]:
mostrar_unicos(encuesta,"q0r651")

0                    Administrativo_9
1                      Agricultura_20
2                           Artista_2
3               Atención a clientes_8
4         Comerciante sector formal_4
5       Comerciante sector informal_5
6           Empleado gubernamental_11
7                        Empresario_6
8                          Fábrica_19
9     Gubernamental administrativo_10
10           No trabaja/ trabajaba_21
11     Profesionista independiente_15
12                       Seguridad_16
13                        ama de casa
14                       arrendatario
15                             cocina
16                            costura
17                          costurera
18                 educación especial
19                empacadora en super
20                   empada domestica
21                           empleada
22                  empleada de hogar
23                 empleada domestica
24                 empleada doméstica
25                 empleado domestico
26          

## Limpieza de nivel 2

### Columnas base

In [537]:
columnas_base2 = ['q0r899','art5','cho5','comerfor3','comerinf3','empresario34','virtual3','atclientes2','adm5','gubadm4','emplead3','OPERADORmaqui2','tecnico3','educati5','indepen5','seguri3','salud5','constru5','fabrica43','agricultura3']
columnas_base2

['q0r899',
 'art5',
 'cho5',
 'comerfor3',
 'comerinf3',
 'empresario34',
 'virtual3',
 'atclientes2',
 'adm5',
 'gubadm4',
 'emplead3',
 'OPERADORmaqui2',
 'tecnico3',
 'educati5',
 'indepen5',
 'seguri3',
 'salud5',
 'constru5',
 'fabrica43',
 'agricultura3']

In [538]:
len(columnas_base2)

20

In [539]:
encuesta[columnas_base2]

,q0r899,art5,cho5,comerfor3,comerinf3,empresario34,virtual3,atclientes2,adm5,gubadm4,emplead3,OPERADORmaqui2,tecnico3,educati5,indepen5,seguri3,salud5,constru5,fabrica43,agricultura3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [540]:
encuesta['q0r899'] = encuesta['q0r899'].replace({'A2': 'Entrenador de gimnasio_1', 'A3': 'Profesor de clases deportivas_2'})
encuesta['art5'] = encuesta['art5'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2','A4': 'Músico_3', 'A5':'Cantante_4'})
encuesta['cho5'] = encuesta['cho5'].replace({'A2': 'Taxista_1', 'A3': 'Uber_2','A4': 'Microbús_3', 'A5':'Camiones_4'})

In [541]:
encuesta['comerfor3'] = encuesta['comerfor3'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})
encuesta['comerinf3'] = encuesta['comerinf3'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2','A4':'Flores y productos en un puesto no establecido_3','A5':'Productos en un puesto no establecido'})


In [542]:
encuesta['empresario34'] = encuesta['empresario34'].replace({'A2': 'Dueño_1','A3':'Director o gerente_2'})
encuesta['atclientes2'] = encuesta['atclientes2'].replace({'A6': 'Recepcionista_2','A4':' Vendedor de tiendas departamentales/promotor de seguros, telefonía_3','A3':'Atención a clientes_1'})
encuesta['adm5'] = encuesta['adm5'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})

encuesta['gubadm4'] = encuesta['gubadm4'].replace({'A2': 'Jefe de departamento_1','A3':'Jefe de área_2','A4':'Subdirector_2','A5':' Subadministrador_3','A6':'Director_4','A7':'Administrador_5','A8':'Director general_6','A9':'Administrador general_7'})

encuesta['educati5'] = encuesta['educati5'].replace({'A2': 'Investigador-profesor en universidad/preparatoria_1','A3':'Educación básica_2','A4':'Técnico en educación (educador o guardería)_3'})

encuesta['indepen5'] = encuesta['indepen5'].replace({'A2': 'Abogado_1','A3':'Psicólogo_2','A4':'Contador_3','A7':' Terapeuta_6'})
encuesta['seguri3'] = encuesta['seguri3'].replace({'A2': 'Guardaespaldas_1','A3':'Patrullero_2','A4':'Vigilante_3','A5':'Empleado de seguridad en bancos_4','A6':'Empleado de seguridad privada_5','A7':'Policía de tránsito_6'})



In [543]:
encuesta['salud5'] = encuesta['salud5'].replace({'A2': 'Médico_1','A3':'Enfermera_2','A4':'Camillero_3','A5':'Paramédico_4'})
encuesta['fabrica43'] = encuesta['fabrica43'].replace({'A2':'Empleado_1'})
encuesta['emplead3'] = encuesta['emplead3'].replace({'A1':'¿DE QUÉ TIPO?_1'})
encuesta['agricultura3'] = encuesta['agricultura3'].replace({'A2':'Agricultor_1','A3':' Granjero_2','A4':'Siembra_3'})

In [544]:
encuesta[columnas_base2]

,q0r899,art5,cho5,comerfor3,comerinf3,empresario34,virtual3,atclientes2,adm5,gubadm4,emplead3,OPERADORmaqui2,tecnico3,educati5,indepen5,seguri3,salud5,constru5,fabrica43,agricultura3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Flores y productos en un puesto no establecido_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### columnas con sufijos
El objetivo es pasar los datos de las columnas con sufijo a las columnas base.
No todas las columnas tienen el mismo sufijo, pero las dos opciones son `"[other]"` y `"[comment]"`:

In [545]:
columnas_sufijo2 = []
for col in columnas_base2:
    if col + '[other]' in encuesta.columns:
        columnas_sufijo2.append(col + '[other]')
    elif col + '[comment]' in encuesta.columns:
        columnas_sufijo2.append(col + '[comment]')
assert len(columnas_sufijo2) == len(columnas_base2)
columnas_sufijo2.sort()
columnas_sufijo2

['OPERADORmaqui2[comment]',
 'adm5[other]',
 'agricultura3[other]',
 'art5[other]',
 'atclientes2[other]',
 'cho5[other]',
 'comerfor3[other]',
 'comerinf3[other]',
 'constru5[other]',
 'educati5[other]',
 'emplead3[comment]',
 'empresario34[other]',
 'fabrica43[other]',
 'gubadm4[other]',
 'indepen5[other]',
 'q0r899[other]',
 'salud5[other]',
 'seguri3[other]',
 'tecnico3[comment]',
 'virtual3[other]']

In [546]:
for col_con_sufijo in columnas_sufijo2:
    columna_sin_sufijo2 = re.sub(r'\[\w+\]', '', col_con_sufijo)
    encuesta[columna_sin_sufijo2] = encuesta[columna_sin_sufijo2].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo2] = np.where(encuesta[columna_sin_sufijo2] == '-oth-',
                                             encuesta[col_con_sufijo],
                                             encuesta[columna_sin_sufijo2])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [547]:
encuesta[columnas_base2]

,q0r899,art5,cho5,comerfor3,comerinf3,empresario34,virtual3,atclientes2,adm5,gubadm4,emplead3,OPERADORmaqui2,tecnico3,educati5,indepen5,seguri3,salud5,constru5,fabrica43,agricultura3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Flores y productos en un puesto no establecido_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asignación de Trabajo

In [548]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)  # columna nueva
for columna in columnas_base2:
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['q0r651'])
encuesta.drop(columns='q0r651', inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

0              abarrotes
1        administrador_2
2        administrador_5
3       administrativo_9
4           agricultor_1
             ...        
118             verduras
119          vigilante_3
120           voluntaria
121      ¿de qué tipo?_1
122    área educativa_14
Name: trabajo, Length: 123, dtype: string

In [549]:
trabajos_nuevos2 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos2

0                 laboratorista
1             vendedora de ropa
2                         hogar
3                     psicologa
4                   ama de casa
                 ...           
203    limpieza casa habitacion
204                    limpieza
205                 trabajadora
206           empleada de hogar
207                       hogar
Name: trabajo, Length: 208, dtype: string

In [550]:
encuesta['trabajo']

0                                               ninguna
1                              no trabaja/ trabajaba_21
2                                      administrativo_9
3                              no trabaja/ trabajaba_21
4      flores y productos en un puesto no establecido_3
                             ...                       
699                                   empleada de hogar
700                            no trabaja/ trabajaba_21
701                                          empleado_1
702                                               hogar
703                                        agricultor_1
Name: trabajo, Length: 704, dtype: string

### Corrección de Trabajos

In [551]:
trabajos_oficiales

0                         entrenador
1                         instructor
2      profesor de clases deportivas
3                              actor
4                             ballet
                   ...              
119                           sastre
120                         zapatero
121                         zapatero
122                         mecánico
123                          ninguna
Length: 124, dtype: object

In [552]:
tesauro_ama_de_casa = {
    "trabajadores domésticos": ["ama de casa", "AMA DE CASA", "ASEO DOMESTICO", "EMPLEADA DOMESTICA",
                                "empleada doméstica", "Empleada doméstica", "HOGAR", "hogar", "casa",
                                "limpieza en casa ajena", "Ama de casa", "Hogar", "trabajadora doméstica",
                                "trabajadora domestica", "Ama de trabajadores domésticos", "empleada domestica",
                                "Limpieza en trabajadores domésticos ajena", "trabajó en trabajadores domésticos",
                                "limpieza en trabajadores domésticos ajena",
                                "empleada de trabajadores domésticos", "empleado domestico", "empleado domestica",
                                "trabajó en trabajadores domésticos", "empleada de trabajadores domésticos",
                                "trabajadores de limpiezaes de trabajadores de limpiezaes de limpieza",
                                "trabajadores de limpiezaes domésticos"]}

In [553]:
for palabra, sinonimos in tesauro_ama_de_casa.items():
    encuesta['trabajo'] = encuesta['trabajo'].replace(sinonimos, palabra, regex=True)

In [554]:
correccion2 = match_most_similar(trabajos_oficiales,
                                 trabajos_nuevos2,
                                 min_similarity=.42,
                                 ignore_index=True)
comparar2 = pd.DataFrame({'original': trabajos_nuevos2, 'correccion': correccion2})
comparar2['hubo_correccion'] = comparar2['original'].ne(comparar2['correccion'])
comparar2.sort_values(by='hubo_correccion', ascending=False)

TypeError: __new__() got an unexpected keyword argument 'ignore_index'

In [555]:
mapeo_correcciones2 = dict(zip(trabajos_nuevos2, correccion2))
mapeo_correcciones2

NameError: name 'correccion2' is not defined

In [556]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()
encuesta['trabajo'] = encuesta['trabajo'].str.strip()
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones2)
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta, 'trabajo')

NameError: name 'mapeo_correcciones2' is not defined

In [557]:
encuesta['trabajo']

0                                               ninguna
1                              no trabaja/ trabajaba_21
2                                      administrativo_9
3                              no trabaja/ trabajaba_21
4      flores y productos en un puesto no establecido_3
                             ...                       
699                 empleada de trabajadores domésticos
700                            no trabaja/ trabajaba_21
701                                          empleado_1
702                             trabajadores domésticos
703                                        agricultor_1
Name: trabajo, Length: 704, dtype: string

### Modificar algunos sinónimos

In [558]:
encuesta.loc[552, 'trabajo']

'trabajó en trabajadores domésticos'

In [559]:
encuesta.iloc[205, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'
encuesta.iloc[552, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'
encuesta.iloc[699, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'
encuesta.iloc[356, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'
encuesta.iloc[624, encuesta.columns.get_loc('trabajo')] = 'trabajadores domésticos'
encuesta.iloc[685, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[644, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[632, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[654, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[671, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[592, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'

### Creación de Categorías

In [560]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion administrativo_9 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion flores y productos en un puesto no establecido_3 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion vendedor de comida_1 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion laboratorista es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion vendedora de ropa es desconocida
La ocupacion psicologa es desconocida
La ocupacion médico_1 es desconocida
La ocupacion técnico en educación (educador o guardería)_3 es desconocida
La ocupacion educación básica_2 es desconocida
La ocupacion no

In [561]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0    actividades elementales y de apoyo_20
1                         administrativo_9
2              comerciante sector formal_5
3            comerciante sector informal_4
4                           desconocido_99
5                             empresario_6
6                               oficios_21
7           profesionista independiente_14
8                        área educativa_13
dtype: string

In [562]:
categorias.value_counts()

desconocido_99                           574
actividades elementales y de apoyo_20    105
comerciante sector formal_5               10
administrativo_9                           8
comerciante sector informal_4              3
empresario_6                               1
oficios_21                                 1
profesionista independiente_14             1
área educativa_13                          1
Name: count, dtype: Int64

In [563]:
encuesta['categoria'] = categorias

In [564]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,no trabaja/ trabajaba_21,desconocido_99
2,administrativo_9,desconocido_99
3,no trabaja/ trabajaba_21,desconocido_99
4,flores y productos en un puesto no establecido_3,desconocido_99
...,...,...
699,trabajadores domésticos,actividades elementales y de apoyo_20
700,no trabaja/ trabajaba_21,desconocido_99
701,empleado_1,desconocido_99
702,trabajadores domésticos,actividades elementales y de apoyo_20


### Asignación de la columna "Trabajas[AbuelaMaterna]"
Esta columna se crea para corregir los datos de zoom 2, donde no se realiza la pregunta: ¿En qué trabajas".La creación de esta columna nos ayudará a poder limpiar aquellos trabajos que esten nombrados como: "no trabaja/trabajaba".

In [565]:
encuesta['Trabajas[AbuelaMaterna]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] == 'no trabaja/ trabajaba':
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbuelaMaterna]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbuelaMaterna]'] = 'Y'

In [566]:
#Esta función nos ayudará a desplazar la columna "Trabajas"
column_order = list(encuesta.columns)

# Reordenar la lista de columnas para que 'Trabajas[Madre]' esté antes de 'Trabajas'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[AbuelaMaterna]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [567]:
encuesta[['Trabajas[AbuelaMaterna]','trabajo', 'categoria']]

,Trabajas[AbuelaMaterna],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,no trabaja/ trabajaba_21,desconocido_99
2,Y,administrativo_9,desconocido_99
3,Y,no trabaja/ trabajaba_21,desconocido_99
4,Y,flores y productos en un puesto no establecido_3,desconocido_99
...,...,...,...
699,Y,trabajadores domésticos,actividades elementales y de apoyo_20
700,Y,no trabaja/ trabajaba_21,desconocido_99
701,Y,empleado_1,desconocido_99
702,Y,trabajadores domésticos,actividades elementales y de apoyo_20


In [568]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/ trabajaba', 'ninguna')

In [569]:
encuesta[['Trabajas[AbuelaMaterna]','trabajo', 'categoria']]

,Trabajas[AbuelaMaterna],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,no trabaja/ trabajaba_21,desconocido_99
2,Y,administrativo_9,desconocido_99
3,Y,no trabaja/ trabajaba_21,desconocido_99
4,Y,flores y productos en un puesto no establecido_3,desconocido_99
...,...,...,...
699,Y,trabajadores domésticos,actividades elementales y de apoyo_20
700,Y,no trabaja/ trabajaba_21,desconocido_99
701,Y,empleado_1,desconocido_99
702,Y,trabajadores domésticos,actividades elementales y de apoyo_20


### Últimas modificaciones sobre categorías

In [570]:
encuesta.loc[207, 'trabajo']

'¿de qué tipo?_1'

In [571]:
encuesta.iloc[682, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[239, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[467, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[145, encuesta.columns.get_loc('trabajo')] = 'ropa'
encuesta.iloc[279, encuesta.columns.get_loc('trabajo')] = 'ropa'
encuesta.iloc[45, encuesta.columns.get_loc('trabajo')] = 'vendedor'
encuesta.iloc[461, encuesta.columns.get_loc('trabajo')] = 'abarrotes'
encuesta.iloc[452, encuesta.columns.get_loc('trabajo')] = 'peluquero'
encuesta.iloc[359, encuesta.columns.get_loc('trabajo')] = 'trabaja en pemex'
encuesta.iloc[118, encuesta.columns.get_loc('trabajo')] = 'comerciante sector informal'
encuesta.iloc[155, encuesta.columns.get_loc('trabajo')] = 'directora del área de enfermería de los hospitales de gobierno'

In [572]:
encuesta.iloc[38, encuesta.columns.get_loc('trabajo')] = 'embajada de estados unidos'
encuesta.iloc[207, encuesta.columns.get_loc('trabajo')] = 'bibliotecario'
encuesta.iloc[498, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[498, encuesta.columns.get_loc('categoria')] = 'actividades elementales y de apoyo_20'

In [573]:
ocupaciones_profin= ["qfb","financiero","economista","relaciones publicas","odontologia"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"

In [574]:
ocupaciones_formal = ["vendedor de comida"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"

In [575]:
ocupaciones_informal = ["comerciante sector informal","ropa","recaudería","autopartes", "vendedor","productos de belleza","sector informal"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [576]:
ocupaciones_empre = ["director o gerente","negocios","arrendatario"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empre), 'categoria'] = "empresario_6"

In [577]:
ocupaciones_limp= ["abarrotes","empacadora en super"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_limp), 'categoria'] = "actividades elementales y de apoyo_20"

In [578]:
ocupaciones_saludd= ["laboratorista","voluntaria"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"

In [579]:
ocupaciones_of = ["peluquero"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"

In [580]:
ocupaciones_admin = ["administrativo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_admin), 'categoria'] = "administrativo_9"

In [581]:
ocupaciones_guber = ["empleado gubernamental",'trabaja en pemex',"embajada de estados unidos","'directora del área de enfermería de los hospitales de gobierno'"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "empleado gubernamental_11"

In [582]:
ocupaciones_cliente= ["papeleria","vendedor de tiendas departamentales/promotor de seguros, telefonía","servicio de correos","turismo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"

In [583]:
ocupaciones_art= ["instrumentalista"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_art), 'categoria'] = "artista_2"
ocupaciones_agri= ["trabajos relacionados con agricultura"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_agri), 'categoria'] = "agricultura_19"
ocupaciones_educ=["bibliotecario"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_educ), 'categoria'] = "área educativa_13"

In [584]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           542
actividades elementales y de apoyo_20    108
comerciante sector formal_5               13
comerciante sector informal_4             11
administrativo_9                           8
profesionista independiente_14             6
atención a clientes_8                      4
área de la salud_16                        2
empleado gubernamental_11                  2
empresario_6                               2
oficios_21                                 2
área educativa_13                          2
agricultura_19                             1
artista_2                                  1
Name: count, dtype: Int64

In [585]:
encuesta[["trabajo","categoria"]]

,trabajo,categoria
0,ninguna,desconocido_99
1,no trabaja/ trabajaba_21,desconocido_99
2,administrativo_9,desconocido_99
3,no trabaja/ trabajaba_21,desconocido_99
4,flores y productos en un puesto no establecido_3,desconocido_99
...,...,...
699,trabajadores domésticos,actividades elementales y de apoyo_20
700,no trabaja/ trabajaba_21,desconocido_99
701,empleado_1,desconocido_99
702,trabajadores domésticos,actividades elementales y de apoyo_20


### Resultado final

In [586]:
mapeo_codigos_categoria2 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria2[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria2

{'desconocido_99': 'A99',
 'actividades elementales y de apoyo_20': 'A20',
 'administrativo_9': 'A09',
 'empresario_6': 'A06',
 'oficios_21': 'A21',
 'profesionista independiente_14': 'A14',
 'comerciante sector formal_5': 'A05',
 'comerciante sector informal_4': 'A04',
 'área educativa_13': 'A13'}

In [587]:
encuesta['categoria'].replace(mapeo_codigos_categoria2, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

0                           A04
1                           A05
2                           A06
3                           A09
4                           A13
5                           A14
6                           A20
7                           A21
8                           A99
9                agricultura_19
10                    artista_2
11        atención a clientes_8
12    empleado gubernamental_11
13          área de la salud_16
Name: categoria, dtype: string

In [588]:
mapeo_codigos_ocupaciones2 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria2[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones2[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones2

{'administrador_2': 'A9901',
 'administrador_5': 'A9902',
 'administrativo_9': 'A9903',
 'agricultor_1': 'A9904',
 'atención a clientes_1': 'A9905',
 'belleza': 'A9906',
 'clases': 'A9907',
 'cocina': 'A9908',
 'cocinera': 'A9909',
 'contador público_1': 'A9910',
 'contador_3': 'A9911',
 'costura': 'A9912',
 'costurera': 'A9913',
 'coturero': 'A9914',
 'desconocido': 'A9915',
 'dirección': 'A9916',
 'director o gerente_2': 'A9917',
 'directora': 'A9918',
 'directora del área de enfermería de los hospitales de gobierno': 'A9919',
 'dueño_1': 'A9920',
 'educación básica_2': 'A9921',
 'empleada': 'A9922',
 'empleado gubernamental_11': 'A9923',
 'empleado_1': 'A9924',
 'empleadora domestica': 'A9925',
 'empresa': 'A9926',
 'empresa electrodomesticos': 'A9927',
 'enfermera_2': 'A9928',
 'flores y productos en un puesto no establecido_3': 'A9929',
 'fundaciones': 'A9930',
 'gastronomica': 'A9931',
 'gerente de tienda': 'A9932',
 'granjero_2': 'A9933',
 'investigador-profesor en universidad/p

In [589]:
encuesta['trabajo'].replace(mapeo_reorganizado_importado,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

0                  A0206
1                  A0401
2                  A0402
3                  A0406
4                  A0409
             ...        
98     vendedora de ropa
99         venta de ropa
100     venta de seguros
101          vigilante_3
102    área educativa_14
Name: trabajo, Length: 103, dtype: string

In [590]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[AbuelaMaterna]', 'categoria': 'Categoria[AbuelaMaterna]'})


In [591]:
encuesta.iloc[10, encuesta.columns.get_loc('Categoria[AbuelaMaterna]')] ='A13'

In [592]:
encuesta[['Trabajas[AbuelaMaterna]', 'Trabajo[AbuelaMaterna]', 'Categoria[AbuelaMaterna]']]

,Trabajas[AbuelaMaterna],Trabajo[AbuelaMaterna],Categoria[AbuelaMaterna]
0,Y,A9921,A99
1,Y,no trabaja/ trabajaba_21,A99
2,Y,administrativo_9,A99
3,Y,no trabaja/ trabajaba_21,A99
4,Y,flores y productos en un puesto no establecido_3,A99
...,...,...,...
699,Y,A2018,A20
700,Y,no trabaja/ trabajaba_21,A99
701,Y,empleado_1,A99
702,Y,A2018,A20


In [593]:
#import json

#trabajos = { ... }

#with open("trabajos(abuelamaterna).json", "w") as archivo:
 #   json.dump(mapeo_codigos_ocupaciones2, archivo)

# Trabajos Abuelo Materno

## Limpieza de Nivel 1

In [594]:
mostrar_unicos(encuesta,"q0r207")

0     -oth-
1        10
2        11
3        12
4        13
5        14
6        15
7        16
8        17
9        18
10       19
11       20
12       21
13       22
14        3
15        4
16        5
17        6
18        7
19        9
20      NaN
Name: q0r207, dtype: object

In [595]:
mapeo_respuestas3 = {
    '2':'Actividades deportivas_1',
    '3':'Artista_2',
    '4':'Chofer_3',
    '5':'Comerciante sector formal_4',
    '6':'Comerciante sector informal_5',
    '7':'Empresario_6',
    '8':' Comercio Virtual_7',
    '9': 'Atención a clientes_8',
    '10':'Administrativo_9',
    '11':'Gubernamental administrativo_10',
    '12':'Empleado gubernamental_11',
    '13':'Operador de maquinaria móvil_12',
    '14':'Personal técnico_13',
    '15':'Área educativa_14',
    '16':'Profesionista independiente_15',
    '17':'Seguridad_16',
    '18':'Área de la salud_17',
    '19':'Construcción_18',
    '20':'Fábrica_19',
    '21':'Agricultura_20',
    '22':'No trabaja/ trabajaba_21'
}

In [596]:
llenar_respuesta_libre(encuesta,"q0r207", identificador_otro='-oth-', sufijo='[other]')

In [597]:
llenar_vacios(encuesta,"q0r207",'ninguna')
encuesta['q0r207'] = encuesta['q0r207'].str.strip().str.lower()

In [598]:
encuesta['q0r207'] = encuesta['q0r207'].replace(mapeo_respuestas3)

In [599]:
mostrar_unicos(encuesta,"q0r207")

0          Administrativo_9
1            Agricultura_20
2                 Artista_2
3     Atención a clientes_8
4                  Chofer_3
              ...          
81                   ventas
82                 zapatero
83                  zapatos
84      Área de la salud_17
85        Área educativa_14
Name: q0r207, Length: 86, dtype: object

## Limpieza de Nivel 2

### Columnas base

In [600]:
columnas_base3 = ['q0r535','ART2','CHOFER21','FORMAL45','INFORMAL53','epresari32','virtual34','clientes34','admi34','gubernam3','empleadogub3','operador3','t3cnic0','educat1vo','ind3p3ndient','s3guridad','salud2','c0nstrucci0','fab343','agr1cultur2']
columnas_base3

['q0r535',
 'ART2',
 'CHOFER21',
 'FORMAL45',
 'INFORMAL53',
 'epresari32',
 'virtual34',
 'clientes34',
 'admi34',
 'gubernam3',
 'empleadogub3',
 'operador3',
 't3cnic0',
 'educat1vo',
 'ind3p3ndient',
 's3guridad',
 'salud2',
 'c0nstrucci0',
 'fab343',
 'agr1cultur2']

In [601]:
len(columnas_base3)

20

In [602]:
encuesta[columnas_base3]

,q0r535,ART2,CHOFER21,FORMAL45,INFORMAL53,epresari32,virtual34,clientes34,admi34,gubernam3,empleadogub3,operador3,t3cnic0,educat1vo,ind3p3ndient,s3guridad,salud2,c0nstrucci0,fab343,agr1cultur2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A5,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,NaN,NaN,NaN,NaN


In [603]:
encuesta['q0r535'] = encuesta['q0r535'].replace({'A2': 'Entrenador de gimnasio_1', 'A3': 'Profesor de clases deportivas_2'})
encuesta['ART2'] = encuesta['ART2'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2','A4': 'Músico_3', 'A5':'Cantante_4'})
encuesta['CHOFER21'] = encuesta['CHOFER21'].replace({'A2': 'Taxista_1', 'A3': 'Uber_2','A4': 'Microbús_3', 'A5':'Camiones_4'})


In [604]:
encuesta['FORMAL45'] = encuesta['FORMAL45'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2','A4':'Flores y productos en un puesto no establecido_3'})
encuesta['INFORMAL53'] = encuesta['INFORMAL53'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})
encuesta['epresari32'] = encuesta['epresari32'].replace({'A2': 'Dueño_1','A3':'Director o gerente_2'})
encuesta['clientes34'] = encuesta['clientes34'].replace({'A5': 'Promotor de seguros_4','A4':'Vendedor de tiendas departamentales_3','A2':'Atención a clientes_1'})
encuesta['admi34'] = encuesta['admi34'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})

In [605]:
encuesta['gubernam3'] = encuesta['gubernam3'].replace({'A2': 'Jefe de departamento_1','A3':'Jefe de área_2','A4': 'subdirector_2','A5':'Subadministrador_3','A6':' Director_4','A7':'Administrador_5','A8':'Director general_6','A9':'Administrador general_7'})
encuesta['educat1vo'] = encuesta['educat1vo'].replace({'A2': 'Investigador-profesor en universidad/preparatoria_1','A3':'Educación básica_2','A4':'Técnico en educación (educador o guardería)_3'})

In [606]:
encuesta['ind3p3ndient'] = encuesta['ind3p3ndient'].replace({'A2': 'Abogado_1','A3':'Psicólogo_2','A4':'Contador_3','A5':'Dentista_4','A6':'Médico independiente_5','A7':' Terapeuta_6'})
encuesta['s3guridad'] = encuesta['s3guridad'].replace({'A2': 'Guardaespaldas_1','A3':'Patrullero_2','A4':'Vigilante_3','A5':'Empleado de seguridad en bancos_4','A6':'Empleado de seguridad privada_5','A7':'Policía de tránsito_6'})
encuesta['salud2'] = encuesta['salud2'].replace({'A2': 'Médico_1','A3':'Enfermera_2','A4':'Camillero_3','A5':'Paramédico_4'})
encuesta['c0nstrucci0'] = encuesta['c0nstrucci0'].replace({'A2': 'Arquitecto_1','A3':'Ingeniero civil/albañil o trabajador general_2'})
encuesta['fab343'] = encuesta['fab343'].replace({'A2':'Empleado_1'})


In [607]:
encuesta['agr1cultur2'] = encuesta['agr1cultur2'].replace({'A2':'Agricultor_1','A3':'Granjero_2','A4':'Siembra_3'})

In [608]:
encuesta['t3cnic0'] = encuesta['t3cnic0[comment]']
encuesta['empleadogub3']=encuesta['empleadogub3[comment]']
encuesta['operador3']=encuesta['operador3[comment]']

In [609]:
encuesta[columnas_base3]

,q0r535,ART2,CHOFER21,FORMAL45,INFORMAL53,epresari32,virtual34,clientes34,admi34,gubernam3,empleadogub3,operador3,t3cnic0,educat1vo,ind3p3ndient,s3guridad,salud2,c0nstrucci0,fab343,agr1cultur2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado de seguridad en bancos_4,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agricultor_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vigilante_3,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-oth-,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vigilante_3,NaN,NaN,NaN,NaN


### Columna con sufijos

In [610]:
columnas_sufijo3 = []
for col in columnas_base3:
    if col + '[other]' in encuesta.columns:
        columnas_sufijo3.append(col + '[other]')
    elif col + '[comment]' in encuesta.columns:
        columnas_sufijo3.append(col + '[comment]')
assert len(columnas_sufijo3) == len(columnas_base3)-1
columnas_sufijo3.sort()
columnas_sufijo3

['CHOFER21[other]',
 'FORMAL45[other]',
 'INFORMAL53[other]',
 'admi34[other]',
 'agr1cultur2[other]',
 'c0nstrucci0[other]',
 'clientes34[other]',
 'educat1vo[other]',
 'empleadogub3[comment]',
 'epresari32[other]',
 'fab343[other]',
 'gubernam3[other]',
 'ind3p3ndient[other]',
 'operador3[comment]',
 'q0r535[other]',
 's3guridad[other]',
 'salud2[other]',
 't3cnic0[comment]',
 'virtual34[other]']

In [611]:
for col_con_sufijo in columnas_sufijo3:
    columna_sin_sufijo3 = re.sub(r'\[\w+\]', '', col_con_sufijo)
    encuesta[columna_sin_sufijo3] = encuesta[columna_sin_sufijo3].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo3] = np.where(encuesta[columna_sin_sufijo3] == '-oth-',
                                             encuesta[col_con_sufijo],
                                             encuesta[columna_sin_sufijo3])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [612]:
encuesta[columnas_base3]

,q0r535,ART2,CHOFER21,FORMAL45,INFORMAL53,epresari32,virtual34,clientes34,admi34,gubernam3,empleadogub3,operador3,t3cnic0,educat1vo,ind3p3ndient,s3guridad,salud2,c0nstrucci0,fab343,agr1cultur2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INGENIERO,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INGENIERO MECANICO ELECTRICO,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado de seguridad en bancos_4,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agricultor_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vigilante_3,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CARPINTERO,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vigilante_3,NaN,NaN,NaN,NaN


### Asignación de Trabajo

In [613]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)  # columna nueva
for columna in columnas_base3:
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['q0r207'])
encuesta.drop(columns='q0r207', inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

0                            abarrotes
1                            abogado_1
2                  actividad religiosa
3                             actuario
4      administraba escritorio publico
                    ...               
211                 ventas en tianguis
212                        veterinario
213                        vigilante_3
214                           zapatero
215                            zapatos
Name: trabajo, Length: 216, dtype: string

In [614]:
trabajos_nuevos3 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos3

0                         ingeniero
1      ingeniero mecanico electrico
2                        agricultor
3                             no se
4                           militar
                   ...             
264                     desconocido
265               puesto de verdura
266                        zapatero
267                      trabajador
268                      carpintero
Name: trabajo, Length: 269, dtype: string

In [615]:
encuesta['trabajo']

0                                ninguna
1                              ingeniero
2           ingeniero mecanico electrico
3      empleado de seguridad en bancos_4
4                           agricultor_1
                     ...                
699                          vigilante_3
700                           carpintero
701                           empleado_1
702                          vigilante_3
703                         agricultor_1
Name: trabajo, Length: 704, dtype: string

### Correción de trabajos

In [616]:
trabajos_oficiales

0                         entrenador
1                         instructor
2      profesor de clases deportivas
3                              actor
4                             ballet
                   ...              
119                           sastre
120                         zapatero
121                         zapatero
122                         mecánico
123                          ninguna
Length: 124, dtype: object

In [617]:
correccion3 = match_most_similar(trabajos_oficiales,
                                 trabajos_nuevos3,
                                 min_similarity=.40,
                                 ignore_index=True)
comparar3 = pd.DataFrame({'original': trabajos_nuevos3, 'correccion': correccion3})
comparar3['hubo_correccion'] = comparar3['original'].ne(comparar3['correccion'])
comparar3.sort_values(by='hubo_correccion', ascending=False)

TypeError: __new__() got an unexpected keyword argument 'ignore_index'

In [618]:
mapeo_correcciones3 = dict(zip(trabajos_nuevos3, correccion3))
mapeo_correcciones3

NameError: name 'correccion3' is not defined

In [619]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()
encuesta['trabajo'] = encuesta['trabajo'].str.strip()
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones3)
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta, 'trabajo')

NameError: name 'mapeo_correcciones3' is not defined

In [620]:
encuesta['trabajo']

0                                ninguna
1                              ingeniero
2           ingeniero mecanico electrico
3      empleado de seguridad en bancos_4
4                           agricultor_1
                     ...                
699                          vigilante_3
700                           carpintero
701                           empleado_1
702                          vigilante_3
703                         agricultor_1
Name: trabajo, Length: 704, dtype: string

### Modificar algunos sinónimos

In [621]:
encuesta.iloc[592, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[21, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[112, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[381, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[171, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[221, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[322, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[283, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[68, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[481, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[229, encuesta.columns.get_loc('trabajo')] ='mecánico'

In [622]:
encuesta.iloc[136, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[239, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[647, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[195, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[112, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[445, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[581, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[662, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[642, encuesta.columns.get_loc('trabajo')] ='empleado en seguridad privada'

In [623]:
encuesta.iloc[309, encuesta.columns.get_loc('trabajo')] ='empleado'
encuesta.iloc[500, encuesta.columns.get_loc('trabajo')] ='empleado'
encuesta.iloc[624, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[454, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[670, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[447, encuesta.columns.get_loc('trabajo')] ='músico'
encuesta.iloc[565, encuesta.columns.get_loc('trabajo')] ='abarrotes'
encuesta.iloc[688, encuesta.columns.get_loc('trabajo')] ='vendedor de comida'
encuesta.iloc[272, encuesta.columns.get_loc('trabajo')] ='dueño'
encuesta.iloc[425, encuesta.columns.get_loc('trabajo')] ='policía'
encuesta.iloc[287, encuesta.columns.get_loc('trabajo')] ='arrendador'
encuesta.iloc[238, encuesta.columns.get_loc('trabajo')] ='arrendador'
encuesta.iloc[3, encuesta.columns.get_loc('trabajo')] ='empleado en seguridad en bancos'
encuesta.iloc[285, encuesta.columns.get_loc('trabajo')] ='empleado en seguridad en bancos'
encuesta.iloc[219, encuesta.columns.get_loc('trabajo')] ='empleado en seguridad en bancos'

In [624]:
encuesta['trabajo']

0                              ninguna
1                            ingeniero
2         ingeniero mecanico electrico
3      empleado en seguridad en bancos
4                         agricultor_1
                    ...               
699                        vigilante_3
700                         carpintero
701                         empleado_1
702                        vigilante_3
703                       agricultor_1
Name: trabajo, Length: 704, dtype: string

### Creación de Categorías

In [625]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion ingeniero mecanico electrico es desconocida
La ocupacion agricultor_1 es desconocida
La ocupacion granjero_2 es desconocida
La ocupacion no se es desconocida
La ocupacion contador público_1 es desconocida
La ocupacion militar es desconocida
La ocupacion microbús_3 es desconocida
La ocupacion dueño_1 es desconocida
La ocupacion vendedor de productos es desconocida
La ocupacion ingeniero civil/albañil o trabajador general_2 es desconocida
La ocupacion trbaja en la delegacion es desconocida
La ocupacion dueño_1 es desconocida
La ocupacion director o gerente_2 es desconocida
La ocupacion médico_1 es desconocida
La ocupacion distribuidor es desconocida
La ocupacion director o gerente_2 es desconocida
La ocupacion flores y productos en un puesto no establecido_3 es desconocida
La ocupacion abogado_1 es desconocida
La ocupacion administrador_2 es desconocida
La ocupacion contador_3 es desconocida
La ocupacion administrativo es desconocida
La ocupacion vendedor por cuenta propia (

In [626]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0     actividades elementales y de apoyo_20
1                            agricultura_19
2                                 artista_2
3               comerciante sector formal_5
4             comerciante sector informal_4
5                           construcción_17
6                            desconocido_99
7                              empresario_6
8                                fábrica_18
9                                oficios_21
10           profesionista independiente_14
11                             seguridad_15
dtype: string

In [627]:
categorias.value_counts()

desconocido_99                           627
oficios_21                                29
comerciante sector informal_4             10
empresario_6                               8
actividades elementales y de apoyo_20      8
profesionista independiente_14             7
seguridad_15                               5
fábrica_18                                 5
construcción_17                            2
agricultura_19                             1
artista_2                                  1
comerciante sector formal_5                1
Name: count, dtype: Int64

In [628]:
encuesta['categoria'] = categorias
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,ingeniero,profesionista independiente_14
2,ingeniero mecanico electrico,desconocido_99
3,empleado en seguridad en bancos,seguridad_15
4,agricultor_1,desconocido_99
...,...,...
699,vigilante_3,desconocido_99
700,carpintero,oficios_21
701,empleado_1,desconocido_99
702,vigilante_3,desconocido_99



### Asignación de la columna "Trabajas[AbueloMaterno]"
Esta columna se crea para corregir los datos de zoom 2, donde no se realiza la pregunta: ¿En qué trabajas".La creación de esta columna nos ayudará a poder limpiar aquellos trabajos que esten nombrados como: "no trabaja/trabajaba".

In [629]:
encuesta['Trabajas[AbueloMaterno]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] == 'no trabaja/ trabajaba':
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbueloMaterno]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbueloMaterno]'] = 'Y'

In [630]:
#Esta función nos ayudará a desplazar la columna "Trabajas"
column_order = list(encuesta.columns)

# Reordenar la lista de columnas para que 'Trabajas[Madre]' esté antes de 'Trabajas'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[AbueloMaterno]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [631]:
encuesta[['Trabajas[AbueloMaterno]','trabajo', 'categoria']]

,Trabajas[AbueloMaterno],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,ingeniero,profesionista independiente_14
2,Y,ingeniero mecanico electrico,desconocido_99
3,Y,empleado en seguridad en bancos,seguridad_15
4,Y,agricultor_1,desconocido_99
...,...,...,...
699,Y,vigilante_3,desconocido_99
700,Y,carpintero,oficios_21
701,Y,empleado_1,desconocido_99
702,Y,vigilante_3,desconocido_99


In [632]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/ trabajaba', 'ninguna')

### Últimas modificaciones sobre categorías

In [633]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,ingeniero,profesionista independiente_14
2,ingeniero mecanico electrico,desconocido_99
3,empleado en seguridad en bancos,seguridad_15
4,agricultor_1,desconocido_99
...,...,...
699,vigilante_3,desconocido_99
700,carpintero,oficios_21
701,empleado_1,desconocido_99
702,vigilante_3,desconocido_99


In [634]:
encuesta.iloc[137, encuesta.columns.get_loc('trabajo')] = 'trabaja en pemex'
encuesta.iloc[141, encuesta.columns.get_loc('trabajo')] = 'trabaja en pemex'
encuesta.iloc[416, encuesta.columns.get_loc('trabajo')] = 'trabaja en pemex'
encuesta.iloc[14, encuesta.columns.get_loc('trabajo')] = 'trabaja en delegación'
encuesta.iloc[152, encuesta.columns.get_loc('trabajo')] = 'construcción'
encuesta.iloc[159, encuesta.columns.get_loc('trabajo')] = 'ferrocarrilero'
encuesta.iloc[231, encuesta.columns.get_loc('trabajo')] = 'electrónica'
encuesta.iloc[611, encuesta.columns.get_loc('trabajo')] = 'lavandería'
encuesta.iloc[626, encuesta.columns.get_loc('trabajo')] = 'hojalatero'
encuesta.iloc[570, encuesta.columns.get_loc('trabajo')] = 'lechero'
encuesta.iloc[111, encuesta.columns.get_loc('trabajo')] = 'bienes raíces'
encuesta.iloc[373, encuesta.columns.get_loc('trabajo')] = 'joyero'
encuesta.iloc[501, encuesta.columns.get_loc('trabajo')] = 'joyero'
encuesta.iloc[222, encuesta.columns.get_loc('trabajo')] = 'patólogo'
encuesta.iloc[197, encuesta.columns.get_loc('trabajo')]= "reparación electrónicos"
encuesta.iloc[111, encuesta.columns.get_loc('trabajo')]= "bienes raíces"
encuesta.iloc[563, encuesta.columns.get_loc('trabajo')]= "ropa"
encuesta.iloc[316, encuesta.columns.get_loc('trabajo')]="vendedor de comida"
encuesta.iloc[121, encuesta.columns.get_loc('trabajo')]="sindicato de educación"

In [635]:
encuesta.iloc[244, encuesta.columns.get_loc('trabajo')]="ganadero"
encuesta.iloc[264, encuesta.columns.get_loc('trabajo')]="ganadero"
encuesta.iloc[438, encuesta.columns.get_loc('trabajo')]="periódicos flores y productos en un puesto no establecido"
encuesta.iloc[237, encuesta.columns.get_loc('trabajo')]="periódicos flores y productos en un puesto no establecido"
encuesta.iloc[332, encuesta.columns.get_loc('trabajo')]="empleado en seguridad privada"

In [636]:
ocupaciones_especificas = ["laboratorio","sindicato de educación"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_especificas), 'categoria'] = "área educativa_13"

In [637]:
ocupaciones_formal = ["comerciante sector formal","vendedor de comida","periódicos flores y productos en un puesto no establecido"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal","uniformes escolares","refacciones","purificadora","ropa"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [638]:
ocupaciones_admin = ["banco"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_admin), 'categoria'] = "administrativo_9"

In [639]:
ocupaciones_empre = ["director o gerente","arrendador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empre), 'categoria'] = "empresario_6"

In [640]:
ocupaciones_guber = ["empleado gubernamental",'trabaja en pemex','trabaja en delegación', "trabajo en luz y fuerza","hacienda"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "empleado gubernamental_11"

In [641]:
ocupaciones_guber = ["gubernamental administrativo", "operativo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "gubernamental administrativo_10"

In [642]:
ocupaciones_seg = ["encargado de la planta","militar","guardia de seguridad","federal","marina","empleado en seguridad privada"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_seg), 'categoria'] = "seguridad_15"

In [643]:
ocupaciones_cons = ["contratista","ingeniero civil/albañil o trabajador general","construcción","albañil",]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cons), 'categoria'] = "construcción_17"

In [644]:
ocupaciones_chof = ["chofer","transporte de materiales","transportista","trailer","transporte colectivo metro","mudanzas","maneja un camión de basura", "empresa de transporte","servicio de transporte", "piloto","ferrocarrilero"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_chof), 'categoria'] = "chofer_3"

In [645]:
ocupaciones_tec = ["tecnico","personal técnico","telegrafista","telecomunicacion","reparación electrónicos"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_tec), 'categoria'] = "personal técnico_12"

In [646]:
ocupaciones_of = ["electrónica","mecánico","electromecánico","hojalatero","petrolero","metalurgia","joyero","almacenista","electricista","lavandería","imprenta","vulcanizador","tabiquera","cobrador","poligrafista","pescador","hojalatero","tornero","lechero","ganadero"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"

In [647]:
ocupaciones_profin= ["economista","actuario","bienes raíces","financiero","psiquiatra","programador","economista","actuario","revisor","periodista","veterinario"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"

In [648]:
ocupaciones_saludd= ["patólogo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"

In [649]:
ocupaciones_cam= ["campesino","agraria"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cam), 'categoria'] = "agricultura_19"

In [650]:
ocupaciones_limp= ["barrendero","limpieza y transporte"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_limp), 'categoria'] = "actividades elementales y de apoyo_20"

In [651]:
ocupaciones_fabrica=["obrero","industria metalurgica"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_fabrica), 'categoria'] = "fábrica_18"

In [652]:
ocupaciones_cliente= ["bienes raíces", "hotel"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"

In [653]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           552
oficios_21                                44
seguridad_15                              14
profesionista independiente_14            14
fábrica_18                                11
comerciante sector informal_4             11
empresario_6                              10
actividades elementales y de apoyo_20     10
chofer_3                                   7
empleado gubernamental_11                  6
construcción_17                            5
personal técnico_12                        4
comerciante sector formal_5                4
agricultura_19                             3
área educativa_13                          3
atención a clientes_8                      2
área de la salud_16                        1
gubernamental administrativo_10            1
administrativo_9                           1
artista_2                                  1
Name: count, dtype: Int64

In [654]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,ingeniero,profesionista independiente_14
2,ingeniero mecanico electrico,desconocido_99
3,empleado en seguridad en bancos,seguridad_15
4,agricultor_1,desconocido_99
...,...,...
699,vigilante_3,desconocido_99
700,carpintero,oficios_21
701,empleado_1,desconocido_99
702,vigilante_3,desconocido_99


### Resultado final

In [655]:
mapeo_codigos_categoria3 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria3[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria3

{'desconocido_99': 'A99',
 'profesionista independiente_14': 'A14',
 'seguridad_15': 'A15',
 'agricultura_19': 'A19',
 'empresario_6': 'A06',
 'oficios_21': 'A21',
 'comerciante sector informal_4': 'A04',
 'fábrica_18': 'A18',
 'actividades elementales y de apoyo_20': 'A20',
 'construcción_17': 'A17',
 'artista_2': 'A02',
 'comerciante sector formal_5': 'A05'}

In [656]:
mapeo_codigos_categoria3['personal técnico_12'] = 'A12'

mapeo_codigos_categoria3

{'desconocido_99': 'A99',
 'profesionista independiente_14': 'A14',
 'seguridad_15': 'A15',
 'agricultura_19': 'A19',
 'empresario_6': 'A06',
 'oficios_21': 'A21',
 'comerciante sector informal_4': 'A04',
 'fábrica_18': 'A18',
 'actividades elementales y de apoyo_20': 'A20',
 'construcción_17': 'A17',
 'artista_2': 'A02',
 'comerciante sector formal_5': 'A05',
 'personal técnico_12': 'A12'}

In [657]:
encuesta['categoria'].replace(mapeo_codigos_categoria3, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

0                                 A02
1                                 A04
2                                 A05
3                                 A06
4                                 A12
5                                 A14
6                                 A15
7                                 A17
8                                 A18
9                                 A19
10                                A20
11                                A21
12                                A99
13                   administrativo_9
14              atención a clientes_8
15                           chofer_3
16          empleado gubernamental_11
17    gubernamental administrativo_10
18                área de la salud_16
19                  área educativa_13
Name: categoria, dtype: string

In [658]:
mapeo_codigos_ocupaciones3 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria3[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones3[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones3

{'abogado_1': 'A9901',
 'actividad religiosa': 'A9902',
 'administraba escritorio publico': 'A9903',
 'administrador_2': 'A9904',
 'administrativo': 'A9905',
 'administrativo_9': 'A9906',
 'agricultor_1': 'A9907',
 'area administrativa': 'A9908',
 'area de limpieza': 'A9909',
 'arquitecto_1': 'A9910',
 'arreglaba cafeteras': 'A9911',
 'artesanías': 'A9912',
 'atención a clientes_1': 'A9913',
 'cacique de rancho': 'A9914',
 'cafetales': 'A9915',
 'camiones_4': 'A9916',
 'carbón': 'A9917',
 'carpinteria': 'A9918',
 'coches y camiones': 'A9919',
 'comerciante sector formal_4': 'A9920',
 'comerciante sector informal_5': 'A9921',
 'construcción_18': 'A9922',
 'contador público_1': 'A9923',
 'contador_3': 'A9924',
 'desconoce': 'A9925',
 'desconocido': 'A9926',
 'director de cine': 'A9927',
 'director general_6': 'A9928',
 'director o gerente_2': 'A9929',
 'distribuidor': 'A9930',
 'dueño de imprenta': 'A9931',
 'dueño de sembradio': 'A9932',
 'dueño de taller': 'A9933',
 'dueño_1': 'A9934',

In [659]:
mapeo_codigos_ocupaciones3["personal técnico"] = "A1201"
mapeo_codigos_ocupaciones3["reparación electrónicos"] = "A1202"
mapeo_codigos_ocupaciones3["telegrafista"] = "A1203"
mapeo_codigos_ocupaciones3["tecnico"] = "A1204"
mapeo_codigos_ocupaciones3["telecomunicacion"] = "A1205"

mapeo_codigos_ocupaciones3

{'abogado_1': 'A9901',
 'actividad religiosa': 'A9902',
 'administraba escritorio publico': 'A9903',
 'administrador_2': 'A9904',
 'administrativo': 'A9905',
 'administrativo_9': 'A9906',
 'agricultor_1': 'A9907',
 'area administrativa': 'A9908',
 'area de limpieza': 'A9909',
 'arquitecto_1': 'A9910',
 'arreglaba cafeteras': 'A9911',
 'artesanías': 'A9912',
 'atención a clientes_1': 'A9913',
 'cacique de rancho': 'A9914',
 'cafetales': 'A9915',
 'camiones_4': 'A9916',
 'carbón': 'A9917',
 'carpinteria': 'A9918',
 'coches y camiones': 'A9919',
 'comerciante sector formal_4': 'A9920',
 'comerciante sector informal_5': 'A9921',
 'construcción_18': 'A9922',
 'contador público_1': 'A9923',
 'contador_3': 'A9924',
 'desconoce': 'A9925',
 'desconocido': 'A9926',
 'director de cine': 'A9927',
 'director general_6': 'A9928',
 'director o gerente_2': 'A9929',
 'distribuidor': 'A9930',
 'dueño de imprenta': 'A9931',
 'dueño de sembradio': 'A9932',
 'dueño de taller': 'A9933',
 'dueño_1': 'A9934',

In [660]:
encuesta['trabajo'].replace(mapeo_reorganizado_importado,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

0                                              A0208
1                                              A0302
2                                              A0303
3                                              A0304
4                                              A0308
                           ...                      
180            vendedor de tiendas departamentales_3
181    vendedor por cuenta propia (negocio propio)_1
182                                    venta de ropa
183                                      vigilante_3
184                                          zapatos
Name: trabajo, Length: 185, dtype: string

In [661]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[AbueloMaterno]', 'categoria': 'Categoria[AbueloMaterno]'})
encuesta[['Trabajas[AbueloMaterno]', 'Trabajo[AbueloMaterno]', 'Categoria[AbueloMaterno]']]

,Trabajas[AbueloMaterno],Trabajo[AbueloMaterno],Categoria[AbueloMaterno]
0,Y,A9921,A99
1,Y,A1411,A14
2,Y,ingeniero mecanico electrico,A99
3,Y,A1502,A15
4,Y,agricultor_1,A99
...,...,...,...
699,Y,vigilante_3,A99
700,Y,A2106,A21
701,Y,empleado_1,A99
702,Y,vigilante_3,A99


In [662]:
#import json

#trabajos = { ... }

#with open("trabajos(abuelomaterno).json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones3, archivo)

# Trabajos Abuela Paterna

## Limpieza de Nivel 1

In [663]:
mostrar_unicos(encuesta,"q0r926")

0     -oth-
1        10
2        11
3        12
4        18
5        19
6         2
7        20
8        21
9        23
10       24
11       25
12        3
13        5
14        6
15        7
16        9
17      NaN
Name: q0r926, dtype: object

In [664]:
mapeo_respuestas4 = {
    '2':'Actividades deportivas_1',
    '3':'Artista_2',
    '4':'Chofer_3',
    '5':'Comerciante sector formal_4',
    '6':'Comerciante sector informal_5',
    '7':'Empresario_6',
    '8':' Comercio Virtual_7',
    '9': 'Atención a clientes_8',
    '10':'Administrativo_9',
    '11':'Gubernamental administrativo_10',
    '12':'Empleado gubernamental_11',
    '16':'Operador de maquinaria móvil_12',
    '17':'Personal técnico_13',
    '18':'Área educativa_14',
    '19':'Profesionista independiente_15',
    '20':'Seguridad_16',
    '21':'Área de la salud_17',
    '22':'Construcción_18',
    '23':'Fábrica_19',
    '24':'Agricultura_20',
    '25':'No trabaja/ trabajaba_21'
}

In [665]:
llenar_respuesta_libre(encuesta, "q0r926", identificador_otro='-oth-', sufijo='[other]')


In [666]:
llenar_vacios(encuesta, "q0r926", 'ninguna')
encuesta['q0r926'] = encuesta['q0r926'].str.strip().str.lower()

In [667]:
encuesta['q0r926'] = encuesta['q0r926'].replace(mapeo_respuestas4)

In [668]:
mostrar_unicos(encuesta,"q0r926")

0            Actividades deportivas_1
1                    Administrativo_9
2                      Agricultura_20
3                           Artista_2
4               Atención a clientes_8
5         Comerciante sector formal_4
6       Comerciante sector informal_5
7           Empleado gubernamental_11
8                        Empresario_6
9                          Fábrica_19
10    Gubernamental administrativo_10
11           No trabaja/ trabajaba_21
12     Profesionista independiente_15
13                       Seguridad_16
14                        ama de casa
15     ama de casa empleada doméstica
16                        arrendadora
17                               aseo
18                            biologa
19                           cocinera
20                         cosmeticos
21                            costura
22                          costurera
23                          desconoce
24                        desconocido
25                 empleada domestica
26          

## Limpieza de Nivel 2

### Columnas base

In [669]:
columnas_base4 = ['DEPORTE33', 'ARTISTAS329', 'CHOFER03', 'FORMAL32', 'INFORMAL56', 'EMPRESARIO3', 'VIRTUAL347', 'CLIENTES63', 'ADMINISTRACION6', 'EMPLEADOGUBERNAM', 'GUBERNAMENTALJ9', 'OPERADORMAQ2', 'TECNICO45', 'EDUCATIVO45', 'INDEPENDIENTE6', 'SEGURIDAD3', 'SALUD42', 'CONSTRUCCION31', 'fabrik4', 'AGRICULTURA1']

columnas_base4


['DEPORTE33',
 'ARTISTAS329',
 'CHOFER03',
 'FORMAL32',
 'INFORMAL56',
 'EMPRESARIO3',
 'VIRTUAL347',
 'CLIENTES63',
 'ADMINISTRACION6',
 'EMPLEADOGUBERNAM',
 'GUBERNAMENTALJ9',
 'OPERADORMAQ2',
 'TECNICO45',
 'EDUCATIVO45',
 'INDEPENDIENTE6',
 'SEGURIDAD3',
 'SALUD42',
 'CONSTRUCCION31',
 'fabrik4',
 'AGRICULTURA1']

In [670]:
len(columnas_base4)

20

In [671]:
encuesta[columnas_base4]

,DEPORTE33,ARTISTAS329,CHOFER03,FORMAL32,INFORMAL56,EMPRESARIO3,VIRTUAL347,CLIENTES63,ADMINISTRACION6,EMPLEADOGUBERNAM,GUBERNAMENTALJ9,OPERADORMAQ2,TECNICO45,EDUCATIVO45,INDEPENDIENTE6,SEGURIDAD3,SALUD42,CONSTRUCCION31,fabrik4,AGRICULTURA1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [672]:
encuesta['ARTISTAS329'] = encuesta['ARTISTAS329'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2','A4': 'Músico_3', 'A5':'Cantante_4'})
encuesta['FORMAL32'] = encuesta['FORMAL32'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})
encuesta['INFORMAL56'] = encuesta['INFORMAL56'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2','A4':'Flores y productos en un puesto no establecido_3'})
encuesta['EMPRESARIO3'] = encuesta['EMPRESARIO3'].replace({'A2': 'Dueño_1','A3':'Director o gerente_2'})

encuesta['CLIENTES63'] = encuesta['CLIENTES63'].replace({'A3': 'Recepcionista_2','A4':'  Vendedor de tiendas departamentales_3','A2':'Atención a clientes_1','A6':' Telefonía_5'})

encuesta['ADMINISTRACION6'] = encuesta['ADMINISTRACION6'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})

encuesta['GUBERNAMENTALJ9'] = encuesta['GUBERNAMENTALJ9'].replace({'A6':'Administrador_5'})
encuesta['EDUCATIVO45'] = encuesta['EDUCATIVO45'].replace({'A2': 'Investigador-profesor en universidad/preparatoria_1','A3':'Educación básica_2','A4':'Técnico en educación (educador o guardería)_3'})
encuesta['INDEPENDIENTE6'] = encuesta['INDEPENDIENTE6'].replace({'A2': 'Abogado_1','A3':'Psicólogo_2','A4':'Contador_3','A7':' Terapeuta_6'})

encuesta['SALUD42'] = encuesta['SALUD42'].replace({'A2': 'Médico_1','A3':'Enfermera_2','A4':'Camillero_3','A5':'Paramédico_4'})

encuesta['fabrik4'] = encuesta['fabrik4'].replace({'A2':'Empleado_1'})
encuesta['AGRICULTURA1'] = encuesta['AGRICULTURA1'].replace({'A2':'Agricultor_1','A3':' Granjero_2','A4':'Siembra_3'})

In [673]:
encuesta['EMPLEADOGUBERNAM']=encuesta['EMPLEADOGUBERNAM[comment]']

In [674]:
encuesta[columnas_base4]

,DEPORTE33,ARTISTAS329,CHOFER03,FORMAL32,INFORMAL56,EMPRESARIO3,VIRTUAL347,CLIENTES63,ADMINISTRACION6,EMPLEADOGUBERNAM,GUBERNAMENTALJ9,OPERADORMAQ2,TECNICO45,EDUCATIVO45,INDEPENDIENTE6,SEGURIDAD3,SALUD42,CONSTRUCCION31,fabrik4,AGRICULTURA1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,Vendedor por cuenta propia (negocio propio)_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Columnas con sufijos

In [675]:
columnas_sufijo4 = []
for col in columnas_base4:
    if col + '[other]' in encuesta.columns:
        columnas_sufijo4.append(col + '[other]')
    elif col + '[comment]' in encuesta.columns:
        columnas_sufijo4.append(col + '[comment]')
assert len(columnas_sufijo4) == len(columnas_base4)
columnas_sufijo4.sort()
columnas_sufijo4

['ADMINISTRACION6[other]',
 'AGRICULTURA1[other]',
 'ARTISTAS329[other]',
 'CHOFER03[other]',
 'CLIENTES63[other]',
 'CONSTRUCCION31[other]',
 'DEPORTE33[other]',
 'EDUCATIVO45[other]',
 'EMPLEADOGUBERNAM[comment]',
 'EMPRESARIO3[other]',
 'FORMAL32[other]',
 'GUBERNAMENTALJ9[other]',
 'INDEPENDIENTE6[other]',
 'INFORMAL56[other]',
 'OPERADORMAQ2[comment]',
 'SALUD42[other]',
 'SEGURIDAD3[other]',
 'TECNICO45[comment]',
 'VIRTUAL347[other]',
 'fabrik4[other]']

In [676]:
for col_con_sufijo in columnas_sufijo4:
    columna_sin_sufijo4 = re.sub(r'\[\w+\]', '', col_con_sufijo)
    encuesta[columna_sin_sufijo4] = encuesta[columna_sin_sufijo4].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo4] = np.where(encuesta[columna_sin_sufijo4] == '-oth-',
                                             encuesta[col_con_sufijo],
                                             encuesta[columna_sin_sufijo4])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [677]:
encuesta[columnas_base4]

,DEPORTE33,ARTISTAS329,CHOFER03,FORMAL32,INFORMAL56,EMPRESARIO3,VIRTUAL347,CLIENTES63,ADMINISTRACION6,EMPLEADOGUBERNAM,GUBERNAMENTALJ9,OPERADORMAQ2,TECNICO45,EDUCATIVO45,INDEPENDIENTE6,SEGURIDAD3,SALUD42,CONSTRUCCION31,fabrik4,AGRICULTURA1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,VENDIA AVON Y ORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,Vendedor por cuenta propia (negocio propio)_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asignación de Trabajo

In [678]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)  # columna nueva
for columna in columnas_base4:
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['q0r926'])
encuesta.drop(columns='q0r926', inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

0                                         abogado_1
1                                   administrador_2
2                                   administrador_5
3                                  administrativo_9
4                                         aerolínea
                          ...                      
87    vendedor por cuenta propia (negocio propio)_1
88                                        vendedora
89                                vendia avon y oro
90                                    venta de ropa
91                                         verduras
Name: trabajo, Length: 92, dtype: string

In [679]:
trabajos_nuevos4 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos4

0                 biologa
1       vendia avon y oro
2                   no se
3                limpieza
4      limpieza del hogar
              ...        
198           ama de casa
199                 hogar
200           ama de casa
201                 hogar
202           arrendadora
Name: trabajo, Length: 203, dtype: string

### Correción de trabajos

In [680]:
tesauro_ama_de_casa = {
    "trabajadores domésticos": ["ama de casa", "AMA DE CASA", "ASEO DOMESTICO", "EMPLEADA DOMESTICA",
                                "empleada doméstica", "Empleada doméstica", "HOGAR", "hogar", "casa",
                                "limpieza en casa ajena", "Ama de casa", "Hogar", "trabajadora doméstica",
                                "trabajadora domestica", "Ama de trabajadores domésticos", "empleada domestica",
                                "Limpieza en trabajadores domésticos ajena", "trabajó en trabajadores domésticos",
                                "limpieza en trabajadores domésticos ajena",
                                "empleada de trabajadores domésticos", "empleado domestico", "empleado domestica",
                                "trabajó en trabajadores domésticos", "empleada de trabajadores domésticos",
                                "trabajadores de limpiezaes de trabajadores de limpiezaes de limpieza",
                                "trabajadores de limpiezaes domésticos"]}

In [681]:
for palabra, sinonimos in tesauro_ama_de_casa.items():
    encuesta['trabajo'] = encuesta['trabajo'].replace(sinonimos, palabra, regex=True)

In [682]:
correccion4 = match_most_similar(trabajos_oficiales,
                                 trabajos_nuevos4,
                                 min_similarity=.40,
                                 ignore_index=True)
comparar4 = pd.DataFrame({'original': trabajos_nuevos4, 'correccion': correccion4})
comparar4['hubo_correccion'] = comparar4['original'].ne(comparar4['correccion'])
comparar4.sort_values(by='hubo_correccion', ascending=False)

TypeError: __new__() got an unexpected keyword argument 'ignore_index'

In [683]:
mapeo_correcciones4 = dict(zip(trabajos_nuevos4, correccion4))
mapeo_correcciones4

NameError: name 'correccion4' is not defined

In [684]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()
encuesta['trabajo'] = encuesta['trabajo'].str.strip()
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones4)
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta, 'trabajo')

NameError: name 'mapeo_correcciones4' is not defined

In [685]:
encuesta['trabajo']

0                                            ninguna
1                           no trabaja/ trabajaba_21
2                                            biologa
3                                  vendia avon y oro
4                           no trabaja/ trabajaba_21
                           ...                      
699                         no trabaja/ trabajaba_21
700    vendedor por cuenta propia (negocio propio)_1
701                         no trabaja/ trabajaba_21
702                          trabajadores domésticos
703                                      arrendadora
Name: trabajo, Length: 704, dtype: string

### Modificar algunos sinónimos

In [686]:
encuesta.iloc[95, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[21, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[207, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[3, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[120, encuesta.columns.get_loc('trabajo')] ='vendedor de comida'
encuesta.iloc[152, encuesta.columns.get_loc('trabajo')] ='vendedor'
encuesta.iloc[404, encuesta.columns.get_loc('trabajo')] ='abarrotes'
encuesta.iloc[430, encuesta.columns.get_loc('trabajo')] ='sastre'
encuesta.iloc[131, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'
encuesta.iloc[125, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'
encuesta.iloc[135, encuesta.columns.get_loc('trabajo')] ='ingeniero'
encuesta.iloc[15, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'
encuesta.iloc[356, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'
encuesta.iloc[546, encuesta.columns.get_loc('trabajo')] ='administrador'
encuesta.iloc[2, encuesta.columns.get_loc('trabajo')] = 'biólogo'
encuesta.iloc[567, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[495, encuesta.columns.get_loc('trabajo')] ='periódicos flores y productos en un puesto no establecido'
encuesta.iloc[292, encuesta.columns.get_loc('trabajo')] ='trabajadores domésticos'

In [687]:
encuesta['trabajo']

0                                            ninguna
1                           no trabaja/ trabajaba_21
2                                            biólogo
3                                           vendedor
4                           no trabaja/ trabajaba_21
                           ...                      
699                         no trabaja/ trabajaba_21
700    vendedor por cuenta propia (negocio propio)_1
701                         no trabaja/ trabajaba_21
702                          trabajadores domésticos
703                                      arrendadora
Name: trabajo, Length: 704, dtype: string

### Creación de Categorías

In [688]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion vendedor de comida_1 es desconocida
La ocupacion no se es desconocida
La ocupacion empleado_1 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion técnico en educación (educador o guardería)_3 es desconocida
La ocupacion dueño_1 es desconocida
La ocupacion director o gerente_2 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion limpieza es desconocida
La ocupacion educación básica_2 es desconocida
La ocupacion director o gerente_2 es desconocida
La ocupacion abogado_1 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion no trabaja/ trabajaba_21 es desconocida
La ocupacion psicó

In [689]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0    actividades elementales y de apoyo_20
1                         administrativo_9
2                                artista_2
3              comerciante sector formal_5
4            comerciante sector informal_4
5                           desconocido_99
6                empleado gubernamental_11
7                               oficios_21
8           profesionista independiente_14
dtype: string

In [690]:
categorias.value_counts()

desconocido_99                           575
actividades elementales y de apoyo_20    101
administrativo_9                          14
comerciante sector informal_4              6
profesionista independiente_14             2
comerciante sector formal_5                2
oficios_21                                 2
artista_2                                  1
empleado gubernamental_11                  1
Name: count, dtype: Int64

In [691]:
encuesta['categoria'] = categorias
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,no trabaja/ trabajaba_21,desconocido_99
2,biólogo,profesionista independiente_14
3,vendedor,comerciante sector informal_4
4,no trabaja/ trabajaba_21,desconocido_99
...,...,...
699,no trabaja/ trabajaba_21,desconocido_99
700,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,no trabaja/ trabajaba_21,desconocido_99
702,trabajadores domésticos,actividades elementales y de apoyo_20


### Asignación de la columna "trabajas[AbuelaPaterna]"

In [692]:
encuesta['Trabajas[AbuelaPaterna]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] == 'no trabaja/ trabajaba':
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbuelaPaterna]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbuelaPaterna]'] = 'Y'

In [693]:
#Esta función nos ayudará a desplazar la columna "Trabajas"
column_order = list(encuesta.columns)

# Reordenar la lista de columnas para que 'Trabajas[Madre]' esté antes de 'Trabajas'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[AbuelaPaterna]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [694]:
encuesta[['Trabajas[AbuelaPaterna]','trabajo', 'categoria']]

,Trabajas[AbuelaPaterna],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,no trabaja/ trabajaba_21,desconocido_99
2,Y,biólogo,profesionista independiente_14
3,Y,vendedor,comerciante sector informal_4
4,Y,no trabaja/ trabajaba_21,desconocido_99
...,...,...,...
699,Y,no trabaja/ trabajaba_21,desconocido_99
700,Y,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,Y,no trabaja/ trabajaba_21,desconocido_99
702,Y,trabajadores domésticos,actividades elementales y de apoyo_20


In [695]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/ trabajaba', 'ninguna')

### Últimas modificaciones sobre categorías


In [696]:
encuesta.iloc[703, encuesta.columns.get_loc('trabajo')] ='arrendador'
encuesta.iloc[173, encuesta.columns.get_loc('trabajo')] ='pedagogo'
encuesta.iloc[418, encuesta.columns.get_loc('trabajo')] ='campesino'
encuesta.iloc[167, encuesta.columns.get_loc('trabajo')] ='cosmetología'
encuesta.iloc[369, encuesta.columns.get_loc('trabajo')] ='cuidado geriátrico'

In [697]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,no trabaja/ trabajaba_21,desconocido_99
2,biólogo,profesionista independiente_14
3,vendedor,comerciante sector informal_4
4,no trabaja/ trabajaba_21,desconocido_99
...,...,...
699,no trabaja/ trabajaba_21,desconocido_99
700,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,no trabaja/ trabajaba_21,desconocido_99
702,trabajadores domésticos,actividades elementales y de apoyo_20


In [698]:
ocupaciones_profin= ["economista","actuario","bienes raíces","financiero","psiquiatra","programador","psiquiatra","traductora","","periodista","química"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"

In [699]:
ocupaciones_empre = ["director o gerente","negocios","arrendador","prestamista"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empre), 'categoria'] = "empresario_6"

In [700]:
ocupaciones_cliente= ["bienes raíces", "supermercado","aerolínea"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"

In [701]:
ocupaciones_especificas = ["servicio social","pedagogo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_especificas), 'categoria'] = "área educativa_13"

In [702]:
ocupaciones_formal = ["comerciante sector formal"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal","sobrerruedas"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [703]:
ocupaciones_seg = ["custodia"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_seg), 'categoria'] = "seguridad_15"

In [704]:
ocupaciones_cam= ["campesino","agraria"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cam), 'categoria'] = "agricultura_19"

In [705]:
ocupaciones_guber = ["hacienda"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "empleado gubernamental_11"

In [706]:
ocupaciones_saludd= ["cosmetología","cuidado geriátrico"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"

In [707]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           559
actividades elementales y de apoyo_20    101
administrativo_9                          14
comerciante sector informal_4              7
profesionista independiente_14             6
área educativa_13                          2
comerciante sector formal_5                2
atención a clientes_8                      2
área de la salud_16                        2
oficios_21                                 2
empresario_6                               2
empleado gubernamental_11                  2
artista_2                                  1
seguridad_15                               1
agricultura_19                             1
Name: count, dtype: Int64

In [708]:
encuesta['trabajo'].value_counts()

trabajo
no trabaja/ trabajaba_21                         311
trabajadores domésticos                           99
ninguna                                           56
vendedor por cuenta propia (negocio propio)_1     24
vendedor de comida_1                              24
                                                ... 
hacienda                                           1
ropa                                               1
atencion en tienda de abarrotes                    1
cosmetología                                       1
arrendador                                         1
Name: count, Length: 82, dtype: Int64

In [709]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,no trabaja/ trabajaba_21,desconocido_99
2,biólogo,profesionista independiente_14
3,vendedor,comerciante sector informal_4
4,no trabaja/ trabajaba_21,desconocido_99
...,...,...
699,no trabaja/ trabajaba_21,desconocido_99
700,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,no trabaja/ trabajaba_21,desconocido_99
702,trabajadores domésticos,actividades elementales y de apoyo_20


### Resultado Final

In [710]:
mapeo_codigos_categoria4 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria4[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria4

{'desconocido_99': 'A99',
 'profesionista independiente_14': 'A14',
 'comerciante sector informal_4': 'A04',
 'actividades elementales y de apoyo_20': 'A20',
 'administrativo_9': 'A09',
 'artista_2': 'A02',
 'comerciante sector formal_5': 'A05',
 'oficios_21': 'A21',
 'empleado gubernamental_11': 'A11'}

In [711]:
mapeo_codigos_categoria4['seguridad_15'] = 'A15'

mapeo_codigos_categoria4

{'desconocido_99': 'A99',
 'profesionista independiente_14': 'A14',
 'comerciante sector informal_4': 'A04',
 'actividades elementales y de apoyo_20': 'A20',
 'administrativo_9': 'A09',
 'artista_2': 'A02',
 'comerciante sector formal_5': 'A05',
 'oficios_21': 'A21',
 'empleado gubernamental_11': 'A11',
 'seguridad_15': 'A15'}

In [712]:
encuesta['categoria'].replace(mapeo_codigos_categoria4, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

0                       A02
1                       A04
2                       A05
3                       A09
4                       A11
5                       A14
6                       A15
7                       A20
8                       A21
9                       A99
10           agricultura_19
11    atención a clientes_8
12             empresario_6
13      área de la salud_16
14        área educativa_13
Name: categoria, dtype: string

In [713]:
mapeo_codigos_ocupaciones4 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria4[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones4[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones4

{'abogado_1': 'A9901',
 'administrador_2': 'A9902',
 'administrador_5': 'A9903',
 'agricultor_1': 'A9904',
 'asistente en consultorio': 'A9905',
 'atencion en tienda de abarrotes': 'A9906',
 'atención a clientes_1': 'A9907',
 'bailarina ballet': 'A9908',
 'cocinera': 'A9909',
 'comerciante sector formal_4': 'A9910',
 'comerciante sector informal_5': 'A9911',
 'comida y ropa': 'A9912',
 'contador_3': 'A9913',
 'costura': 'A9914',
 'costurera': 'A9915',
 'desconoce': 'A9916',
 'desconocido': 'A9917',
 'director o gerente_2': 'A9918',
 'dueño_1': 'A9919',
 'educación básica_2': 'A9920',
 'empleado en una tienda de ropa': 'A9921',
 'empleado_1': 'A9922',
 'enfermera_2': 'A9923',
 'flores y productos en un puesto no establecido_3': 'A9924',
 'granjero_2': 'A9925',
 'investigador-profesor en universidad/preparatoria_1': 'A9926',
 'limpieza': 'A9927',
 'mesera': 'A9928',
 'médico_1': 'A9929',
 'ninguna': 'A9930',
 'no aplica': 'A9931',
 'no sabe': 'A9932',
 'no se': 'A9933',
 'no sé': 'A9934'

In [714]:
mapeo_codigos_ocupaciones4["custodia"] = "A1501"

In [715]:
encuesta['trabajo'].replace(mapeo_reorganizado_importado,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

0                                             A0207
1                                             A0411
2                                             A0413
3                                             A0415
4                                             A0502
                          ...                      
74    técnico en educación (educador o guardería)_3
75                             vendedor de comida_1
76    vendedor por cuenta propia (negocio propio)_1
77                                        vendedora
78                                    venta de ropa
Name: trabajo, Length: 79, dtype: string

In [716]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[AbuelaPaterna]', 'categoria': 'Categoria[AbuelaPaterna]'})

In [717]:
encuesta[['Trabajas[AbuelaPaterna]', 'Trabajo[AbuelaPaterna]', 'Categoria[AbuelaPaterna]']]


,Trabajas[AbuelaPaterna],Trabajo[AbuelaPaterna],Categoria[AbuelaPaterna]
0,Y,A9921,A99
1,Y,no trabaja/ trabajaba_21,A99
2,Y,A1403,A14
3,Y,A0415,A04
4,Y,no trabaja/ trabajaba_21,A99
...,...,...,...
699,Y,no trabaja/ trabajaba_21,A99
700,Y,vendedor por cuenta propia (negocio propio)_1,A99
701,Y,no trabaja/ trabajaba_21,A99
702,Y,A2018,A20


In [718]:
#import json

#trabajos = { ... }

#with open("trabajos(abuelapaterna).json", "w") as archivo:
 #   json.dump(mapeo_codigos_ocupaciones4, archivo)

# Trabajos Abuelo Paterno

## Limpieza de nivel 1

In [719]:
mostrar_unicos(encuesta,"q0r800")

0     -oth-
1        10
2        11
3        12
4        15
5        16
6        17
7        18
8        19
9         2
10       20
11       21
12       22
13       23
14       24
15        3
16        4
17        5
18        6
19        7
20        9
21      NaN
Name: q0r800, dtype: object

In [720]:
mapeo_respuestas5 = {
    '2': "Actividades deportivas_1",
    '3': "Artista_2",
    '4': "Chofer_3",
    '5': "Comerciante sector formal_4",
    '6': "Comerciante sector informal_5",
    "7": "Empresario_6",
    "8": "Comercio Virtual_7",
    "9": "Atención a clientes_8",
    "10": "Administrativo_9",
    "11": "Gubernamental administrativo_10",
    "12": "Empleado gubernamental_11",
    "15": "Operador de maquinaria móvil_12",
    "16": "Personal técnico_13",
    "17": "Área educativa_14",
    "18": "Profesionista independiente_15",
    "19": "Seguridad_16",
    "20": "Área de la salud_17",
    "21": "Construcción_18",
    "22": "Fábrica_19",
    "23": "Agricultura_20",
    "24": "No trabaja/trabajaba_21"
}


In [721]:
llenar_respuesta_libre(encuesta, "q0r800", identificador_otro='-oth-', sufijo='[other]')
llenar_vacios(encuesta, "q0r800", 'ninguna')

In [722]:
encuesta['q0r800'] = encuesta['q0r800'].str.strip().str.lower()

In [723]:
encuesta['q0r800'] = encuesta['q0r800'].replace(mapeo_respuestas5)
mostrar_unicos(encuesta, "q0r800")

0     Actividades deportivas_1
1             Administrativo_9
2               Agricultura_20
3                    Artista_2
4        Atención a clientes_8
                ...           
79           vendedor de casas
80         vendedor de seguros
81                    zapatero
82         Área de la salud_17
83           Área educativa_14
Name: q0r800, Length: 84, dtype: object

## Limpieza de Nivel 2

### Columnas base

In [724]:
columnas_base5 = [
    "DEPORTS",
    "ARTISTA9",
    "CHOFER2",
    "FORMAL3",
    "INFOR45",
    "EMPRESA2",
    "VIRTUAL21",
    "ATNCIONCLINTS",
    "ADMINISTRACION",
    "EMPLE3ADOGUB",
    "GUB3RNAM",
    "MAQU1NAR1A",
    "T3NCIC0",
    "EDUCAT1V0",
    "IND3P3NDI3NT",
    "S3GURID2D",
    "S2LUD",
    "C0NSTRUCC10N",
    "FABRUIK",
    "AGRICULTUR3"
]
columnas_base5

['DEPORTS',
 'ARTISTA9',
 'CHOFER2',
 'FORMAL3',
 'INFOR45',
 'EMPRESA2',
 'VIRTUAL21',
 'ATNCIONCLINTS',
 'ADMINISTRACION',
 'EMPLE3ADOGUB',
 'GUB3RNAM',
 'MAQU1NAR1A',
 'T3NCIC0',
 'EDUCAT1V0',
 'IND3P3NDI3NT',
 'S3GURID2D',
 'S2LUD',
 'C0NSTRUCC10N',
 'FABRUIK',
 'AGRICULTUR3']

In [725]:
len(columnas_base5)

20

In [726]:
encuesta[columnas_base5]

,DEPORTS,ARTISTA9,CHOFER2,FORMAL3,INFOR45,EMPRESA2,VIRTUAL21,ATNCIONCLINTS,ADMINISTRACION,EMPLE3ADOGUB,GUB3RNAM,MAQU1NAR1A,T3NCIC0,EDUCAT1V0,IND3P3NDI3NT,S3GURID2D,S2LUD,C0NSTRUCC10N,FABRUIK,AGRICULTUR3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A2,NaN
702,NaN,NaN,A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [727]:
encuesta['ARTISTA9'] = encuesta['ARTISTA9'].replace({'A2': 'Fotógrafo_1', 'A3': 'Pintor_2','A4': 'Músico_3', 'A5':'Cantante_4'})
encuesta['CHOFER2'] = encuesta['CHOFER2'].replace({'A2': 'Taxista_1', 'A3': 'Uber_2','A4': 'Microbús_3', 'A5':'Camiones_4'})

encuesta['INFOR45'] = encuesta['INFOR45'].replace({'A2': 'Vendedor de comida_1','A3':' Periódicos_2','A4':'Flores y productos en un puesto no establecido_3','A5':'Productos en un puesto no establecido'})

encuesta['FORMAL3'] = encuesta['FORMAL3'].replace({'A2': 'Vendedor por cuenta propia (negocio propio)_1'})

encuesta['EMPRESA2'] = encuesta['EMPRESA2'].replace({'A2': 'Dueño_1','A3':'Director o gerente_2'})
encuesta['ATNCIONCLINTS'] = encuesta['ATNCIONCLINTS'].replace({'A5': 'Promotor de seguros_4','A4':'Vendedor de tiendas departamentales_3','A2':'Atención a clientes_1','A3':'Recepcionista_2', 'A6':'Telefonía_5'})
encuesta['ADMINISTRACION'] = encuesta['ADMINISTRACION'].replace({'A2': 'Contador público_1','A3':'Administrador_2','A4':'Recursos humanos_3'})

encuesta['GUB3RNAM'] = encuesta['GUB3RNAM'].replace({'A2': 'Jefe de departamento_1','A3':'Jefe de área, subdirector_2','A4':'Subadministrador_3','A5':' Director_4','A6':'Administrador_5','A7':'Director general_6'})


In [728]:
encuesta['EDUCAT1V0'] = encuesta['EDUCAT1V0'].replace(
    {'A2': 'Investigador-profesor en universidad/preparatoria_1', 'A3': 'Educación básica_2',
     'A4': 'Técnico en educación (educador o guardería)_3'})
encuesta['IND3P3NDI3NT'] = encuesta['IND3P3NDI3NT'].replace(
    {'A2': 'Abogado_1', 'A3': 'Psicólogo_2', 'A4': 'Contador_3', 'A5': 'Dentista_4', 'A6': 'Médico independiente_5',
     'A7': ' Terapeuta_6'})
encuesta['S3GURID2D'] = encuesta['S3GURID2D'].replace(
    {'A2': 'Guardaespaldas_1', 'A3': 'Patrullero_2', 'A4': 'Vigilante_3', 'A5': 'Empleado de seguridad en bancos_4',
     'A6': 'Empleado de seguridad privada_5', 'A7': 'Policía de tránsito_6'})
encuesta['S2LUD'] = encuesta['S2LUD'].replace(
    {'A2': 'Médico_1', 'A3': 'Enfermera_2', 'A4': 'Camillero_3', 'A5': 'Paramédico_4'})
encuesta['C0NSTRUCC10N'] = encuesta['C0NSTRUCC10N'].replace(
    {'A2': 'Arquitecto_1', 'A3': 'Ingeniero civil/albañil o trabajador general_2'})
encuesta['FABRUIK'] = encuesta['FABRUIK'].replace({'A2': 'Empleado_1'})

encuesta['AGRICULTUR3'] = encuesta['AGRICULTUR3'].replace({'A2':'Agricultor_1','A3':'Granjero_2','A4':'Siembra_3'})


In [729]:
encuesta['T3NCIC0'] = encuesta['T3NCIC0[comment]']
encuesta['EMPLE3ADOGUB']=encuesta['EMPLE3ADOGUB[comment]']
encuesta['MAQU1NAR1A']=encuesta['MAQU1NAR1A[comment]']

In [730]:
encuesta[columnas_base5]

,DEPORTS,ARTISTA9,CHOFER2,FORMAL3,INFOR45,EMPRESA2,VIRTUAL21,ATNCIONCLINTS,ADMINISTRACION,EMPLE3ADOGUB,GUB3RNAM,MAQU1NAR1A,T3NCIC0,EDUCAT1V0,IND3P3NDI3NT,S3GURID2D,S2LUD,C0NSTRUCC10N,FABRUIK,AGRICULTUR3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investigador-profesor en universidad/preparato...,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Médico_1,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agricultor_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,-oth-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,Vendedor por cuenta propia (negocio propio)_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
702,NaN,NaN,Taxista_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Columna con sufijos

In [731]:
columnas_sufijo5 = []
for col in columnas_base5:
    if col + '[other]' in encuesta.columns:
        columnas_sufijo5.append(col + '[other]')
    elif col + '[comment]' in encuesta.columns:
        columnas_sufijo5.append(col + '[comment]')
assert len(columnas_sufijo5) == len(columnas_base3)
columnas_sufijo5.sort()
columnas_sufijo5

['ADMINISTRACION[other]',
 'AGRICULTUR3[other]',
 'ARTISTA9[other]',
 'ATNCIONCLINTS[other]',
 'C0NSTRUCC10N[other]',
 'CHOFER2[other]',
 'DEPORTS[other]',
 'EDUCAT1V0[other]',
 'EMPLE3ADOGUB[comment]',
 'EMPRESA2[other]',
 'FABRUIK[other]',
 'FORMAL3[other]',
 'GUB3RNAM[other]',
 'IND3P3NDI3NT[other]',
 'INFOR45[other]',
 'MAQU1NAR1A[comment]',
 'S2LUD[other]',
 'S3GURID2D[other]',
 'T3NCIC0[comment]',
 'VIRTUAL21[other]']

In [732]:
for col_con_sufijo in columnas_sufijo5:
    columna_sin_sufijo5 = re.sub(r'\[\w+\]', '', col_con_sufijo)
    encuesta[columna_sin_sufijo5] = encuesta[columna_sin_sufijo5].combine_first(encuesta[col_con_sufijo])
    encuesta[columna_sin_sufijo5] = np.where(encuesta[columna_sin_sufijo5] == '-oth-',
                                             encuesta[col_con_sufijo],
                                             encuesta[columna_sin_sufijo5])
    encuesta.drop(columns=col_con_sufijo, inplace=True)

In [733]:
encuesta[columnas_base5]

,DEPORTS,ARTISTA9,CHOFER2,FORMAL3,INFOR45,EMPRESA2,VIRTUAL21,ATNCIONCLINTS,ADMINISTRACION,EMPLE3ADOGUB,GUB3RNAM,MAQU1NAR1A,T3NCIC0,EDUCAT1V0,IND3P3NDI3NT,S3GURID2D,S2LUD,C0NSTRUCC10N,FABRUIK,AGRICULTUR3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Investigador-profesor en universidad/preparato...,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Médico_1,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agricultor_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,CERAMICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,Vendedor por cuenta propia (negocio propio)_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Empleado_1,NaN
702,NaN,NaN,Taxista_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asignación de trabajo

In [734]:
encuesta['trabajo'] = pd.Series(dtype='string', index=encuesta.index)  # columna nueva
for columna in columnas_base5:
    encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta[columna])
    encuesta.drop(columns=columna, inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].combine_first(encuesta['q0r800'])
encuesta.drop(columns='q0r800', inplace=True)
encuesta['trabajo'] = encuesta['trabajo'].str.strip().str.lower()
mostrar_unicos(encuesta,'trabajo')

0                abarrotes
1                abogado_1
2          administrador_2
3          administrador_5
4           administrativo
              ...         
189               verduras
190            veterinario
191            vigilante_3
192               zapatero
193    área de la salud_17
Name: trabajo, Length: 194, dtype: string

In [735]:
trabajos_nuevos5 = encuesta['trabajo'][~encuesta['trabajo'].str.contains('_\d+$|ninguna')].reset_index(drop=True)
trabajos_nuevos5

0                                 carpintero
1                                 no se sabe
2                                      no se
3                                 supervisor
4                                     omitir
                       ...                  
255    productos en un puesto no establecido
256                        ingeniero tecnico
257                      trabajador y músico
258                                 ceramica
259                               arrendador
Name: trabajo, Length: 260, dtype: string

In [736]:
encuesta['trabajo']

0                                                ninguna
1      investigador-profesor en universidad/preparato...
2                                               médico_1
3                                             carpintero
4                                           agricultor_1
                             ...                        
699                                             ceramica
700        vendedor por cuenta propia (negocio propio)_1
701                                           empleado_1
702                                            taxista_1
703                                           arrendador
Name: trabajo, Length: 704, dtype: string

### Correción de trabajos

In [737]:
correccion5 = match_most_similar(trabajos_oficiales,
                                 trabajos_nuevos5,
                                 min_similarity=.40,
                                 ignore_index=True)
comparar5 = pd.DataFrame({'original': trabajos_nuevos5, 'correccion': correccion5})
comparar5['hubo_correccion'] = comparar5['original'].ne(comparar5['correccion'])
comparar5.sort_values(by='hubo_correccion', ascending=False)

TypeError: __new__() got an unexpected keyword argument 'ignore_index'

In [738]:
mapeo_correcciones5 = dict(zip(trabajos_nuevos5, correccion5))
mapeo_correcciones5

NameError: name 'correccion5' is not defined

In [739]:
encuesta['trabajo'] = encuesta['trabajo'].str.lower()
encuesta['trabajo'] = encuesta['trabajo'].str.strip()
encuesta['trabajo'] = encuesta['trabajo'].replace(mapeo_correcciones5)
encuesta['trabajo'] = encuesta['trabajo'].str.replace(r'_\d+', '', regex=True)
mostrar_unicos(encuesta, 'trabajo')

NameError: name 'mapeo_correcciones5' is not defined

In [740]:
encuesta['trabajo']

0                                                ninguna
1      investigador-profesor en universidad/preparato...
2                                               médico_1
3                                             carpintero
4                                           agricultor_1
                             ...                        
699                                             ceramica
700        vendedor por cuenta propia (negocio propio)_1
701                                           empleado_1
702                                            taxista_1
703                                           arrendador
Name: trabajo, Length: 704, dtype: string

### Modificar algunos sinónimos

In [741]:
encuesta.iloc[194, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[285, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[172, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[646, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[260, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[568, encuesta.columns.get_loc('trabajo')] ='mecánico'
encuesta.iloc[580, encuesta.columns.get_loc('trabajo')] ='mecánico'

encuesta.iloc[556, encuesta.columns.get_loc('trabajo')] ='policía'
encuesta.iloc[359, encuesta.columns.get_loc('trabajo')] ='negocio'
encuesta.iloc[300, encuesta.columns.get_loc('trabajo')] ='plomero'
encuesta.iloc[58, encuesta.columns.get_loc('trabajo')] = 'plomero'
encuesta.iloc[234, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[192, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[128, encuesta.columns.get_loc('trabajo')] ='trabajadores de limpieza'
encuesta.iloc[342, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[324, encuesta.columns.get_loc('trabajo')] = 'policía'
encuesta.iloc[394, encuesta.columns.get_loc('trabajo')] = 'chofer'
encuesta.iloc[284, encuesta.columns.get_loc('trabajo')] = 'militar'
encuesta.iloc[613, encuesta.columns.get_loc('trabajo')] = 'militar'
encuesta.iloc[104, encuesta.columns.get_loc('trabajo')] = 'empleado'
encuesta.iloc[335, encuesta.columns.get_loc('trabajo')] = 'ingeniero'
encuesta.iloc[171, encuesta.columns.get_loc('trabajo')] = 'historiador'
encuesta.iloc[524, encuesta.columns.get_loc('trabajo')] = 'ganadero'
encuesta.iloc[490, encuesta.columns.get_loc('trabajo')] = 'comerciante sector informal'
encuesta.iloc[425, encuesta.columns.get_loc('trabajo')] = 'chofer'
encuesta.iloc[120, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[584, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[662, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[529, encuesta.columns.get_loc('trabajo')] = 'mecánico'
encuesta.iloc[59, encuesta.columns.get_loc('trabajo')] = 'técnico en barcos de cargo'
encuesta.iloc[82, encuesta.columns.get_loc('trabajo')] = 'técnico radiólogo'

In [742]:
encuesta['trabajo']

0                                                ninguna
1      investigador-profesor en universidad/preparato...
2                                               médico_1
3                                             carpintero
4                                           agricultor_1
                             ...                        
699                                             ceramica
700        vendedor por cuenta propia (negocio propio)_1
701                                           empleado_1
702                                            taxista_1
703                                           arrendador
Name: trabajo, Length: 704, dtype: string

### Creación de Categorías

In [743]:
categorias = pd.Series(dtype='string', index=encuesta.index)
for i, ocupacion in encuesta['trabajo'][encuesta['trabajo'].notna()].items():
    registrada = False
    if ocupacion == 'ninguna':
        categorias.loc[i] = 'desconocido_99'
        registrada = True
    else:
        for categoria, ocupaciones_dentro_de_categoria in codigos_ocupaciones.items():
            if ocupacion in ocupaciones_dentro_de_categoria:
                categorias.loc[i] = categoria
                registrada = True
                break
        if not registrada:
            print(f'La ocupacion {ocupacion} es desconocida')
            categorias.loc[i] = 'desconocido_99'

La ocupacion investigador-profesor en universidad/preparatoria_1 es desconocida
La ocupacion médico_1 es desconocida
La ocupacion agricultor_1 es desconocida
La ocupacion no se sabe es desconocida
La ocupacion granjero_2 es desconocida
La ocupacion siembra_3 es desconocida
La ocupacion empleado_1 es desconocida
La ocupacion dueño_1 es desconocida
La ocupacion arquitecto_1 es desconocida
La ocupacion no se es desconocida
La ocupacion vendedor de comida_1 es desconocida
La ocupacion ingeniero civil/albañil o trabajador general_2 es desconocida
La ocupacion taxista_1 es desconocida
La ocupacion contador_3 es desconocida
La ocupacion supervisor es desconocida
La ocupacion vendedor por cuenta propia (negocio propio)_1 es desconocida
La ocupacion omitir es desconocida
La ocupacion abogado_1 es desconocida
La ocupacion dueño_1 es desconocida
La ocupacion vendedor por cuenta propia (negocio propio)_1 es desconocida
La ocupacion jefe de departamento_1 es desconocida
La ocupacion no trabaja/trab

In [744]:
categorias.drop_duplicates().sort_values().reset_index(drop=True)

0     actividades elementales y de apoyo_20
1                                 artista_2
2               comerciante sector formal_5
3             comerciante sector informal_4
4                           construcción_17
5                            desconocido_99
6                              empresario_6
7                                fábrica_18
8                                oficios_21
9            profesionista independiente_14
10                             seguridad_15
dtype: string

In [745]:
categorias.value_counts()

desconocido_99                           639
oficios_21                                21
comerciante sector formal_5               10
actividades elementales y de apoyo_20      9
profesionista independiente_14             8
fábrica_18                                 6
construcción_17                            4
empresario_6                               2
seguridad_15                               2
artista_2                                  2
comerciante sector informal_4              1
Name: count, dtype: Int64

In [746]:
encuesta['categoria'] = categorias
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,investigador-profesor en universidad/preparato...,desconocido_99
2,médico_1,desconocido_99
3,carpintero,oficios_21
4,agricultor_1,desconocido_99
...,...,...
699,ceramica,desconocido_99
700,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,empleado_1,desconocido_99
702,taxista_1,desconocido_99


### Asignación de la columna "Trabajas[AbueloPaterno]"
Esta columna se crea para corregir los datos de zoom 2, donde no se realiza la pregunta: ¿En qué trabajas".La creación de esta columna nos ayudará a poder limpiar aquellos trabajos que esten nombrados como: "no trabaja/trabajaba".

In [747]:
encuesta['Trabajas[AbueloPaterno]'] = ''

# Recorremos cada fila del dataframe
for index, row in encuesta.iterrows():
    # Verificamos la condición en la columna 'trabajo' de cada fila
    if row['trabajo'] == 'no trabaja/trabajaba':
        # Si la condición es verdadera, establecemos el valor 'N' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbueloPaterno]'] = 'N'
    else:
        # Si la condición es falsa, establecemos el valor 'Y' en la columna 'Trabajas'
        encuesta.at[index, 'Trabajas[AbueloPaterno]'] = 'Y'

In [748]:
#Esta función nos ayudará a desplazar la columna "Trabajas"
column_order = list(encuesta.columns)

# Reordenar la lista de columnas para que 'Trabajas[Madre]' esté antes de 'Trabajas'
column_order.insert(column_order.index('trabajo'), column_order.pop(column_order.index('Trabajas[AbueloPaterno]')))

# Reasignar el orden de las columnas en el dataframe
encuesta = encuesta[column_order]

In [749]:
 encuesta[['Trabajas[AbueloPaterno]','trabajo', 'categoria']]

,Trabajas[AbueloPaterno],trabajo,categoria
0,Y,ninguna,desconocido_99
1,Y,investigador-profesor en universidad/preparato...,desconocido_99
2,Y,médico_1,desconocido_99
3,Y,carpintero,oficios_21
4,Y,agricultor_1,desconocido_99
...,...,...,...
699,Y,ceramica,desconocido_99
700,Y,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,Y,empleado_1,desconocido_99
702,Y,taxista_1,desconocido_99


In [750]:
encuesta['trabajo'] = encuesta['trabajo'].str.strip().replace('no trabaja/trabajaba', 'ninguna')

### Últimas modificaciones sobre categorías


In [751]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,investigador-profesor en universidad/preparato...,desconocido_99
2,médico_1,desconocido_99
3,carpintero,oficios_21
4,agricultor_1,desconocido_99
...,...,...
699,ceramica,desconocido_99
700,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,empleado_1,desconocido_99
702,taxista_1,desconocido_99


In [752]:
encuesta.iloc[164, encuesta.columns.get_loc('trabajo')] = 'trabaja en pemex'
encuesta.iloc[385, encuesta.columns.get_loc('trabajo')] = 'tráiler'
encuesta.iloc[660, encuesta.columns.get_loc('trabajo')] = 'trabajador en ciner'
encuesta.iloc[570, encuesta.columns.get_loc('trabajo')] = 'podológo'
encuesta.iloc[492, encuesta.columns.get_loc('trabajo')] = 'ferrocarrilero'
encuesta.iloc[533, encuesta.columns.get_loc('trabajo')] = 'piloto'
encuesta.iloc[512, encuesta.columns.get_loc('trabajo')] = 'transportista'
encuesta.iloc[179, encuesta.columns.get_loc('trabajo')] = 'vendedor'
encuesta.iloc[204, encuesta.columns.get_loc('trabajo')] = 'chofer'
encuesta.iloc[221, encuesta.columns.get_loc('trabajo')] = 'chofer'
encuesta.iloc[30, encuesta.columns.get_loc('trabajo')] = 'fábrica'
encuesta.iloc[130, encuesta.columns.get_loc('trabajo')] = 'trabajadores de limpieza'
encuesta.iloc[91, encuesta.columns.get_loc('trabajo')] = 'terapeuta'
encuesta.iloc[330, encuesta.columns.get_loc('trabajo')] = 'vendedor de comida'
encuesta.iloc[385, encuesta.columns.get_loc('trabajo')] = 'tráiler'
encuesta.iloc[563, encuesta.columns.get_loc('trabajo')]= "ropa"
encuesta.iloc[316, encuesta.columns.get_loc('trabajo')]="educación básica"
encuesta.iloc[121, encuesta.columns.get_loc('trabajo')]="ninguna"
encuesta.iloc[133, encuesta.columns.get_loc('trabajo')]="anestesiólogo"
encuesta.iloc[158, encuesta.columns.get_loc('trabajo')]="luz y fuerza"
encuesta.iloc[367, encuesta.columns.get_loc('trabajo')]="vendedor de comida"
encuesta.iloc[176, encuesta.columns.get_loc('trabajo')]="mecánico"
encuesta.iloc[88, encuesta.columns.get_loc('trabajo')]="bolsa de valores"

In [753]:
encuesta.iloc[332, encuesta.columns.get_loc('trabajo')]="vendedor de comida"
encuesta.iloc[389, encuesta.columns.get_loc('trabajo')]="ninguna"
encuesta.iloc[660, encuesta.columns.get_loc('trabajo')]="trabajador en cine"
encuesta.iloc[386, encuesta.columns.get_loc('trabajo')]="agricultor"

In [754]:
ocupaciones_profin= ["historiador","quimico","periodista","veterinario","profesionista independiente","terapeuta","psiquiatra"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_profin), 'categoria'] = "profesionista independiente_14"
ocupaciones_especificas = ["laboratorista"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_especificas), 'categoria'] = "área educativa_13"

In [755]:
ocupaciones_of = ["ceramica","bolero","panadero","petrolero","plomero","joyero","mecánico","armero", "bombero","ganadero"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_of), 'categoria'] = "oficios_21"

In [756]:
ocupaciones_tec = ["técnico radiólogo","reparación aparatos electronicos","técnico en barcos de cargo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_tec), 'categoria'] = "personal técnico_12"

In [757]:
ocupaciones_dep = ["luchador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_dep), 'categoria'] = "actividades deportivas_1"

In [758]:
ocupaciones_saludd= ["optometrista","podológo","anestesiólogo","área de la salud","supervisor de salubridad"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_saludd), 'categoria'] = "área de la salud_16"

In [759]:
ocupaciones_guber = ["empleado gubernamental", 'trabaja en pemex', 'trabaja en delegación', "luz y fuerza","compañia de luz", "hacienda","jurisprudencia","presidente de la republica","diputado","bolsa de valores"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "empleado gubernamental_11"

ocupaciones_guber = ["gubernamental administrativo"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_guber), 'categoria'] = "gubernamental administrativo_10"

In [760]:
ocupaciones_chof = ["chofer","transportista","tráiler","transporte colectivo metro","mudanzas","maneja un camión de basura", "empresa de transporte","servicio de transporte", "piloto","ferrocarrilero","tranvia"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_chof), 'categoria'] = "chofer_3"

In [761]:
ocupaciones_empresario = ["director o gerente", "socio","empresario","negocio","arrendador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_empresario), 'categoria'] = "empresario_6"

In [762]:
ocupaciones_formal = ["comerciante sector formal","vendedor de comida"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_formal), 'categoria'] = "comerciante sector formal_5"
ocupaciones_informal = ["comerciante sector informal","purificadora","periódicos","vendedor"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_informal), 'categoria'] = "comerciante sector informal_4"

In [763]:
ocupaciones_fabrica=["obrero","fábrica","supervisor"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_fabrica), 'categoria'] = "fábrica_18"

In [764]:
ocupaciones_cons = ["contratista","ingeniero civil/albañil o trabajador general", "caminos y puentes"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cons), 'categoria'] = "construcción_17"

In [765]:
ocupaciones_seg = ["seguridad","militar"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_seg), 'categoria'] = "seguridad_15"

In [766]:
ocupaciones_limp= ["trabajadores de limpieza"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_limp), 'categoria'] = "actividades elementales y de apoyo_20"

In [767]:
ocupaciones_cam= ["agricultor"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cam), 'categoria'] = "agricultura_19"
ocupaciones_cliente= ["trabajador en cine","asegurador"]
encuesta.loc[encuesta['trabajo'].isin(ocupaciones_cliente), 'categoria'] = "atención a clientes_8"

In [768]:
encuesta['categoria'].value_counts()

categoria
desconocido_99                           567
oficios_21                                37
comerciante sector formal_5               13
profesionista independiente_14            13
actividades elementales y de apoyo_20     10
fábrica_18                                10
seguridad_15                               9
chofer_3                                   9
empleado gubernamental_11                  8
construcción_17                            5
empresario_6                               5
área de la salud_16                        4
comerciante sector informal_4              4
personal técnico_12                        3
artista_2                                  2
atención a clientes_8                      2
agricultura_19                             1
actividades deportivas_1                   1
área educativa_13                          1
Name: count, dtype: Int64

In [769]:
encuesta[['trabajo', 'categoria']]

,trabajo,categoria
0,ninguna,desconocido_99
1,investigador-profesor en universidad/preparato...,desconocido_99
2,médico_1,desconocido_99
3,carpintero,oficios_21
4,agricultor_1,desconocido_99
...,...,...
699,ceramica,oficios_21
700,vendedor por cuenta propia (negocio propio)_1,desconocido_99
701,empleado_1,desconocido_99
702,taxista_1,desconocido_99


In [770]:
encuesta['trabajo'].value_counts()

trabajo
ninguna                                           95
vendedor por cuenta propia (negocio propio)_1     45
no se                                             39
no trabaja/trabajaba_21                           31
ingeniero civil/albañil o trabajador general_2    30
                                                  ..
asegurador                                         1
capataz                                            1
reclutador de personal                             1
quimico                                            1
arrendador                                         1
Name: count, Length: 174, dtype: Int64

### Resultado final

In [771]:
mapeo_codigos_categoria5 = {}
for cat in categorias.drop_duplicates():
    numero = re.findall(r'_(\d+)', cat)[0]
    mapeo_codigos_categoria5[cat] = 'A' + str(numero).zfill(2)
mapeo_codigos_categoria5

{'desconocido_99': 'A99',
 'oficios_21': 'A21',
 'construcción_17': 'A17',
 'empresario_6': 'A06',
 'comerciante sector formal_5': 'A05',
 'fábrica_18': 'A18',
 'profesionista independiente_14': 'A14',
 'actividades elementales y de apoyo_20': 'A20',
 'seguridad_15': 'A15',
 'artista_2': 'A02',
 'comerciante sector informal_4': 'A04'}

In [772]:
mapeo_codigos_categoria5['personal técnico_12'] = 'A12'
mapeo_codigos_categoria5['actividades deportivas_1'] = 'A01'
mapeo_codigos_categoria5

{'desconocido_99': 'A99',
 'oficios_21': 'A21',
 'construcción_17': 'A17',
 'empresario_6': 'A06',
 'comerciante sector formal_5': 'A05',
 'fábrica_18': 'A18',
 'profesionista independiente_14': 'A14',
 'actividades elementales y de apoyo_20': 'A20',
 'seguridad_15': 'A15',
 'artista_2': 'A02',
 'comerciante sector informal_4': 'A04',
 'personal técnico_12': 'A12',
 'actividades deportivas_1': 'A01'}

In [773]:
encuesta['categoria'].replace(mapeo_codigos_categoria5, inplace=True)
encuesta['categoria'].drop_duplicates().sort_values().reset_index(drop=True)

0                           A01
1                           A02
2                           A04
3                           A05
4                           A06
5                           A12
6                           A14
7                           A15
8                           A17
9                           A18
10                          A20
11                          A21
12                          A99
13               agricultura_19
14        atención a clientes_8
15                     chofer_3
16    empleado gubernamental_11
17          área de la salud_16
18            área educativa_13
Name: categoria, dtype: string

In [774]:
mapeo_codigos_ocupaciones5 = {}
for cat in categorias.drop_duplicates():
    codigo_cat = mapeo_codigos_categoria5[cat]
    ocupaciones_en_categoria = encuesta['trabajo'][
        encuesta['categoria'] == codigo_cat].drop_duplicates().sort_values().reset_index(drop=True)
    for i, ocupacion in ocupaciones_en_categoria.items():
        mapeo_codigos_ocupaciones5[ocupacion] = codigo_cat + str(i + 1).zfill(2)
mapeo_codigos_ocupaciones5

{'abogado_1': 'A9901',
 'administrador_2': 'A9902',
 'administrador_5': 'A9903',
 'administrativo': 'A9904',
 'agricultor_1': 'A9905',
 'arquitecto_1': 'A9906',
 'atención a clientes_1': 'A9907',
 'ayudante en un establo': 'A9908',
 'banco': 'A9909',
 'camiones_4': 'A9910',
 'camioneta': 'A9911',
 'cantante_4': 'A9912',
 'capataz': 'A9913',
 'carpinteria': 'A9914',
 'cartero': 'A9915',
 'cerrajero': 'A9916',
 'cocina': 'A9917',
 'comerciante sector informal_5': 'A9918',
 'contador general': 'A9919',
 'contador público_1': 'A9920',
 'contador_3': 'A9921',
 'dentista_4': 'A9922',
 'desconoce': 'A9923',
 'desconocido': 'A9924',
 'director o gerente_2': 'A9925',
 'dueño_1': 'A9926',
 'educación básica': 'A9927',
 'educación básica_2': 'A9928',
 'electricista': 'A9929',
 'empleado de confianza': 'A9930',
 'empleado delegacion': 'A9931',
 'empleado gubernamental_11': 'A9932',
 'empleado_1': 'A9933',
 'empresa de camiones': 'A9934',
 'empresa productora de platos de loza': 'A9935',
 'flores y

In [775]:
mapeo_codigos_ocupaciones5["luchador"] = "A0101"
mapeo_codigos_ocupaciones5["reparación aparatos electronicos"] = "A1201"
mapeo_codigos_ocupaciones5["técnico en barcos de cargo"] = "A1202"
mapeo_codigos_ocupaciones5["técnico radiólogo"] = "A1203"
mapeo_codigos_ocupaciones5

{'abogado_1': 'A9901',
 'administrador_2': 'A9902',
 'administrador_5': 'A9903',
 'administrativo': 'A9904',
 'agricultor_1': 'A9905',
 'arquitecto_1': 'A9906',
 'atención a clientes_1': 'A9907',
 'ayudante en un establo': 'A9908',
 'banco': 'A9909',
 'camiones_4': 'A9910',
 'camioneta': 'A9911',
 'cantante_4': 'A9912',
 'capataz': 'A9913',
 'carpinteria': 'A9914',
 'cartero': 'A9915',
 'cerrajero': 'A9916',
 'cocina': 'A9917',
 'comerciante sector informal_5': 'A9918',
 'contador general': 'A9919',
 'contador público_1': 'A9920',
 'contador_3': 'A9921',
 'dentista_4': 'A9922',
 'desconoce': 'A9923',
 'desconocido': 'A9924',
 'director o gerente_2': 'A9925',
 'dueño_1': 'A9926',
 'educación básica': 'A9927',
 'educación básica_2': 'A9928',
 'electricista': 'A9929',
 'empleado de confianza': 'A9930',
 'empleado delegacion': 'A9931',
 'empleado gubernamental_11': 'A9932',
 'empleado_1': 'A9933',
 'empresa de camiones': 'A9934',
 'empresa productora de platos de loza': 'A9935',
 'flores y

In [776]:
encuesta['trabajo'].replace(mapeo_reorganizado_importado,inplace=True)
encuesta['trabajo'].drop_duplicates().sort_values().reset_index(drop=True)

0                                              A0101
1                                              A0205
2                                              A0209
3                                              A0302
4                                              A0304
                           ...                      
163                              vendedor de seguros
164                               vendedor en ferias
165    vendedor por cuenta propia (negocio propio)_1
166                                      vigilante_3
167                              área de la salud_17
Name: trabajo, Length: 168, dtype: string

In [777]:
encuesta = encuesta.rename(columns={'trabajo': 'Trabajo[AbueloPaterno]', 'categoria': 'Categoria[AbueloPaterno]'})
encuesta[['Trabajas[AbueloPaterno]', 'Trabajo[AbueloPaterno]', 'Categoria[AbueloPaterno]']]

,Trabajas[AbueloPaterno],Trabajo[AbueloPaterno],Categoria[AbueloPaterno]
0,Y,A9921,A99
1,Y,investigador-profesor en universidad/preparato...,A99
2,Y,médico_1,A99
3,Y,A2106,A21
4,Y,agricultor_1,A99
...,...,...,...
699,Y,A2107,A21
700,Y,vendedor por cuenta propia (negocio propio)_1,A99
701,Y,empleado_1,A99
702,Y,taxista_1,A99


In [778]:
#import json

#trabajos = { ... }

#with open("trabajos(abuelopaterno).json", "w") as archivo:
#    json.dump(mapeo_codigos_ocupaciones5, archivo)

# Corregir trabajos

In [779]:
mapeo_codigos_ocupaciones
mapeo_codigos_ocupaciones1
mapeo_codigos_ocupaciones2
mapeo_codigos_ocupaciones3
mapeo_codigos_ocupaciones4
mapeo_codigos_ocupaciones5


{'abogado_1': 'A9901',
 'administrador_2': 'A9902',
 'administrador_5': 'A9903',
 'administrativo': 'A9904',
 'agricultor_1': 'A9905',
 'arquitecto_1': 'A9906',
 'atención a clientes_1': 'A9907',
 'ayudante en un establo': 'A9908',
 'banco': 'A9909',
 'camiones_4': 'A9910',
 'camioneta': 'A9911',
 'cantante_4': 'A9912',
 'capataz': 'A9913',
 'carpinteria': 'A9914',
 'cartero': 'A9915',
 'cerrajero': 'A9916',
 'cocina': 'A9917',
 'comerciante sector informal_5': 'A9918',
 'contador general': 'A9919',
 'contador público_1': 'A9920',
 'contador_3': 'A9921',
 'dentista_4': 'A9922',
 'desconoce': 'A9923',
 'desconocido': 'A9924',
 'director o gerente_2': 'A9925',
 'dueño_1': 'A9926',
 'educación básica': 'A9927',
 'educación básica_2': 'A9928',
 'electricista': 'A9929',
 'empleado de confianza': 'A9930',
 'empleado delegacion': 'A9931',
 'empleado gubernamental_11': 'A9932',
 'empleado_1': 'A9933',
 'empresa de camiones': 'A9934',
 'empresa productora de platos de loza': 'A9935',
 'flores y

In [780]:
mapeo_unificado_ocupaciones = {
    **mapeo_codigos_ocupaciones,
    **mapeo_codigos_ocupaciones1,
    **mapeo_codigos_ocupaciones2,
    **mapeo_codigos_ocupaciones3,
    **mapeo_codigos_ocupaciones4,
    **mapeo_codigos_ocupaciones5
}

In [781]:
mapeo_unificado_ocupaciones

{'abogada': 'A9901',
 'administración': 'A9902',
 'administrativa': 'A9903',
 'agente de seguros': 'A9904',
 'agente telefonico en inova': 'A9905',
 'asistente del director de operaciones': 'A9906',
 'asistente documental': 'A9907',
 'asistente ejecutiva': 'A9908',
 'auxiliar administrativo': 'A9905',
 'clases de arte': 'A9910',
 'clases de danza': 'A9911',
 'coaching': 'A9912',
 'cocinera': 'A9909',
 'consultoria': 'A9914',
 'costura': 'A9914',
 'costurera': 'A9915',
 'departamento de ventas aereolinea': 'A9917',
 'desconocido': 'A9924',
 'directora': 'A9918',
 'directora general': 'A9920',
 'diseñadora gráfica': 'A9921',
 'em': 'A9922',
 'empleada': 'A9922',
 'empleada de limpieza': 'A9924',
 'empleada lavandería': 'A9925',
 'empleada tienda neto': 'A9926',
 'en farmacia': 'A9927',
 'estudiando': 'A9928',
 'estudiando otra carrera': 'A9929',
 'evaciones psicologicas': 'A9930',
 'farmacologia': 'A9931',
 'homeopatia': 'A9932',
 'ingeniera en sistemas y computacion': 'A9933',
 'ingenie

In [782]:
len(mapeo_unificado_ocupaciones)

577

In [783]:
mapeo_unificado = {k: v for d in [mapeo_codigos_ocupaciones, mapeo_codigos_ocupaciones1, mapeo_codigos_ocupaciones2, mapeo_codigos_ocupaciones3, mapeo_codigos_ocupaciones4, mapeo_codigos_ocupaciones5] for k, v in d.items()}

In [784]:
mapeo_unificado

{'abogada': 'A9901',
 'administración': 'A9902',
 'administrativa': 'A9903',
 'agente de seguros': 'A9904',
 'agente telefonico en inova': 'A9905',
 'asistente del director de operaciones': 'A9906',
 'asistente documental': 'A9907',
 'asistente ejecutiva': 'A9908',
 'auxiliar administrativo': 'A9905',
 'clases de arte': 'A9910',
 'clases de danza': 'A9911',
 'coaching': 'A9912',
 'cocinera': 'A9909',
 'consultoria': 'A9914',
 'costura': 'A9914',
 'costurera': 'A9915',
 'departamento de ventas aereolinea': 'A9917',
 'desconocido': 'A9924',
 'directora': 'A9918',
 'directora general': 'A9920',
 'diseñadora gráfica': 'A9921',
 'em': 'A9922',
 'empleada': 'A9922',
 'empleada de limpieza': 'A9924',
 'empleada lavandería': 'A9925',
 'empleada tienda neto': 'A9926',
 'en farmacia': 'A9927',
 'estudiando': 'A9928',
 'estudiando otra carrera': 'A9929',
 'evaciones psicologicas': 'A9930',
 'farmacologia': 'A9931',
 'homeopatia': 'A9932',
 'ingeniera en sistemas y computacion': 'A9933',
 'ingenie

In [785]:
# Paso 1: Agrupar por los tres primeros dígitos del código
from collections import defaultdict

# Crear un diccionario por grupos
grupos = defaultdict(list)

# Llenar el diccionario con los grupos
for nombre, codigo in mapeo_unificado.items():
    prefijo = codigo[:3]
    grupos[prefijo].append((nombre, codigo))

# Paso 2: Reordenar alfabéticamente y reasignar los códigos
mapeo_reorganizado = {}

for prefijo, items in grupos.items():
    # Ordenar los nombres alfabéticamente
    items_ordenados = sorted(items, key=lambda x: x[0])

    # Reasignar los códigos secuenciales
    for idx, (nombre, _) in enumerate(items_ordenados, start=1):
        nuevo_codigo = f"{prefijo}{idx:02d}"
        mapeo_reorganizado[nombre] = nuevo_codigo

In [786]:
#mapeo_reorganizado

In [787]:
#with open('mapeo_reorganizado.json', 'w', encoding='utf-8') as f:
#    json.dump(mapeo_reorganizado, f, ensure_ascii=False, indent=4)

In [788]:
#archivo_json = 'mapeo_reorganizado.json'

# Leer el archivo JSON y convertirlo en un diccionario
#with open(archivo_json, 'r', encoding='utf-8') as f:
#    mapeo_reorganizado_importado = json.load(f)

In [789]:
#mapeo_reorganizado_importado

In [790]:
#encuesta.to_csv('zoom2_limpio_Erick_oficial.csv', index=False)